### Newsbot to write a daily AI news summary using langgraph
- Save a list of HTML files from sources.yaml (tech news sites)
- Extract URLs for the news stories
- Filter URLs to remove duplicates, articles seen before, and non-AI articles (using a ChatGPT prompt)
- Perform headline topic analysis and sort by topic to help the AI structure the response by topic
- Scrape and summarize individual articles
- Compose and email the summary


In [1]:
# to selectively re-import as needed
# import sys
# del sys.modules['ainb_llm']
# del sys.modules['ainb_const']
# del sys.modules['ainb_utilities']
# del sys.modules['ainb_webscrape']


In [2]:
from datetime import datetime
import os
import yaml
import sqlite3
import json
from collections import Counter
import uuid
from typing import TypedDict, Annotated

import operator
import pickle

import langchain
from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.prompts import (ChatPromptTemplate, MessagesPlaceholder, PromptTemplate,
                                    SystemMessagePromptTemplate, HumanMessagePromptTemplate)
# from langchain.agents import create_tool_calling_agent, AgentExecutor
# from langchain.memory import ChatMessageHistory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
# from langchain.tools import BaseTool, StructuredTool, tool
# from langchain_community.chat_models import ChatOllama
# from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import SimpleJsonOutputParser, JsonOutputParser, StrOutputParser
# from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
# from langchain_core.outputs import LLMResult

from langgraph.graph import StateGraph, START, END

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt

import sklearn
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

import bs4
# from bs4 import BeautifulSoup
# import requests
from urllib.parse import urljoin, urlparse

import multiprocessing
# from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText

import openai
from openai import OpenAI

import trafilatura

VERBOSE=1
from ainb_const import (DOWNLOAD_DIR, PAGES_DIR,
                        MODEL, LOWCOST_MODEL, HIGHCOST_MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE, SQLITE_DB,
                        HOSTNAME_SKIPLIST, SITE_NAME_SKIPLIST,
                       )
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article,
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file,
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory,
                            process_url_queue_factory, get_google_news_redirects)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries,
                      fetch_openai_summary, count_tokens, trunc_tokens, 
                      categorize_headline, categorize_df, clean_topics,
                      get_site_name, fetch_missing_site_names)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


In [3]:
print(f"LangChain         {langchain.__version__}")
print(f"OpenAI            {openai.__version__}")
# print(f"smtplib           {smtplib.sys.version}")
print(f"trafilatura       {trafilatura.__version__}")
# print(f"bs4               {bs4.__version__}")
print(f"numpy             {np.__version__}")
print(f"pandas            {pd.__version__}")
print(f"sklearn           {sklearn.__version__}")
print(f"umap              {umap.__version__}")


LangChain         0.3.0
OpenAI            1.47.0
trafilatura       1.12.2
numpy             1.26.4
pandas            2.2.3
sklearn           1.5.2
umap              0.5.6


In [4]:
# a basic LLM call with langchain
model = ChatOpenAI(model=MODEL)

model.invoke([
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content='Listen to me. You are beautiful. You are perfect and I love you.'),
])


2024-09-25 08:13:03,768 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Ascoltami. Sei bellissima. Sei perfetta e ti amo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 34, 'total_tokens': 50, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'chatgpt-4o-latest', 'system_fingerprint': 'fp_61e551c4c2', 'finish_reason': 'stop', 'logprobs': None}, id='run-31f54c37-30b4-438d-9070-e9df837db9df-0', usage_metadata={'input_tokens': 34, 'output_tokens': 16, 'total_tokens': 50})

In [5]:
# use template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
parser = StrOutputParser()
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "hi"})


2024-09-25 08:13:04,616 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Ciao'

In [6]:
prompt_inputs = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

system_template = 'Translate the following into {language}:'
user_template = 'Listen to me. You are {adjective1}. You are {adjective2} and I love you.'

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

parser = StrOutputParser()

chain = prompt_template | model | parser

start_time = datetime.now()
for tpl in prompt_inputs:
    response = ""
    # stream tokens as they are generated
    for r in chain.stream(tpl):
        print(r, end="")
        response += r
end_time = datetime.now()

difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")


2024-09-25 08:13:04,993 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.

2024-09-25 08:13:05,813 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.

2024-09-25 08:13:06,838 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Escúchame. Eres perfecto/a. Eres hermoso/a y te amo.

2024-09-25 08:13:07,964 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Ascoltami. Sei fantastico/a. Sei magnifico/a e ti amo.

2024-09-25 08:13:09,639 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Figyelj rám! Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.

Elapsed seconds: 5.650129


In [7]:
# same but send all at once using asyncio

async def async_langchain(chain, input_dict):
    response = await chain.ainvoke(input_dict)
    return response


prompt_templates = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

start_time = datetime.now()
tasks = []
for d in prompt_templates:
    task = asyncio.create_task(async_langchain(chain, d))
    tasks.append(task)
responses = await asyncio.gather(*tasks)
end_time = datetime.now()


difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")
print("\n".join(responses))


2024-09-25 08:13:11,344 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-25 08:13:11,349 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-25 08:13:11,351 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-25 08:13:11,754 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-25 08:13:12,110 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




Elapsed seconds: 1.824427
Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.
Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.
Escúchame. Eres perfecto/a. Eres hermoso/a y te amo.
Ascoltami. Sei straordinario. Sei magnifico e ti amo.
Figyelj rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.


In [8]:
DEBUG = True
N_BROWSERS = 4

In [9]:
# class to maintain settings and state within graph

class AgentState(TypedDict):
    AIdf: dict                #  the current working set of headlines (pandas dataframe not supported)
    before_date: str          #  ignore stories before this date for deduplication (force reprocess since)
    do_download: bool         #  if False use existing files, else download from sources
    sources: dict             #  sources to scrap
    sources_reverse: dict     #  map file names to sources
    headline_str: str         #  headline text email
    bullets: str              #  bullet point summary email
    summary: str              #  final summary
    n_edits: int              #  count edit iterations so we don't keep editing forever
    edit_complete: bool       #  edit will update if no more edits to make
    # message thread with OpenAI
    messages: Annotated[list[AnyMessage], operator.add]


In [10]:
# Initialize reading configurations from YAML file

def fn_initialize(state: AgentState) -> AgentState:
    """
    Initializes the agent state by loading source configurations from SOURCECONFIG (sources.yaml) .

    Args:
        state (AgentState): The current state of the agent.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        AgentState: The updated state of the agent.

    Raises:
        yaml.YAMLError: If there is an error while loading the YAML file.

    """

    #  load sources to scrape from sources.yaml
    with open(SOURCECONFIG, "r") as stream:
        try:
            state['sources'] = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

    log(f"Initialized {len(state['sources'])} items in sources from {SOURCECONFIG}")

    # make a reverse dict to map file titles to source names
    state['sources_reverse'] = {}
    for k, v in state['sources'].items():
        log(f"{k} -> {v['url']} -> {v['title']}.html")
        v['sourcename'] = k
        # map filename (title) to source name
        state['sources_reverse'][v['title']] = k

    log(f"Initialized {len(state['sources_reverse'])} items in sources_reverse")

    return state


if DEBUG:
    test_state = AgentState()
    fn_initialize(test_state)


2024-09-25 08:13:12,143 - AInewsbot - INFO - Initialized 17 items in sources from sources.yaml
2024-09-25 08:13:12,143 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-09-25 08:13:12,144 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology.html
2024-09-25 08:13:12,144 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Business Insider Tech.html
2024-09-25 08:13:12,144 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> FT Technology.html
2024-09-25 08:13:12,144 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Feedly AI.html
2024-09-25 08:13:12,145 - AInewsbot -

In [11]:
# scrape sources with selenium and save local files in DOWNLOAD_DIR (htmldata)
def fn_download_sources(state: AgentState) -> AgentState:
    """
    Scrapes sources and saves HTML files.
    If state["do_download"] is True, deletes all files in DOWNLOAD_DIR (htmldata) and scrapes fresh copies.
    If state["do_download"] is False, uses existing files in DOWNLOAD_DIR.
    Uses state["sources"] for config info on sources to scrape
    For each source, saves the current filename to state["sources"][sourcename]['latest']

    Args:
        state (AgentState): The current state of the agent.
        do_delete (bool, optional): Whether to delete files in DOWNLOAD_DIR. Defaults to True.

    Returns:
        AgentState: The updated state of the agent.
    """

    if state["do_download"]:
        # empty download directory
        delete_files(DOWNLOAD_DIR)

        # save each file specified from sources
        log(f"Saving HTML files using {N_BROWSERS} browsers")

        # Create a queue for multiprocessing and populate it
        queue = multiprocessing.Queue()
        for item in state["sources"].values():
            queue.put(item)

        # Function to take the queue and pop entries off and process until none are left
        # lets you create an array of functions with different args
        callable = process_source_queue_factory(queue)

        saved_pages = launch_drivers(N_BROWSERS, callable)
        for sourcename, file in saved_pages:
            log(f"Downloaded {sourcename} to {file}")
            state['sources'][sourcename]['latest'] = file
        log(f"Saved {len(saved_pages)} HTML files")

    else:   # don't delete, just get list of existing files
        log(f"Web fetch disabled, using existing files in {DOWNLOAD_DIR}")
        # Get the current date
        datestr = datetime.now().strftime("%m_%d_%Y")
        files = [os.path.join(DOWNLOAD_DIR, file)
                 for file in os.listdir(DOWNLOAD_DIR)]
        # filter files with today's date ending in .html
        files = [
            file for file in files if datestr in file and file.endswith(".html")]
        log(f"Found {len(files)} previously downloaded files")
        for file in files:
            log(file)

        saved_pages = []
        for file in files:
            filename = os.path.basename(file)
            # locate date like '01_14_2024' in filename
            position = filename.find(" (" + datestr)
            basename = filename[:position]
            # match to source name
            sourcename = state["sources_reverse"].get(basename)
            if sourcename is None:
                log(f"Skipping {basename}, no sourcename metadata")
                continue
            state["sources"][sourcename]['latest'] = file

    return state

if DEBUG:
    test_state["do_download"] = True
#     test_state["before_date"] = '2024-09-22 12:00:00'
    _ = fn_download_sources(test_state)


2024-09-25 08:13:15,304 - AInewsbot - INFO - Saving HTML files using 4 browsers
2024-09-25 08:13:15,319 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:13:15,320 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:13:15,321 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:13:15,321 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:13:31,488 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:13:31,488 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:13:31,488 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:13:31,489 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:13:31,489 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-09-25 08:13:31,489 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-09-25 08:13:31,490 - AInewsbot - INFO -

2024-09-25 08:14:59,069 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (09_25_2024 08_14_59 AM).html as utf-8
2024-09-25 08:14:59,071 - AInewsbot - INFO - Processing Reddit
2024-09-25 08:14:59,074 - AInewsbot - INFO - get_files(Reddit multiple subreddits) - starting get_files https://www.reddit.com/r/ChatGPT+ChatGPTCoding+MacOS+MachineLearning+OpenAI+ProgrammerHumor+Windows10+battlestations+buildapc+cordcutters+dataisbeautiful+gadgets+hardware+linux+msp+programming+realtech+software+talesfromtechsupport+tech+technews+technology+techsupportgore+windows/top/?sort=top&t=day
2024-09-25 08:14:59,514 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteErro

2024-09-25 08:15:57,651 - AInewsbot - INFO - get_files(Feedly AI) - Saving Feedly AI (09_25_2024 08_15_57 AM).html as utf-8
2024-09-25 08:15:57,652 - AInewsbot - INFO - Quit webdriver
2024-09-25 08:15:59,385 - AInewsbot - INFO - returned 17
2024-09-25 08:15:59,386 - AInewsbot - INFO - Downloaded FT Tech to htmldata/FT Technology (09_25_2024 08_14_37 AM).html
2024-09-25 08:15:59,386 - AInewsbot - INFO - Downloaded Google News to htmldata/Google News - Technology - Artificial intelligence (09_25_2024 08_14_59 AM).html
2024-09-25 08:15:59,387 - AInewsbot - INFO - Downloaded Reddit to htmldata/Reddit multiple subreddits (09_25_2024 08_15_31 AM).html
2024-09-25 08:15:59,387 - AInewsbot - INFO - Downloaded Ars Technica to htmldata/Ars Technica (09_25_2024 08_14_39 AM).html
2024-09-25 08:15:59,387 - AInewsbot - INFO - Downloaded Hacker News 2 to htmldata/Hacker News Page 2 (09_25_2024 08_14_49 AM).html
2024-09-25 08:15:59,387 - AInewsbot - INFO - Downloaded NYT Tech to htmldata/New York Times

In [14]:
def fn_extract_urls(state: AgentState) -> AgentState:
    """
    Extracts news URLs from the latest HTML files matching the patterns defined in the state['sources'] configuration info.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the extracted URLs stored in state['AIdf'].
    """
    # Parse news URLs and titles from downloaded HTML files
    log("Parsing html files")
    all_urls = []
    for sourcename, sourcedict in state['sources'].items():
        filename = sourcedict.get('latest')
        if not filename:
            log(f"no filename found for {sourcename}")
            continue

        log(sourcename + ' -> ' + filename)
        links = parse_file(state['sources'][sourcename])
        log(f"{len(links)} links found")
        all_urls.extend(links)

    log(f"Saved {len(all_urls)} links")

    # make a pandas dataframe of all the links found
    AIdf = (
        pd.DataFrame(all_urls)
        .groupby("url")
        .first()
        .reset_index()
        .sort_values("src")[["src", "title", "url"]]
        .reset_index(drop=True)
        .reset_index(drop=False)
        .rename(columns={"index": "id"})
    )
    state['AIdf'] = AIdf.to_dict()

    return state


if DEBUG:
    _ = fn_extract_urls(test_state)



2024-09-25 08:20:01,804 - AInewsbot - INFO - Parsing html files
2024-09-25 08:20:01,805 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (09_25_2024 08_14_39 AM).html
2024-09-25 08:20:01,836 - AInewsbot - INFO - parse_file - found 252 raw links
2024-09-25 08:20:01,841 - AInewsbot - INFO - parse_file - found 28 filtered links
2024-09-25 08:20:01,841 - AInewsbot - INFO - 28 links found
2024-09-25 08:20:01,842 - AInewsbot - INFO - Bloomberg Tech -> htmldata/Bloomberg Technology (09_25_2024 08_14_38 AM).html
2024-09-25 08:20:01,867 - AInewsbot - INFO - parse_file - found 210 raw links
2024-09-25 08:20:01,870 - AInewsbot - INFO - parse_file - found 25 filtered links
2024-09-25 08:20:01,870 - AInewsbot - INFO - 25 links found
2024-09-25 08:20:01,870 - AInewsbot - INFO - Business Insider -> htmldata/Business Insider Tech (09_25_2024 08_14_36 AM).html
2024-09-25 08:20:01,893 - AInewsbot - INFO - parse_file - found 309 raw links
2024-09-25 08:20:01,898 - AInewsbot - INFO - parse_file 

In [15]:
# s/b 17 if some like bloomberg or wsj are missing, maybe got a robot block, download missing one manually and then re-run
print(len(pd.DataFrame(test_state["AIdf"]).groupby('src').count()))
pd.DataFrame(test_state["AIdf"]).groupby('src').count()


17


,id,title,url
src,,,
Ars Technica,28,28,28
Bloomberg Tech,20,20,20
Business Insider,51,51,51
FT Tech,62,62,62
Feedly AI,63,63,63
Google News,63,63,63
Hacker News,21,21,21
Hacker News 2,24,24,24
HackerNoon,91,91,91


In [16]:
# filter and clean URLs for new AI stories

def fn_filter_urls(state: AgentState) -> AgentState:
    """
    Filters the URLs in state["AIdf"] to include only those that have not been previously seen,
    and are related to AI according to the response from a ChatGPT prompt.

    Args:
        state (AgentState): The current state of the agent.
        before_date (str, optional): The date before which the URLs should be filtered. Defaults to "".

    Returns:


        AgentState: The updated state of the agent with the filtered URLs stored in state["AIdf"].

    """
    # filter to URL not previously seen
    AIdf = pd.DataFrame(state["AIdf"])

    AIdf = filter_unseen_urls_db(AIdf, before_date=state["before_date"])

    if len(AIdf) == 0:
        log("No new URLs, returning")
        return state

    # dedupe identical headlines
    # filter similar titles differing by type of quote or something
    AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)
    AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
    AIdf = AIdf.sort_values("src") \
        .groupby("title_clean") \
        .first() \
        .reset_index(drop=True) \
        .drop(columns=['id']) \
        .reset_index() \
        .rename(columns={'index': 'id'})
    log(f"Found {len(AIdf)} unique AI headlines")

    # structured response format
    json_schema = {
        "name": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "isai_array": {
                    "type": "array",
                    "items": {
                        "type": "object",
                                "properties": {
                                    "id": {
                                        "type": "number"
                                    },
                                    "isAI": {
                                        "type": "boolean"
                                    }
                                },
                        "required": ["id", "isAI"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["isai_array"],
            "additionalProperties": False
        }
    }

    # filter AI-related headlines using a prompt
    pages = paginate_df(AIdf)
    enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT, json_schema=json_schema))
    filter_df = pd.DataFrame(enriched_urls)
    
    try:  # for idempotency
        AIdf = AIdf.drop(columns=['isAI'])
    except:
        pass

    # merge returned df with isAI column into original df on id column
    AIdf = pd.merge(AIdf, filter_df, on="id", how="outer")
    log(AIdf.columns)
    # set hostname based on actualurl
    AIdf['actual_url'] = AIdf['url']    # ideally resolve redirects but Google News blocks
    AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)

    # update SQLite database with all seen URLs (we are doing this using url and ignoring redirects)
    conn = sqlite3.connect(SQLITE_DB)
    cursor = conn.cursor()
    for row in AIdf.itertuples():
        insert_article(conn, cursor, row.src, row.hostname, row.title,
                       row.url, row.actual_url, row.isAI, datetime.now().date())

    # keep headlines that are related to AI
    AIdf = AIdf.loc[AIdf["isAI"]==1] \
        .reset_index(drop=True)  \
        .reset_index()  \
        .drop(columns=["id"])  \
        .rename(columns={'index': 'id'})

    log(f"Found {len(AIdf)} AI headlines")

    # update actual URLs for Google News redirects
    # I think Google changed something so this no longer works, instead of a 301 redirct
    # get a javascript page that redirects. Also tomorrow we might see different URLs for same stories
    # AIdf = get_google_news_redirects(AIdf)

    conn = sqlite3.connect('articles.db')
    query = "select * from sites"
    sites_df = pd.read_sql_query(query, conn)
    sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}
    conn.close()

    # get clean site_name
    AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))

    # if any missing clean site names, populate them using OpenAI
    missing_site_names =  len(AIdf.loc[AIdf['site_name']==""])
    if missing_site_names:
        log(f"Asking OpenAI for {missing_site_names} missing site names")
        responses = asyncio.run(fetch_missing_site_names(AIdf))
        # update site_dict from responses
        new_urls = []
        for r in responses:
            if r['url'].startswith('https://'):
                r['url'] = r['url'][8:]
            new_urls.append(r['url'])
            sites_dict[r['url']] = r['site_name']
            log(f"Looked up {r['url']} -> {r['site_name']}")
        # update sites table with new names
        for url in new_urls:
            sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
            log(f"Updated {url}, '->', {sites_dict[url]}")
            conn.execute(sqlstr, (url, sites_dict[url]))
            conn.commit()
        # reapply to AIdf with updated sites
        AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))
    else:
        log("No missing site names")

    # drop banned slop sites
    AIdf = AIdf.loc[~AIdf["hostname"].str.lower().isin(HOSTNAME_SKIPLIST)]
    AIdf = AIdf.loc[~AIdf["site_name"].str.lower().isin(SITE_NAME_SKIPLIST)]

    state["AIdf"] = AIdf.to_dict()
    return state


if DEBUG:
    before_date="2024-09-23 10:00:00"
    test_state["before_date"] = before_date
#     test_state["before_date"] = None
    _ = fn_filter_urls(test_state)


2024-09-25 08:20:17,375 - AInewsbot - INFO - Querying SQLite with where_clause: WHERE timestamp < '2024-09-23 10:00:00'
2024-09-25 08:20:18,004 - AInewsbot - INFO - URLs in orig_df: 999
2024-09-25 08:20:18,006 - AInewsbot - INFO - Existing URLs in DB: 149940
2024-09-25 08:20:18,033 - AInewsbot - INFO - New URLs in df filtered by URL: 677
2024-09-25 08:20:18,112 - AInewsbot - INFO - Existing src+title: 25
2024-09-25 08:20:18,113 - AInewsbot - INFO - New URLs in df filtered by src+title: 652
2024-09-25 08:20:18,123 - AInewsbot - INFO - Found 641 unique AI headlines
2024-09-25 08:20:19,486 - AInewsbot - INFO - Applying prompt to 13 pages using gpt-4o-mini
2024-09-25 08:20:19,487 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:19,489 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:19,489 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:19,489 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:19,490 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:19,494 - AInewsbot - I

In [17]:
print(pd.DataFrame(test_state['AIdf']).columns)
len(pd.DataFrame(test_state['AIdf']))


Index(['id', 'src', 'title', 'url', 'isAI', 'actual_url', 'hostname',
       'site_name'],
      dtype='object')


182

In [18]:
# Function for topic extraction

def clean_topics(row, lcategories):
    """
    Cleans the extracted_topics and assigned_topics by removing certain common topics and combining them into a single list.

    Args:
        row (pandas.Series): The row containing the extracted_topics and assigned_topics.
        lcategories (set): The set of lowercase categories.

    Returns:
        list: The cleaned and combined list of topics.
    """
    extracted_topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(extracted_topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]

    return combined


async def do_cat(AIdf, categories):
    """
    Sends a prompt to ChatGPT to select topics for the title for each row in AIdf
    which match the topics in categories.

    Args:
        AIdf (pandas.DataFrame): The DataFrame containing the headlines.
        categories (list): The list of topics to match with the headlines.

    Returns:
        dict: A dictionary where the keys are the row IDs and the values are lists
        of selected topics for each headline.

    """

    catdict = {}
    async with aiohttp.ClientSession() as session:
        for i, row in enumerate(AIdf.itertuples()):
            tasks = []
            log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
            h = row.title
            log(h)
            for c in categories:
                task = asyncio.create_task(categorize_headline(h, c, session))
                tasks.append(task)
            responses = await asyncio.gather(*tasks)
            catdict[row.id] = [item for sublist in responses for item in sublist]
            log(str(catdict[row.id]))

    return catdict


def fn_topic_analysis(state: AgentState) -> AgentState:

    """
    Extracts and selects topics for each headline in the state['AIdf'] dataframe, scrubs them, and stores them back in the dataframe.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the extracted and selected topics stored in state['AIdf'].
    """
    AIdf = pd.DataFrame(state["AIdf"])
    pages = paginate_df(AIdf)
    # apply topic extraction prompt to AI headlines
    log("start free-form topic extraction")
    json_schema = {
        "name": "extracted_topics",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "extracted_topics": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "id": {
                                "type": "number",
                            },
                            "topics": {
                                "type": "array",
                                "items": {
                                    "type": "string",
                                },
                            },
                        },
                        "required": ["id", "topics"],
                        "additionalProperties": False,
                    }
                }
            },
            "required": ["extracted_topics"],
            "additionalProperties": False,
        }
    }
    response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT, json_schema=json_schema))
    topic_df = pd.DataFrame(response)
    topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})

    log(f"{len(topic_df)} free-form topics extracted")
    all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
    item_counts = Counter(all_topics)
    filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
    categories = sorted(CANONICAL_TOPICS)
    # use categories that are canonical or show up twice in freeform
    lcategories = set([c.lower() for c in categories] + [c.lower() for c in filtered_topics])
    # new topics
    log([c for c in filtered_topics if c not in categories])

    catdict = asyncio.run(categorize_headline(AIdf, categories=categories))
    topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))

    topic_df["topics"] = topic_df.apply(lambda t: clean_topics(t, lcategories), axis=1)
    topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)

    try: # for idempotency
        AIdf = AIdf.drop(columns=['topic_str', 'title_topic_str'])
    except:
        pass

    AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
    AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)

    state["AIdf"] = AIdf.to_dict()
    return state

if DEBUG:
    _ = fn_topic_analysis(test_state)


2024-09-25 08:20:28,224 - AInewsbot - INFO - start free-form topic extraction
2024-09-25 08:20:28,224 - AInewsbot - INFO - Applying prompt to 4 pages using gpt-4o-mini
2024-09-25 08:20:28,225 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:28,225 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:28,225 - AInewsbot - INFO - sent 50 items 
2024-09-25 08:20:28,225 - AInewsbot - INFO - sent 32 items 
2024-09-25 08:20:38,027 - AInewsbot - INFO - got dict with 32 items 
2024-09-25 08:20:39,195 - AInewsbot - INFO - got dict with 50 items 
2024-09-25 08:20:39,870 - AInewsbot - INFO - got dict with 50 items 
2024-09-25 08:20:40,555 - AInewsbot - INFO - got dict with 50 items 
2024-09-25 08:20:40,559 - AInewsbot - INFO - Processed 182 responses.
2024-09-25 08:20:40,562 - AInewsbot - INFO - 182 free-form topics extracted
2024-09-25 08:20:40,564 - AInewsbot - INFO - ['regulation', 'legislation', 'james cameron', 'stability ai', 'board', 'enterprise', 'pcs', 'chatbot', 'investment', 'health

2024-09-25 08:21:01,398 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,399 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,400 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,401 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,402 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,403 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,404 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,404 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,405 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:01,405 - AInewsbot - INFO - sent 2 items 
2024-09-25 08:21:02,237 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:21:03,804 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:03,832 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:03,858 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:03,879 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:03,953 - AInewsbot - INFO - got dict wit

2024-09-25 08:21:22,162 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:21:24,210 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,213 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,214 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,232 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,241 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,256 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,840 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:24,881 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:26,853 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:26,855 - AInewsbot - INFO - Processed 182 responses.
2024-09-25 08:21:26,856 - AInewsbot - INFO - Authors & Writing, topic 11 of 154
2024-09-25 08:21:26,857 - AInewsbot - INFO - Applying prompt to 10 pages using gpt-4o-mini
2024-09-25 08:21:26,858 - AInewsbot - INFO - sent 20 items 
2024-09-25

2024-09-25 08:21:45,408 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:21:45,410 - AInewsbot - INFO - Processed 182 responses.
2024-09-25 08:21:45,411 - AInewsbot - INFO - Bill Gates, topic 16 of 154
2024-09-25 08:21:45,411 - AInewsbot - INFO - Applying prompt to 10 pages using gpt-4o-mini
2024-09-25 08:21:45,412 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,413 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,414 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,416 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,417 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,418 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,419 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,419 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,420 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:21:45,420 - AInewsbot - INFO - sent 2 items 
2024-09-25 08:21:46,532 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:21:48,377 - AIne

2024-09-25 08:22:04,371 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:22:04,372 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:22:04,373 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:22:04,374 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:22:04,374 - AInewsbot - INFO - sent 2 items 
2024-09-25 08:22:05,156 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:22:06,637 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:06,661 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:06,906 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:06,909 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:06,930 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:06,955 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:07,255 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:07,640 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:22:07,672 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 0

...

2024-09-25 08:30:06,599 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:06,600 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:06,602 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:06,603 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:06,604 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:06,604 - AInewsbot - INFO - sent 2 items 
2024-09-25 08:30:07,567 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:30:11,149 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:12,202 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:12,245 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:12,251 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:12,287 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:12,489 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:14,279 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:14,318 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:14,4

2024-09-25 08:30:36,520 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:36,893 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:36,967 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:36,984 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:37,611 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:37,613 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:37,614 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:37,616 - AInewsbot - INFO - Processed 182 responses.
2024-09-25 08:30:37,617 - AInewsbot - INFO - Streaming, topic 135 of 154
2024-09-25 08:30:37,618 - AInewsbot - INFO - Applying prompt to 10 pages using gpt-4o-mini
2024-09-25 08:30:37,619 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:37,620 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:37,621 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:37,622 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:37,622 - AInewsbot - INFO 

2024-09-25 08:30:55,955 - AInewsbot - INFO - Taiwan, topic 140 of 154
2024-09-25 08:30:55,956 - AInewsbot - INFO - Applying prompt to 10 pages using gpt-4o-mini
2024-09-25 08:30:55,958 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,959 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,960 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,961 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,962 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,963 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,963 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,964 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,965 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:30:55,966 - AInewsbot - INFO - sent 2 items 
2024-09-25 08:30:56,760 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:30:58,707 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:58,710 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:30:58,712 - AInewsbo

2024-09-25 08:31:16,339 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:16,339 - AInewsbot - INFO - sent 2 items 
2024-09-25 08:31:17,139 - AInewsbot - INFO - got dict with 2 items 
2024-09-25 08:31:19,295 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,299 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,301 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,302 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,914 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,930 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,934 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:19,997 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:20,178 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:20,181 - AInewsbot - INFO - Processed 182 responses.
2024-09-25 08:31:20,181 - AInewsbot - INFO - Travel, topic 146 of 154
2024-09-25 08:31:20,182 - AInewsbot - INFO - Applyin

2024-09-25 08:31:40,801 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:40,806 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:40,940 - AInewsbot - INFO - got dict with 20 items 
2024-09-25 08:31:40,944 - AInewsbot - INFO - Processed 182 responses.
2024-09-25 08:31:40,945 - AInewsbot - INFO - Virtual Assistants, topic 151 of 154
2024-09-25 08:31:40,946 - AInewsbot - INFO - Applying prompt to 10 pages using gpt-4o-mini
2024-09-25 08:31:40,947 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,948 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,950 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,951 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,951 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,952 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,953 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,954 - AInewsbot - INFO - sent 20 items 
2024-09-25 08:31:40,955 - AInewsbot - INFO - sent 20 items 
2024-09-25

In [19]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):
    """
    Generates a name for a cluster based on a list of headline topics.

    Parameters:
    session (aiohttp.ClientSession): The client session for making async HTTP requests.
    topic_list_str (str): A string containing the list of headline topics.
    max_retries (int, optional): The maximum number of retries in case of an error. Defaults to 3.
    model (str, optional): The model to use for generating the topic name. Defaults to LOWCOST_MODEL.

    Returns:
    dict: A dictionary containing the generated topic name.

    Example Usage:
    title_topic_str_list = "Headline 1 (Topic: Topic 1)\n\nHeadline 2 (Topic: Topic 2)"
    result = await write_topic_name(session, title_topic_str_list)
    print(result)

    Output:
    {"topic_title": "Generated Topic Name"}
    ```
    """
    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses.
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
#             print(topic_list_str)
            response = asyncio.run(fetch_openai(session, payload))
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}


async def afn_topic_clusters(state: AgentState) -> AgentState:
    """
    Fetches embeddings for the headlines, creates clusters of similar articles using DBSCAN, and sorts
    using the clusters and a traveling salesman shortest traversal in embedding space.

    Parameters:
    state (AgentState): The state of the agent.

    Returns:
    AgentState: The updated state of the agent.

    """
    AIdf = pd.DataFrame(state["AIdf"])

    log(f"Fetching embeddings for {len(AIdf)} headlines")
    embedding_model = 'text-embedding-3-large'
    client = OpenAI()
    response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                        model=embedding_model)
    embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

    # greedy traveling salesman sort
    log(f"Sort with nearest_neighbor_sort sort")
    sorted_indices = nearest_neighbor_sort(embedding_df)
    AIdf['sort_order'] = sorted_indices

    # do dimensionality reduction on embedding_df and cluster analysis
    log(f"Perform dimensionality reduction")
    with open("reducer.pkl", 'rb') as file:
        # Load the model from the file
        reducer = pickle.load(file)
    reduced_data = reducer.transform(embedding_df)
    log(f"Cluster with DBSCAN")
    dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
    AIdf['cluster'] = dbscan.fit_predict(reduced_data)
    AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999

    # sort first by clusters found by DBSCAN, then by semantic ordering
    AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
        .reset_index(drop=True) \
        .reset_index() \
        .drop(columns=["id"]) \
        .rename(columns={'index': 'id'})

    # show clusters
    cluster_topics = []
    with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
        async with aiohttp.ClientSession() as session:
            for i in range(30):
                tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
                if len(tmpdf) ==0:
                    break
                display(tmpdf)
                title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
                cluster_topic = await write_topic_name(session, title_topic_str_list)
                cluster_topic = cluster_topic['topic_title']
                cluster_topics.append(cluster_topic)
                log(f"I dub this cluster: {cluster_topic}")
    state["cluster_topics"] = cluster_topics
    AIdf["cluster_name"] = AIdf['cluster'].apply(lambda i: cluster_topics[i] if i < len(cluster_topics) else "")
    state["AIdf"] = AIdf.to_dict()
    return state

# TODO: could add a quality rating for stories based on site reputation, length, complexity of story
# could then add the quality rating to the summaries and tell the prompt to favor high-quality stories
# could put summaries into vector store and retrieve stories by topic. but then you will have to deal
# with duplicates across categories, ask the prompt to dedupe

def fn_topic_clusters(state: AgentState) -> AgentState:
    "call async afn_topic_clusters on state"
    asyncio.run(afn_topic_clusters(state))
    return state


if DEBUG:
    _ = fn_topic_clusters(test_state)


2024-09-25 08:31:58,263 - AInewsbot - INFO - Fetching embeddings for 182 headlines
2024-09-25 08:31:59,023 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-25 08:32:00,029 - AInewsbot - INFO - Sort with nearest_neighbor_sort sort
2024-09-25 08:32:00,082 - AInewsbot - INFO - Perform dimensionality reduction
2024-09-25 08:32:03,376 - AInewsbot - INFO - Cluster with DBSCAN


,id,title_topic_str
0,0,"Stability AI says filmmaker James Cameron has joined its board of directors; Stability AI CEO Prem Akkaraju was previously CEO of VFX company WETA Digital (Topics: Art & Design, Big Tech, Board Of Directors, Entertainment, Gen AI, Hollywood, James Cameron, Prem Akkaraju, Stability AI, Tv & Film & Movies, Vfx)"
1,1,"'Titanic' director James Cameron joins Stability AI board (Topics: Big Tech, Board, Entertainment, Film Industry, Gen AI, Hollywood, James Cameron, Stability AI, Tv & Film & Movies)"
2,2,"James Cameron is joining Stability AIs board of directors (Topics: Big Tech, Board Of Directors, Entertainment, Gen AI, Governance, Hollywood, Intellectual Property, James Cameron, Politics, Science, Stability AI, Tv & Film & Movies)"
3,3,"Terminator Director James Cameron Joins Stability AI Board of Directors (Topics: Art & Design, Big Tech, Board Of Directors, Entertainment, Gen AI, Hollywood, James Cameron, Stability AI, Terminator, Tv & Film & Movies)"
4,4,"James Cameron Joins Board of Stability AI in Coup for Tech Firm (Topics: Big Tech, Board, Entertainment, Gen AI, Governance, Hollywood, Intellectual Property, James Cameron, Politics, Science, Stability AI, Tv & Film & Movies)"


2024-09-25 08:32:04,348 - AInewsbot - INFO - I dub this cluster: James Cameron Joins Stability AI


,id,title_topic_str
5,5,"AI PCs will dominate shipments by 2026, but not because of demand (Topics: Economics, Gen AI, Hardware, Market Trends, Pcs, Products, Shipments)"
6,6,"AI GPU Clusters, from Your Laptop, with Livebook (Topics: Gen AI, Gpu Clusters, Hardware, Laptops, Livebook, Nvidia, Products, Science)"
7,7,"AI PCs will dominate shipments by 2026, but not because of demandBuying computers without the tech to get harder as chipmakers compete for cash, say analystsAI + ML4 hrs|13 (Topics: Chipmakers, Economics, Finance, Gen AI, Hardware, Market Competition, Pcs, Products, Semiconductor Chips)"


2024-09-25 08:32:05,024 - AInewsbot - INFO - I dub this cluster: AI and Hardware Market Trends


...

2024-09-25 08:32:10,956 - AInewsbot - INFO - I dub this cluster: AI in Streaming and Entertainment


,id,title_topic_str
49,49,"Meta's chatbot could soon sound like John Cena or Judi Dench but definitely not OpenAI's Scarlett Johansson soundalike (Topics: Big Tech, Chatbot, Chatbots, Entertainment, Gen AI, Hollywood, Intellectual Property, John Cena, Judi Dench, Language Models, Mark Zuckerberg, Meta, Music, OpenAI, Products, Speech Recognition & Synthesis, Tv & Film & Movies, Virtual Assistants, Voice Technology)"
50,50,"Meta Turns to Awkwafina, John Cena to Draw Users to AI (Topics: Awkwafina, Big Tech, Chatbots, Entertainment, Gen AI, Hollywood, John Cena, Mark Zuckerberg, Meta, Music, Tv & Film & Movies, User Engagement, Virtual Assistants)"
51,51,"Metas AI chatbot to start speaking in the voices of Judi Dench, John Cena, others; Report (Topics: AI Chatbot, Big Tech, Chatbots, Entertainment, Gen AI, Hollywood, Intellectual Property, John Cena, Judi Dench, Language Models, Mark Zuckerberg, Meta, Music, Products, Speech Recognition & Synthesis, Tv & Film & Movies, Virtual Assistants, Voice Technology)"
52,52,"Meta loves celebrities and celebrities love Meta. They're set to be the voices behind the company's new AI bots. (Topics: AI Bots, Big Tech, Celebrities, Chatbots, Entertainment, Gen AI, Hollywood, Mark Zuckerberg, Meta, Music, Speech Recognition & Synthesis, Tv & Film & Movies, Virtual Assistants, Voice Technology)"
53,53,"Source: Meta plans to launch the voices of Judi Dench, Kristen Bell, John Cena, Awkwafina, and Keegan-Michael Key for its Meta AI chatbot at Connect this week (Topics: AI Chatbot, Awkwafina, Big Tech, Chatbots, Code Assistants, Entertainment, Gen AI, Hollywood, John Cena, Judi Dench, Keegan-Michael Key, Kristen Bell, Language Models, Mark Zuckerberg, Meta, Speech Recognition & Synthesis, Tv & Film & Movies, Virtual Assistants, Voice Technology)"


2024-09-25 08:32:11,390 - AInewsbot - INFO - I dub this cluster: Meta's Celebrity Voice Chatbots


,id,title_topic_str
54,54,"Is Math the Path to Chatbots That Dont Make Stuff Up? (Topics: Accuracy, Chatbots, Chatgpt, Code Assistants, Cognitive Science, Disinformation, Education, Ethics, Gen AI, Language Models, Math, Opinion, Perplexity, Safety And Alignment, Science, Testing, Virtual Assistants)"
55,55,"The more I learn how to use ChatGPT, the more I realize how basic it is. (Topics: AI Limitations, Chatbots, Chatgpt, Education, Gen AI, Language Models, OpenAI, Opinion, Review, User Experience, Virtual Assistants)"
56,56,"ChatGPT allowed me to sue my landlord without needing legal representation! (Topics: Bias And Fairness, Chatbots, Chatgpt, Claude, Code Assistants, Copilot, Ethics, Gen AI, Governance, Inequality, Intellectual Property, Landlord, Language Models, Lawsuit, Legal Issues, Legal Representation, OpenAI, Opinion, Privacy, Society & Culture, Uk, Virtual Assistants)"


2024-09-25 08:32:12,099 - AInewsbot - INFO - I dub this cluster: Chatbots and AI Limitations


In [20]:
# scrape individual pages
def fn_download_pages(state: AgentState) -> AgentState:
    """
    Uses several Selenium browser sessions to download all the pages referenced in the
    state["AIdf"] DataFrame and store their pathnames.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the downloaded pages' pathnames stored in the `state["AIdf"]` DataFrame.
    """
    log("Queuing URLs for scraping")
    AIdf = pd.DataFrame(state['AIdf'])
    queue = multiprocessing.Queue()

    count = 0
    for row in AIdf.itertuples():
#         if row.cluster < 999:
        queue.put((row.id, row.url, row.title))
        count +=1
    # scrape urls in queue asynchronously
    num_browsers = 4

    callable = process_url_queue_factory(queue)

    log(f"fetching {count} pages using {num_browsers} browsers")
    saved_pages = launch_drivers(num_browsers, callable)

    pages_df = pd.DataFrame(saved_pages)
    if len(pages_df):
        pages_df.columns = ['id', 'url', 'title', 'path']

        try: # for idempotency
            AIdf = AIdf.drop(columns=['path'])
        except:
            pass        
        AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")
    state["AIdf"] = AIdf.to_dict()
    return state


if DEBUG:
    _ = fn_download_pages(test_state)



2024-09-25 08:32:12,147 - AInewsbot - INFO - Queuing URLs for scraping
2024-09-25 08:32:12,151 - AInewsbot - INFO - fetching 182 pages using 4 browsers
2024-09-25 08:32:12,154 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:32:12,155 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:32:12,155 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:32:12,156 - AInewsbot - INFO - get_driver - 44926 Initializing webdriver
2024-09-25 08:32:28,834 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:32:28,834 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:32:28,835 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:32:28,835 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-25 08:32:28,835 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-09-25 08:32:28,836 - AInewsbot - INFO - get_driver - In

2024-09-25 08:33:41,481 - AInewsbot - INFO - Processing https://phys.org/news/2024-09-sweet-machine-reveals-factors-successful.html
2024-09-25 08:33:41,482 - AInewsbot - INFO - get_url(https://phys.org/news/2024-09-sweet-machine-reveals-factors-successful.html) - starting get_url https://phys.org/news/2024-09-sweet-machine-reveals-factors-successful.html
2024-09-25 08:33:43,043 - AInewsbot - INFO - get_url(AI GPU Clusters, from Your Laptop, with Livebook) - Saving AI_GPU_Clusters__from_Your_Laptop__with_Livebook_20240925_083343.html as utf-8
2024-09-25 08:33:43,044 - AInewsbot - INFO - Processing https://time.com/7023237/vinod-khosla-interview/
2024-09-25 08:33:43,044 - AInewsbot - INFO - get_url(https://time.com/7023237/vinod-khosla-interview/) - starting get_url https://time.com/7023237/vinod-khosla-interview/
2024-09-25 08:33:43,551 - AInewsbot - INFO - get_url(AI PCs will dominate shipments by 2026, but not because of demandBuying computers without the tech to get harder as chipmak

2024-09-25 08:34:06,560 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMixgFBVV95cUxNRVlqLVFWYVVrVkppaUY1Nk56STVoQVlJVHBISUlkQ3Zyc1BLdGJNa1NNN1VZMVN1QXBtMEo1ODRVSXNXVTZVbDVBbnN4VTBzdVhoMGtkb2lCUVRKWVZEN1o2SV9hdTJRWUtNYW45TFJKb2hCLUpBNDNrSEFPV0kxTEQ3LTlVNjhjQnlDRmVkNWFTVW80cWZkM09KYUpHbnZVNS1IZGtrdWpMSVZSMFNlV1hVcGM4T3JTNkZZOV9TOHZOMVNmYmc) - starting get_url https://news.google.com/read/CBMixgFBVV95cUxNRVlqLVFWYVVrVkppaUY1Nk56STVoQVlJVHBISUlkQ3Zyc1BLdGJNa1NNN1VZMVN1QXBtMEo1ODRVSXNXVTZVbDVBbnN4VTBzdVhoMGtkb2lCUVRKWVZEN1o2SV9hdTJRWUtNYW45TFJKb2hCLUpBNDNrSEFPV0kxTEQ3LTlVNjhjQnlDRmVkNWFTVW80cWZkM09KYUpHbnZVNS1IZGtrdWpMSVZSMFNlV1hVcGM4T3JTNkZZOV9TOHZOMVNmYmc
2024-09-25 08:34:10,961 - AInewsbot - INFO - get_url(Altera brings more AI to the edge and cloud with new programmable chips) - Saving Altera_brings_more_AI_to_the_edge_and_cloud_with_new_programmable_chips_20240925_083410.html as utf-8
2024-09-25 08:34:10,964 - AInewsbot - INFO - Processing https://techcrunch.com/2024/09/2

2024-09-25 08:34:34,864 - AInewsbot - INFO - get_url(Coatue-backed Distyl AI founded by Palantir alums, raises $20 million funding round, sources say) - Saving Coatue-backed_Distyl_AI_founded_by_Palantir_alums__raises__20_million_funding_round__sources_say_20240925_083434.html as utf-8
2024-09-25 08:34:34,865 - AInewsbot - INFO - Processing https://www.constellationr.com/blog-news/insights/google-cloud-rolls-out-new-gemini-models-ai-agents-customer-engagement-suite
2024-09-25 08:34:34,866 - AInewsbot - INFO - get_url(https://www.constellationr.com/blog-news/insights/google-cloud-rolls-out-new-gemini-models-ai-agents-customer-engagement-suite) - starting get_url https://www.constellationr.com/blog-news/insights/google-cloud-rolls-out-new-gemini-models-ai-agents-customer-engagement-suite
2024-09-25 08:34:39,755 - AInewsbot - INFO - get_url(Manchester-based Vsim, which is building a robotics simulation service, raised a $21.5M seed from EQT Ventures and others, taking its total funding to

2024-09-25 08:35:00,780 - AInewsbot - INFO - Processing https://www.itpro.com/business/business-strategy/in-the-age-of-ai-finding-staff-with-soft-skills-has-become-a-critical-enterprise-focus
2024-09-25 08:35:00,786 - AInewsbot - INFO - get_url(https://www.itpro.com/business/business-strategy/in-the-age-of-ai-finding-staff-with-soft-skills-has-become-a-critical-enterprise-focus) - starting get_url https://www.itpro.com/business/business-strategy/in-the-age-of-ai-finding-staff-with-soft-skills-has-become-a-critical-enterprise-focus
2024-09-25 08:35:06,394 - AInewsbot - INFO - get_url(Google supercharges enterprise contact centers with Gemini 1.5 Flash) - Saving Google_supercharges_enterprise_contact_centers_with_Gemini_1_5_Flash_20240925_083506.html as utf-8
2024-09-25 08:35:06,396 - AInewsbot - INFO - Processing https://www.shrm.org/topics-tools/flagships/ai-hi/the-hr-revolution--parenting-ai-and-shaping-the-future-of-work
2024-09-25 08:35:06,396 - AInewsbot - INFO - get_url(https://ww

...

2024-09-25 08:39:45,488 - AInewsbot - INFO - get_url(How agentic AI could improve enterprise data operations) - Saving How_agentic_AI_could_improve_enterprise_data_operations_20240925_083945.html as utf-8
2024-09-25 08:39:45,489 - AInewsbot - INFO - Processing https://www.bloomberg.com/news/articles/2024-09-25/vietnam-premier-targets-fdi-to-fund-ai-big-data-development
2024-09-25 08:39:45,489 - AInewsbot - INFO - get_url(https://www.bloomberg.com/news/articles/2024-09-25/vietnam-premier-targets-fdi-to-fund-ai-big-data-development) - starting get_url https://www.bloomberg.com/news/articles/2024-09-25/vietnam-premier-targets-fdi-to-fund-ai-big-data-development
2024-09-25 08:39:49,043 - AInewsbot - INFO - get_url(TikTokers used AI to translate Hitler's speeches: report) - Saving TikTokers_used_AI_to_translate_Hitler_s_speeches__report_20240925_083949.html as utf-8
2024-09-25 08:39:49,044 - AInewsbot - INFO - Processing https://news.google.com/read/CBMiigFBVV95cUxNOEpKaENuemtaM05xc1AzZnR1b

2024-09-25 08:40:15,088 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMirAFBVV95cUxNVVNtN05QWWIyUHJlcmdjalFkYWZiVXZ4UmZXTVN6NklQMnBaaEdNTzhzRDFia28tRHdyWTA3STBVRzluWC1fMEdoNjdOcFVjRGtEbEJkblBBWmNULUg3WVk5ME5FakNXZ2l6ckkyZXAwNEhNTU9sNjVtNVFHcDB0VlAwTmlIbjlmTHFPcVo0VWNxd0JpVkNEcjlYRlVMNUIwOGs3bUFWYkdVQUxO) - starting get_url https://news.google.com/read/CBMirAFBVV95cUxNVVNtN05QWWIyUHJlcmdjalFkYWZiVXZ4UmZXTVN6NklQMnBaaEdNTzhzRDFia28tRHdyWTA3STBVRzluWC1fMEdoNjdOcFVjRGtEbEJkblBBWmNULUg3WVk5ME5FakNXZ2l6ckkyZXAwNEhNTU9sNjVtNVFHcDB0VlAwTmlIbjlmTHFPcVo0VWNxd0JpVkNEcjlYRlVMNUIwOGs3bUFWYkdVQUxO
2024-09-25 08:40:17,858 - AInewsbot - INFO - get_url(How to Build Custom AI Agents with Zero Coding Skills) - Saving How_to_Build_Custom_AI_Agents_with_Zero_Coding_Skills_20240925_084017.html as utf-8
2024-09-25 08:40:17,860 - AInewsbot - INFO - Processing https://mynorthwest.com/3991060/can-ai-make-video-games-more-immersive-some-studios-turn-to-ai-fueled-npcs-for-more-interaction/
2024-09-2

2024-09-25 08:40:47,851 - AInewsbot - INFO - Processing https://hackernoon.com/large-language-models-as-optimizers-meta-prompt-for-math-optimization
2024-09-25 08:40:47,852 - AInewsbot - INFO - get_url(https://hackernoon.com/large-language-models-as-optimizers-meta-prompt-for-math-optimization) - starting get_url https://hackernoon.com/large-language-models-as-optimizers-meta-prompt-for-math-optimization
2024-09-25 08:40:49,093 - AInewsbot - INFO - get_url(StackGen uses AI to auto-generate infrastructure from code. Check out the pitch deck the startup used to land a $12.3 million supersized seed round.) - Saving StackGen_uses_AI_to_auto-generate_infrastructure_from_code__Check_out_the_pitch_deck_the_startup_used_to_land_a__12_3_million_supersized_seed_round__20240925_084049.html as utf-8
2024-09-25 08:40:49,094 - AInewsbot - INFO - Processing https://www.creativeboom.com/news/pentagrams-identity-for-pienso/
2024-09-25 08:40:49,095 - AInewsbot - INFO - get_url(https://www.creativeboom.c

2024-09-25 08:41:16,389 - AInewsbot - INFO - Processing https://venturebeat.com/ai/openai-academy-launches-with-1m-in-developer-credits-for-devs-in-low-and-middle-income-countries/
2024-09-25 08:41:16,389 - AInewsbot - INFO - get_url(https://venturebeat.com/ai/openai-academy-launches-with-1m-in-developer-credits-for-devs-in-low-and-middle-income-countries/) - starting get_url https://venturebeat.com/ai/openai-academy-launches-with-1m-in-developer-credits-for-devs-in-low-and-middle-income-countries/
2024-09-25 08:41:21,535 - AInewsbot - INFO - get_url(Can AI sit there in a fleece vest?: John Mulaneys Salesforce roast was a masterclass in corporate comedy) - Saving Can_AI_sit_there_in_a_fleece_vest___John_Mulaneys_Salesforce_roast_was_a_masterclass_in_corporate_comedy_20240925_084121.html as utf-8
2024-09-25 08:41:21,542 - AInewsbot - INFO - Processing https://hackernoon.com/everything-we-know-about-prompt-optimization-today
2024-09-25 08:41:21,543 - AInewsbot - INFO - get_url(https://ha

2024-09-25 08:41:49,385 - AInewsbot - INFO - Processing https://sea.mashable.com/tech-industry/34372/time-is-up-to-regulate-ai-un-advisor-says
2024-09-25 08:41:49,386 - AInewsbot - INFO - get_url(https://sea.mashable.com/tech-industry/34372/time-is-up-to-regulate-ai-un-advisor-says) - starting get_url https://sea.mashable.com/tech-industry/34372/time-is-up-to-regulate-ai-un-advisor-says
2024-09-25 08:41:55,368 - AInewsbot - INFO - get_url(His 'Terminator' Was A Cautionary Tale About Artificial Intelligence; Now, James Cameron Joins Stability AI Board Of Directors) - Saving His__Terminator__Was_A_Cautionary_Tale_About_Artificial_Intelligence__Now__James_Cameron_Joins_Stability_AI_Board_Of_Directors_20240925_084155.html as utf-8
2024-09-25 08:41:55,369 - AInewsbot - INFO - Quit webdriver
2024-09-25 08:41:55,627 - AInewsbot - INFO - get_url(I Helped Invent Generative AI, and I Know When You're Using ChatGPT) - Saving I_Helped_Invent_Generative_AI__and_I_Know_When_You_re_Using_ChatGPT_2024

In [21]:
# summarize individual pages

def fn_summarize_pages(state: AgentState) -> AgentState:
    """
    Reads all the articles, summarizes each one using a ChatGPT prompt, and sends an email with the summaries.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent.

    """
    log("Starting summarize")
    AIdf = pd.DataFrame(state['AIdf'])
    responses = asyncio.run(fetch_all_summaries(AIdf))
    log(f"Received {len(responses)} summaries")
    response_dict = {}
    for i, response in responses:
        try:
            response_str = response["choices"][0]["message"]["content"]
            response_dict[i] = response_str
        except Exception as exc:
            print(exc)

    markdown_str = ''
    bullets = []

    for i, row in enumerate(AIdf.itertuples()):
        topics = []
        if row.cluster_name:
            topics.append(row.cluster_name)
        if row.topic_str:
            topics.append(row.topic_str)
        topic_str = ", ".join(topics)

        mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
        bullets.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
        display(Markdown(mdstr.replace("$","\\$")))
        markdown_str += mdstr

    state['bullets'] = bullets
    # Convert Markdown to HTML
    html_str = markdown.markdown(markdown_str, extensions=['extra'])
    # save bullets
    with open('bullets.md', 'w') as f:
        f.write(markdown_str)
    # send email
    log("Sending bullet points email")
    subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
    send_gmail(subject, html_str)

    return state


if DEBUG:
    _ = fn_summarize_pages(test_state)



2024-09-25 08:42:09,321 - AInewsbot - INFO - Starting summarize
2024-09-25 08:42:09,539 - AInewsbot - INFO - fetch_all_summaries - Page title: James Cameron Joins AI Company Stable Diffusion as Board Member
Social card title: James Cameron Joins Board of Stability AI in Coup for Tech Firm
Social card description: The director says that “the intersection of generative AI and CGI image creation is the next wave” of film technology, as he joins the company behind the popular Stable Diffusion AI model.

2024-09-25 08:42:10,049 - AInewsbot - INFO - fetch_all_summaries - Page title: 'Titanic' director James Cameron joins Stability AI board | Reuters
Social card title: 'Titanic' director James Cameron joins Stability AI board
Social card description: Stability AI said on Tuesday James Cameron, the director of "Titanic" and "The Terminator" movies, had joined the artificial intelligence startup's board.

2024-09-25 08:42:10,101 - AInewsbot - INFO - fetch_all_summaries - Page title: James Camer

2024-09-25 08:42:11,210 - AInewsbot - INFO - fetch_all_summaries - Page title: How to use ChatGPT advanced voice mode from OpenAI
Social card title: OpenAI just launched advanced voice mode for audio chats with ChatGPT. Here's how to use it
Social card description: If you're a paying customer, ChatGPT will respond to your spoken questions and comments quickly. But there's a limit to how much you can use it in a single day.

2024-09-25 08:42:11,257 - AInewsbot - INFO - fetch_all_summaries - Page title: What ChatGPT 'Thinks' About OpenAI Investor's AI Essay - Business Insider
Social card title: An OpenAI investor just published more than 10,300 words about the future of AI. We asked ChatGPT if it agreed.
Social card description: OpenAI's ChatGPT had a bone to pick with some of investor Vinod Khosla's bullish predictions about the future of AI and its potential risks.

2024-09-25 08:42:11,272 - AInewsbot - INFO - fetch_all_summaries - Page title: Bloomberg - Are you a robot?

2024-09-25 0

2024-09-25 08:42:12,623 - AInewsbot - INFO - fetch_all_summaries - Page title: Sam Altman says OpenAI could be "a few thousand days" from its superintelligence benchmark | Windows Central
Social card title: Sam Altman claims superintelligence might only be "a few thousand days" away from OpenAI's doorstep, but there are a lot of details to figure out
Social card description: OpenAI CEO says superintelligent AI might be "a few thousand days" from the AI firm's grasp.

2024-09-25 08:42:12,684 - AInewsbot - INFO - fetch_all_summaries - Page title: Experts Say Getting AI Tech to Scale Will Cost Us More Than Money - Business Insider
Social card title: Sam Altman believes wars will be fought over AI unless there's massive spending on infrastructure. But that comes with a cost.
Social card description: Sam Altman says the development of AI infrastructure is key to avoiding war over the tech. Experts say such development costs more than just money.

2024-09-25 08:42:12,739 - AInewsbot - INFO -

2024-09-25 08:42:13,723 - AInewsbot - INFO - fetch_all_summaries - Page title: Microsoft debuts Correction tool to combat AI hallucinations, but it's not a silver bullet for accuracy | Windows Central
Social card title: Microsoft unveils new Correction tool to address AI hallucinations but there are early concerns — "Trying to eliminate hallucinations from generative AI is like trying to eliminate hydrogen from water"
Social card description: Microsoft's new correction tool will address AI hallucinations, but experts warn there are critical issues abound.

2024-09-25 08:42:13,759 - AInewsbot - INFO - fetch_all_summaries - Page title: Revolutionizing Product Management: The Confluence of AI, NLP and Agile Methodologies | HackerNoon
Social card title: Revolutionizing Product Management: The Confluence of AI, NLP and Agile Methodologies | HackerNoon
Social card description: How AI, NLP, and Agile methodologies are transforming product management, driving data-driven decisions, customer in

2024-09-25 08:42:15,193 - AInewsbot - INFO - fetch_all_summaries - Page title: Microsoft unveils 'trustworthy AI' features to fix hallucinations and boost privacy | VentureBeat
Social card title: Microsoft unveils ‘trustworthy AI’ features to fix hallucinations and boost privacy
Social card description: Microsoft unveils new 'Trustworthy AI' features to combat hallucinations, enhance privacy, and improve security in AI systems, setting new industry standards for responsible AI development.

2024-09-25 08:42:15,216 - AInewsbot - INFO - fetch_all_summaries - Page title: Hacker plants false memories in ChatGPT to steal user data in perpetuity | Ars Technica
Social card title: Hacker plants false memories in ChatGPT to steal user data in perpetuity
Social card description: Emails, documents, and other untrusted content can plant malicious memories.

2024-09-25 08:42:15,228 - AInewsbot - INFO - fetch_all_summaries - Page title: wsj.com

2024-09-25 08:42:15,230 - trafilatura.core - WARNING -

...

[179. An Official OpenAI X Account Just Got Hacked By Crypto Scammers - Mashable](https://in.mashable.com/tech/82774/an-official-openai-x-account-just-got-hacked-by-crypto-scammers)  

 Big Tech, Crypto Scammers, Cryptocurrency, Cybersecurity, Disinformation, Economics, Finance, Fintech, Gen AI, Governance, Hacking, Ilya Sutskever, Intellectual Property, Legal Issues, OpenAI, Privacy, Privacy & Surveillance, Safety And Alignment, Scams, X Account  

- OpenAI's official X account was hacked, leading to a fraudulent post claiming to offer a new crypto token called "\$OPENAI" for users to claim.
- The scam post directed users to a phishing site to connect their crypto wallets, risking wallet drainage through malicious transactions.
- This incident is not isolated; OpenAI has faced previous hacks affecting key employees' accounts and sensitive information retrieval. 



[180. I Helped Invent Generative AI, and I Know When You're Using ChatGPT - Inc.](https://www.inc.com/joe-procopio/i-helped-invent-generative-ai-i-know-when-youre-using-chatgpt.html)  

 Authors & Writing, Big Tech, Chatbots, Chatgpt, Cognitive Science, Ethics, Gen AI, Generative AI, Ilya Sutskever, Innovation, Intellectual Property, Language Models, OpenAI, Opinion, Privacy, Science  

- The content appears to be primarily boilerplate with no substantial information provided.
- There are no specific articles or topics highlighted in the text.
- The text contains elements suggesting a login or user action is required. 



[181. His 'Terminator' Was A Cautionary Tale About Artificial Intelligence; Now, James Cameron Joins Stability AI Board Of Directors - Reddit](https://www.reddit.com/r/technology/comments/1forl8w/his_terminator_was_a_cautionary_tale_about/)  

 AI Doom, Artificial General Intelligence, Board Of Directors, Cognitive Science, Consciousness, Entertainment, Ethics, Gen AI, History, Hollywood, James Cameron, Opinion, Safety And Alignment, Science, Singularity, Society & Culture, Stability AI, Tv & Film & Movies  

- James Cameron joins the board of Stability AI, which raises questions about the implications of AI as portrayed in his film "Terminator."
- The text discusses the potential for AI systems like Stable Diffusion to develop self-awareness, albeit in a humorous context.
- It highlights skepticism regarding the realism of AI in fiction and suggests that having creative writers on an AI board may be more about connections than expertise. 



[182. "Time is up" to regulate AI, UN advisor says - Mashable](https://sea.mashable.com/tech-industry/34372/time-is-up-to-regulate-ai-un-advisor-says)  

 Advisors, AI Doom, Ethics, Gen AI, Governance, Policy And Regulation, Regulation, Safety And Alignment, Un  

- The UN emphasizes the urgent need for global cooperation and regulation of artificial intelligence (AI), highlighting the potential risks and challenges it poses, akin to nuclear weapons.
- AI nationalism, driven by intense competition among nations for technological superiority, complicates international efforts for standardized governance and ethical frameworks for AI development.
- A recent UN report recommends forming an independent international scientific panel and a Global AI fund to promote equitable investment and governance in AI, while advocating for prompt and comprehensive regulatory responses before the opportunity window closes. 



2024-09-25 08:42:37,702 - AInewsbot - INFO - Sending bullet points email


In [23]:
def fn_propose_cats(state: AgentState) -> AgentState:
    # ask chatgpt for top categories

    model = ChatOpenAI(
        model=MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template("{p}") | model | SimpleJsonOutputParser()
    response = chain.invoke({ "p": TOP_CATEGORIES_PROMPT + "\n\n".join(state["bullets"])})
    suggested_categories = []
    for k, v in response.items():
        suggested_categories.extend(v)
    state["cluster_topics"] = list(set(state["cluster_topics"] + suggested_categories))
    state["cluster_topics"].sort()
    return state


if DEBUG:
    _ = fn_propose_cats(test_state)
    print("edit the following proposed topic list and update state['cluster_topics']:")
    print('"' + '",\n"'.join(test_state["cluster_topics"]) + '"')


2024-09-25 08:42:55,147 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


edit the following proposed topic list and update state['cluster_topics']:
"AI Hardware and Semiconductor Innovations","
"AI Investment and Economic Trends","
"AI PCs to dominate shipments by 2026","
"AI and Hardware Market Trends","
"AI and Venture Capital Funding","
"AI and the Future of Work","
"AI in Healthcare","
"AI in Streaming and Entertainment","
"AI startups brace for down rounds","
"AI's environmental impact debate","
"AI's growing influence on video games","
"AI's impact on healthcare communication","
"AI's influence on global politics","
"AI's role in cybersecurity","
"AI's role in deepfake porn crisis in South Korea","
"AI's role in enterprise data operations","
"AI's role in future medicine","
"AI's role in political ad regulation","
"AI's role in stock market predictions","
"AI-generated malware found in the wild","
"Chatbots and AI Limitations","
"Google Gemini and AI Innovations","
"Google integrates Gemini AI into Workspace","
"Hollywood supports California AI safety

In [27]:
test_state["cluster_topics"] = [
"AI Chips",
"AI Investment Trends",
"AI PCs",
"Venture Capital",
"AI and the Future of Work",
"AI in Healthcare",
"AI in Streaming and Entertainment",
"AI startups brace for down rounds",
"AI's environmental impact",
"AI's in video games",
"AI's in politics",
"AI's in cybersecurity",
"Deepfake porn crisis in South Korea",
"Enterprise IT",
"AI in the stock market",
"AI-generated malware",
"AI Limitations",
"Google Gemini",
"Intel",
"James Cameron Joins Stability AI",
"Meta's Celebrity Voice Chatbots",
"OpenAI",
"Sam Altman"
]


In [28]:
FINAL_SUMMARY_PROMPT = """You are ASA, an advanced summarization assistant, a sophisticated AI system designed to
write a compelling summary of news input. You are able to categorize information,
and identify trends from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in a markdown format,
structured according to an input format template.

News items  are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - TechCrunch](https://techcrunch.com/2024/09/19/apple-intelligence-is-now-live-in-public-beta-heres-what-it-offers-and-how-to-enable-it)

Topics: Apple, Big Tech, Features, Gen AI, Intelligence, Machine Learning, Products, Public Beta, Virtual Assistants

- Apple Intelligence is now live in public beta for users in the U.S. enrolled in the public beta program, featuring generative AI capabilities like advanced writing tools and a revamped Siri.
- The platform is currently only available in U.S. English and is not accessible in the EU or China due to regulatory issues; it supports iPhone 15 Pro, Pro Max, and the new iPhone 16 line.
- Key features include photo editing tools like "Clean Up," a Smart Reply function in Mail, and improvements to Siri’s understanding and on-device task knowledge.

ASA Output Format Template:

# Engaging-topic-title-1

- item-title-1a - [source-name-1a](item-url-1a)
- item-title-1b - [source-name-1b](item-url-1b)
- item-title-1c - [source-name-1c](item-url-1c)

# Engaging-topic-title-2

- item-title-2a - [source-name-2a](item-url-2a)
- item-title-2b - [source-name-2b](item-url-2b)

Example ASA Output Format:

# A military AI revolution

- Eric Schmidt on AI warfare - [FT](https://www.ft.com/content/fe136479-9504-4588-869f-900f2b3452c4)
- Killer robots are real in Ukraine war. - [Yahoo News](https://uk.news.yahoo.com/ai-killer-robots-warning-ukraine-war-133415411.html)

ASA Instructions:
Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Group news items into related topics.
Each topic should have a snappy, punchy, clever, alliterative, possibly punny title.
Each output item bullet should contain one sentence with one link.
Each topic chould contain the most significant facts from the news items without commentary or elaboration.
Each output item bullet should not repeat points or information from previous bullet points.
You will write each item in the professional but engaging, narrative style of a tech reporter
for a national publication, providing balanced, professional, informative, providing accurate,
clear, concise summaries in a neutral tone.

Check carefully that you only use information provided in the input below, that you include
a link in each output item, and that any bullet point does not repeat information or links previously provided.

Topic suggestions:
{cat_str}

Input:
{bullet_str}

"""

In [29]:
def fn_compose_summary(state: AgentState) -> AgentState:

    cat_str = "\n".join(state['cluster_topics'])
    bullet_str = "\n~~~\n".join(state["bullets"])

    model = ChatOpenAI(
        model=HIGHCOST_MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template(FINAL_SUMMARY_PROMPT) | model | SimpleJsonOutputParser()
    response = chain.invoke({ "cat_str": cat_str, "bullet_str": bullet_str})
    print(response)
    state["summary"] = response.content
    return state


if DEBUG:
    _ = fn_compose_summary(test_state)


2024-09-25 09:19:11,397 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported value: 'temperature' does not support 0.3 with this model. Only the default (1) value is supported.", 'type': 'invalid_request_error', 'param': 'temperature', 'code': 'unsupported_value'}}

In [38]:
bullet_str = """You are ASA, an advanced summarization assistant, a sophisticated AI system designed to
write a compelling summary of news input. You are able to categorize information,
and identify trends from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in a markdown format,
structured according to an input format template.

News items  are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - TechCrunch](https://techcrunch.com/2024/09/19/apple-intelligence-is-now-live-in-public-beta-heres-what-it-offers-and-how-to-enable-it)

Topics: Apple, Big Tech, Features, Gen AI, Intelligence, Machine Learning, Products, Public Beta, Virtual Assistants

- Apple Intelligence is now live in public beta for users in the U.S. enrolled in the public beta program, featuring generative AI capabilities like advanced writing tools and a revamped Siri.
- The platform is currently only available in U.S. English and is not accessible in the EU or China due to regulatory issues; it supports iPhone 15 Pro, Pro Max, and the new iPhone 16 line.
- Key features include photo editing tools like "Clean Up," a Smart Reply function in Mail, and improvements to Siri’s understanding and on-device task knowledge.

ASA Output Format Template:

# Engaging-topic-title-1

- item-title-1a - [source-name-1a](item-url-1a)
- item-title-1b - [source-name-1b](item-url-1b)
- item-title-1c - [source-name-1c](item-url-1c)

# Engaging-topic-title-2

- item-title-2a - [source-name-2a](item-url-2a)
- item-title-2b - [source-name-2b](item-url-2b)

Example ASA Output Format:

# A military AI revolution

- Eric Schmidt on AI warfare - [FT](https://www.ft.com/content/fe136479-9504-4588-869f-900f2b3452c4)
- Killer robots are real in Ukraine war. - [Yahoo News](https://uk.news.yahoo.com/ai-killer-robots-warning-ukraine-war-133415411.html)

ASA Instructions:
Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Group news items into related topics.
Each topic should have a snappy, punchy, clever, alliterative, possibly punny title.
Each output item bullet should contain one sentence with one link.
Each topic chould contain the most significant facts from the news items without commentary or elaboration.
Each output item bullet should not repeat points or information from previous bullet points.
You will write each item in the professional but engaging, narrative style of a tech reporter
for a national publication, providing balanced, professional, informative, providing accurate,
clear, concise summaries in a neutral tone.

Check carefully that you only use information provided in the input below, that you include
a link in each output item, and that any bullet point does not repeat information or links previously provided.

Topic suggestions:
AI Chips
AI Investment Trends
AI PCs
Venture Capital
AI and the Future of Work
AI in Healthcare
AI in Streaming and Entertainment
AI startups brace for down rounds
AI's environmental impact
AI's in video games
AI's in politics
AI's in cybersecurity
Deepfake porn crisis in South Korea
Enterprise IT
AI in the stock market
AI-generated malware
AI Limitations
Google Gemini
Intel
James Cameron Joins Stability AI
Meta's Celebrity Voice Chatbots
OpenAI
Sam Altman

Input:

~~~
[AI PCs will dominate shipments by 2026, but not because of demand - The Register](https://go.theregister.com/feed/www.theregister.com/2024/09/25/analysts_ai_pcs_shipments_gartner/)

Topics: Economics, Gen AI, Hardware, Market Trends, Pcs, Products, Shipments 

- AI PCs are expected to dominate PC shipments by 2026, accounting for 43% of the market by 2025, significantly increasing from previous years.
- The shift towards AI PCs is driven by the decreasing availability of computers without AI silicon rather than a specific killer application.
- Analysts note that while businesses see the value of AI, many struggle with immediate use cases, using AI PCs primarily for future-proofing.


~~~
[AI GPU Clusters, from Your Laptop, with Livebook - fly.io](https://fly.io/blog/ai-gpu-clusters-from-your-laptop-livebook/)

Topics: Gen AI, Gpu Clusters, Hardware, Laptops, Livebook, Nvidia, Products, Science 

- Livebook, FLAME, and Nx are Elixir components that enable powerful AI scaling from a laptop, allowing seamless transitions between local and remote computations.
- FLAME provides a serverless computing experience by managing application resource pooling, enabling developers to execute code elastically across multiple compute nodes without complex infrastructure setups.
- Livebook can start a runtime on Fly.io's cloud, connecting to existing Elixir applications for real-time monitoring while supporting data processing and AI tasks, such as hyperparameter tuning with models like BERT.


~~~
[AI PCs will dominate shipments by 2026, but not because of demandBuying computers without the tech to get harder as chipmakers compete for cash, say analystsAI + ML4 hrs|13 - The Register](https://www.theregister.com/2024/09/25/analysts_ai_pcs_shipments_gartner/)

Topics: Chipmakers, Economics, Finance, Gen AI, Hardware, Market Competition, Pcs, Products, Semiconductor Chips 

- Analysts forecast that AI PCs will dominate the market, accounting for 43% of PC shipments by 2025 and potentially becoming the only laptops available for businesses by 2026.
- Shipments of AI PCs are projected to increase dramatically, with a near doubling to 43 million units in 2024 and reaching 114 million units in 2025.
- The trend towards AI PCs is driven by the increasing necessity of integrated neural processing units (NPUs), despite a lack of immediate killer applications, leading businesses to purchase them for future-proofing.


~~~
[AI startups should brace for a wave of down rounds, investors say. Here's how it might play out. - Business Insider](https://www.businessinsider.com/ai-down-rounds-rise-valuations-investors-2024-9)

Topics: Down Rounds, Economics, Finance, Funding, Gen AI, Investment, Jobs & Careerslabor Market, Opinion, Startups, Stocks, Venture Capital 

- AI startups have raised over $35 billion in 2024, but many are experiencing down rounds and flat valuations as investor optimism declines.
- The trend of acquihires and secondary share sales is increasing, as startups struggle to justify inflated valuations from previous funding rounds.
- Data shows a rise in down rounds from 6.5% in 2022 to 11.4% in 2024, reflecting a challenging environment for AI startups heavily investing in costly infrastructure without sufficient revenue.


~~~
[Machine learning reveals factors for successful crowdfunding - Phys.org](https://phys.org/news/2024-09-sweet-machine-reveals-factors-successful.html)

Topics: Crowdfunding, Economics, Finance, Funding, Machine Learning, Science, Success Factors, Venture Capital 

- Machine learning techniques, particularly Deep Learning, have proven superior in predicting crowdfunding success compared to traditional statistical methods, facilitating the identification of key success factors in campaigns.
- Analysis of over 100,000 Kickstarter projects indicated that factors such as campaign monetary goal, social capital, reward options, and duration significantly influence the chances of reaching funding goals.
- Optimal campaign strategies include setting monetary goals below $100,000, a project duration of 10-15 days, and offering around 15 reward options, while avoiding domains like "gadgets" that exhibit lower likelihoods of success.


~~~
[Q&A with Vinod Khosla on investing in AI and OpenAI, AI risks, the impact of AI in the labor market, regulation, dominance within the global AI race, and more - Time](https://time.com/7023237/vinod-khosla-interview/)

Topics: AI Doom, Artificial General Intelligence, Bias And Fairness, Big Tech, Consciousness, Economics, Ethics, Finance, Gen AI, Governance, India, Investment, Job Automation, Jobs & Careerslabor Market, Language Models, Legal Issues, OpenAI, Opinion, Policy And Regulation, Politics, Regulation, Safety And Alignment, Science, Society & Culture, Venture Capital, Vinod Khosla 

- Vinod Khosla advocates for AI's potential to replace skilled jobs, proposing a future where AI assists professionals, thereby increasing efficiency and reducing labor demand.
- Khosla emphasizes the need for caution in AI development and regulation, balancing innovation with safety and the risks involved, particularly regarding explainability and AI's societal impact.
- He believes that national models of AI are necessary for various countries to ensure independence, and he expresses optimism about AI becoming accessible and >equitable globally, akin to the evolution of internet technology.


~~~
[About 15,000 US doctors and assistants are using an AI feature in MyChart, a ubiquitous communications platform at US hospitals, to draft messages for patients (Teddy Rosenbluth/New York Times) - The New York Times](https://www.nytimes.com/2024/09/24/health/ai-patient-messages-mychart.html)

Topics: Customer Service, Doctors, Gen AI, Health & Fitness, Healthcare, Job Automation, Jobs & Careerslabor Market, Mychart, Patient Communication, Science, Society & Culture, Speech Recognition & Synthesis, Virtual Assistants 

- Physicians are increasingly using artificial intelligence to draft responses to patient messages in order to manage the high volume of inquiries.
- Many patients may be unaware that the replies they receive are generated by AI rather than directly from their doctors.
- The AI feature is currently being utilized by approximately 15,000 doctors and assistants across more than 150 health systems, integrated into the MyChart platform.


~~~
[How AI is shaping the future of medicine - Fast Company](https://www.fastcompany.com/91196757/how-ai-is-shaping-the-future-of-medicine)

Topics: Cognitive Science, Future, Gen AI, Health & Fitness, Healthcare, Job Automation, Jobs & Careerslabor Market, Medicine, Science 

- AI is significantly enhancing patient care through improvements in medical imaging, drug discovery, and personalized medicine, with tools like Optomed’s portable Fundus camera enabling early disease detection.
- The integration of AI in healthcare operations improves efficiency by optimizing resource management and enhancing decision-making, while also introducing AI-driven solutions like chatbots for better patient engagement.
- Challenges such as data quality, privacy concerns, and potential biases must be addressed to ensure equitable and effective AI solutions in healthcare.


~~~
[I use an AI assistant as a doctor and cut hours off my workload. But, I'm still cautious about AI's future. - Business Insider](https://www.businessinsider.com/doctor-ai-assistant-saves-time-reduces-stress-2024-9)

Topics: AI Doom, Assistant, Bias And Fairness, Caution, Code Assistants, Doctor, Ethics, Gen AI, Health & Fitness, Healthcare, Job Automation, Opinion, Privacy, Safety And Alignment, Science, Society & Culture, Virtual Assistants, Workload 

- Dr. Jennifer J. Bryan has significantly reduced her work hours and stress by using Suki AI for note-taking, cutting down her documentation time from 10-20 hours a week to about 15 minutes a day.
- She believes AI can assist in medical transcription but is cautious about its role in other areas of healthcare, emphasizing the importance of human interaction and the potential for AI errors affecting patient safety.
- While she appreciates the benefits of AI in her workflow, she maintains that it will not replace doctors, although it may diminish the need for medical scribes.


~~~
[Intel launches Xeon 6 and Gaudi 3 AI chips to boost AI and HPC performance - VentureBeat](https://venturebeat.com/ai/intel-launches-xeon-6-and-gaudi-3-ai-chips-to-boost-ai-and-hpc-performance/)

Topics: Big Tech, Gaming, Gaudi 3, Hardware, Hpc Performance, Intel, Manufacturing, Nvidia, Products, Science, Semiconductor Chips, Xeon 6 

- Intel has launched the Xeon 6 processors with performance cores and Gaudi 3 AI accelerators to enhance AI and HPC performance, significantly improving efficiency and throughput.
- The Xeon 6 features double the performance of its predecessor, while Gaudi 3 is optimized for large-scale generative AI tasks and offers advanced capabilities for deep neural network computations.
- Intel emphasizes a flexible ecosystem for AI deployment, partnering with OEMs for tailored solutions, and providing access to the Intel Tiber portfolio for improved AI application deployment and testing.


~~~
[Intel launches Gaudi 3 accelerator for AI: Slower than Nvidia's H100 AI GPU, but also cheaper - Google News](https://news.google.com/read/CBMi2gFBVV95cUxOS2pLUThfejc5d0t2aVlybVdvOW9KWHRYTHd1cEczY1lKWFFlWnF1QWtSejMwQUZ5OUp4TEpVaE5rWjdqQUxSMWxhV3VGXzBIWjh1T0hxbzdJX3VyUFVyQkg5Wk91R0ZOYlJZWk5zMmVHMTdiU1VUTEJHRU9EWndxbkcyR3BRTW1QaXprTDVwOU9mNWk1X25PR3kySC1TUU0xLUF0cVJLWDMxb0RfMFQzQXpRYVRFVUxrdS1hWl96UnpEdVY5RF9yLVBtTmxNd1B0TDM4cExMSFZpdw)

Topics: Big Tech, Gaming, Gaudi 3, Gpu, Hardware, Intel, Manufacturing, Nvidia, Products, Science, Semiconductor Chips 

- Intel has launched the Gaudi 3 AI accelerator, which is slower than Nvidia's H100 but aims to compete on lower price and total cost of ownership.  
- The Gaudi 3 features improvements over its predecessor, including 64 tensor processor cores and a memory capacity of 128GB, while simplifying some operation supports to FP8 and BFloat16.  
- Intel claims Gaudi 3 provides up to 20% more throughput and a 2x price/performance ratio compared to Nvidia's H100 for certain applications, with broader availability expected by the fourth quarter.  


~~~
[Altera brings more AI to the edge and cloud with new programmable chips - VentureBeat](https://venturebeat.com/ai/altera-brings-more-ai-to-the-edge-and-cloud-with-new-programmable-chips/)

Topics: Big Tech, Cloud Computing, Gen AI, Hardware, Intel, Neuromorphic Computing, Products, Programmable Chips, Science, Semiconductor Chips 

- Altera has launched new FPGA products, including Agilex 3 FPGAs aimed at enhancing AI capabilities for edge and cloud applications, featuring integrated security enhancements and improved performance.  
- The company aims to be the leading FPGA provider globally, focusing on delivering optimized solutions for diverse markets like data centers, automotive, and industrial applications.  
- Upcoming software updates, such as the Quartus Prime Pro 24.3 release, will expedite FPGA design and development, providing the ecosystem with improved tools and resources for faster time-to-market.  


~~~
[OpenAI Rolls Out New Voice Assistant to All Paid ChatGPT Users - Google News](https://news.google.com/read/CBMiswFBVV95cUxOc1E5YXFEMFAzamZqOHB0bm53UERBeXNmUUl1cXZncEg3QjZBanhSUk5EOWkzVDBmTEs3bG9MX3M3akdXRkEzN1d5TEpYWU43TXJZU1pXLUFFVDVkcUQycFdrWnVwTFAweXozQmNKYXJzXzFmMTRzbEZCbTR1YTlfallseXFyZndzV24yei05QWZsM1JRN1dRN1o3RE1qY0h4N0hRNnpTN01fcFVXX2pJWFc4UQ)

Topics: Big Tech, Chatbots, Chatgpt, Claude, Code Assistants, Customer Service, Gen AI, Language Models, Music, OpenAI, Products, Rollout, Speech Recognition & Synthesis, Virtual Assistants, Voice Assistant 

- Content primarily consists of a verification prompt asking users to confirm they are not a robot.
- Instructions are provided for ensuring browser compatibility with JavaScript and cookies.
- Contact information for support inquiries is included, along with a reference ID.


~~~
[Duolingo unveils an interactive AI feature to video call with its Lily mascot, as part of its $30/month Duolingo Max tier; the dialogue is generated by GPT-4o - Forbes](https://www.forbes.com/sites/richardnieva/2024/09/24/duolingo-luis-von-ahn-billionaire-ai-tutor/)

Topics: AI Feature, Big Tech, Chatbots, Chatgpt, Cognitive Science, Deals, Duolingo, Education, Entertainment, Gen AI, Language Models, Lily Mascot, OpenAI, Products, Science, Speech Recognition & Synthesis, Video Calls, Virtual Assistants 

- Luis von Ahn, founder of Duolingo, is focused on integrating AI into the platform to create an automated AI tutor, which he believes could enhance access to language learning despite potential job losses in translation and lesson writing.
- Duolingo has introduced new AI features including interactive video calls with AI characters, aimed at providing personalized learning experiences, and is experiencing significant user growth and revenue increases.
- Von Ahn expresses concerns about the broader implications of AI on job markets and education, advocating for smart regulation to ensure equitable access, while preparing for potential backlash from the deployment of AI technologies.


~~~
[OpenAI Begins Rollout of Advanced Voice to All Plus and Team Subscribers - Google News](https://news.google.com/read/CBMixgFBVV95cUxNRVlqLVFWYVVrVkppaUY1Nk56STVoQVlJVHBISUlkQ3Zyc1BLdGJNa1NNN1VZMVN1QXBtMEo1ODRVSXNXVTZVbDVBbnN4VTBzdVhoMGtkb2lCUVRKWVZEN1o2SV9hdTJRWUtNYW45TFJKb2hCLUpBNDNrSEFPV0kxTEQ3LTlVNjhjQnlDRmVkNWFTVW80cWZkM09KYUpHbnZVNS1IZGtrdWpMSVZSMFNlV1hVcGM4T3JTNkZZOV9TOHZOMVNmYmc)

Topics: Big Tech, Chatgpt, Customer Service, Gen AI, Language Models, Music, OpenAI, Products, Speech Recognition & Synthesis, Subscription, Virtual Assistants, Voice Assistant 

- OpenAI is rolling out its Advanced Voice feature to all Plus and Team subscribers, offering nine lifelike voices and improved user experience.
- The feature is currently unavailable in the EU, UK, and select European countries, and includes safeguards against misuse, such as filters to block copyrighted content requests.
- The restaurant industry is increasingly adopting digital payment technologies to enhance customer experience and meet evolving consumer demands, signaling a shift towards more seamless and efficient operations.


~~~
[OpenAI rolls out Advanced Voice Mode with more voices and a new look - TechCrunch](https://techcrunch.com/2024/09/24/openai-rolls-out-advanced-voice-mode-with-more-voices-and-a-new-look/)

Topics: Art & Design, Big Tech, Chatgpt, Claude, Code Assistants, Customer Service, Features, Gen AI, Language Models, Music, OpenAI, Products, Speech Recognition & Synthesis, Virtual Assistants, Voice Mode 

- OpenAI is launching Advanced Voice Mode (AVM) for ChatGPT's Plus and Teams customers, featuring a new design and five additional voices, totaling nine options inspired by nature.
- The rollout also includes enhancements such as improved accent recognition, Custom Instructions, and Memory for personalized interactions, but some features like video and screen sharing are not yet available.
- AVM will not be accessible in several regions, including the EU and the U.K., and has shown improvements since its alpha test despite initial glitches.


~~~
[OpenAI finally brings humanlike ChatGPT Advanced Voice Mode to U.S. Plus, Team users - VentureBeat](https://venturebeat.com/ai/openai-finally-brings-humanlike-chatgpt-advanced-voice-mode-to-u-s-plus-team-users/)

Topics: Big Tech, Chatbots, Chatgpt, Claude, Code Assistants, Customer Service, Gen AI, Language Models, Music, OpenAI, Products, Speech Recognition & Synthesis, U.S. Rollout, Virtual Assistants, Voice Mode 

- OpenAI has launched the ChatGPT Advanced Voice Mode, which allows users to interact with the chatbot using a more human-like conversational voice interface, initially available to ChatGPT Plus and Team subscribers in the U.S.
- The update introduces nine total voices, including five new options, and enhances personalization with features like "custom instructions" and "memory" for user preferences.
- Despite previous delays and controversies regarding voice likenesses to real individuals, OpenAI emphasized extensive safety testing and is now expanding the availability of AI voice capabilities across its platforms.


~~~
[OpenAI just launched advanced voice mode for audio chats with ChatGPT. Here's how to use it - Google News](https://news.google.com/read/CBMikAFBVV95cUxNaGJxMFd1VVlxRUVQWEhoX0YtaWxtVTdQU0FZaFl0a2RHRVlaN3p0MFRDd3owSEdPS0lUMVpjWWpuV3d2VjJUNEhQMW9RaHctaVRHeXRCQjU4S1BzYUpDQU41TWltandpMlFRczNZSjdpUkVWZTBhMzVNalNMRzNWdWNRWjdMZXlqSDQzT2xwLUnSAZYBQVVfeXFMTlpDS2kwdHQxMWpqM3BYVl9vRFFNamhsTllXLTNOVnNuLThobVVJUXFtcGJ5SW9aODZ6Q25mRGYtRDhNd3hHdlR2NmhvZ1ZXMkVXWElIWVNmYWQxZmgwWlZab2JFejViQnNwbFo3b0lyTnpuUE5NeEtOT2ZuM2p6UXEtRUFmUlhhZWdjQ1JiVjBKOF9lamJn)

Topics: Audio Chats, Big Tech, Chatbots, Chatgpt, Claude, Code Assistants, Customer Service, Gen AI, Language Models, Music, OpenAI, Products, Review, Speech Recognition & Synthesis, Virtual Assistants, Voice Mode 

- OpenAI has launched an advanced voice mode for ChatGPT, allowing premium subscribers to engage in more fluid audio conversations with faster response times and nine voice options.
- The feature is currently unavailable in certain EU countries and has usage limits, with notifications displayed when time is close to expiring.
- Competitors like Google and Meta are also introducing voice features, making the AI chatbot market increasingly competitive.


~~~
[An OpenAI investor just published more than 10,300 words about the future of AI. We asked ChatGPT if it agreed. - Business Insider](https://www.businessinsider.com/vinod-khosla-ai-essay-predictions-chatgpt-response-2024-9)

Topics: Big Tech, Chatbots, Chatgpt, Claude, Future Of AI, Gen AI, Investor, Language Models, OpenAI, Opinion, Science, Venture Capital 

- Vinod Khosla published a comprehensive essay discussing the optimistic future of AI, highlighting potential benefits such as improved healthcare and job redefinitions, while acknowledging some dystopian risks.
- OpenAI's ChatGPT model agreed with Khosla on several transformative potential aspects of AI but expressed skepticism about certain overly optimistic predictions and emphasized the underestimated risks involved in AI integration.
- ChatGPT advocated for a balanced perspective on AI development, stressing the need to navigate challenges and moral dilemmas responsibly for a more equitable future.


~~~
[Ujet, which uses AI to help businesses with customer support, raised $76M led by Sapphire Ventures, valuing the company at $500M, and promotes its COO to co-CEO - Bloomberg](https://www.bloomberg.com/news/articles/2024-09-24/customer-support-startup-ujet-gets-new-co-ceo-76-million-in-funding)

Topics: Big Tech, Customer Service, Customer Support, Deals, Economics, Finance, Funding, Gen AI, Ipos, Products, Sapphire Ventures, Science, Stocks, Ujet, Venture Capital, Virtual Assistants 

- The page prompts users to confirm they are not a robot by clicking a box.
- Users are advised to ensure their browser supports JavaScript and cookies, and that they are not blocking them.
- Further inquiries regarding this verification process can be directed to the support team with a reference ID provided.


~~~
[Coatue-backed Distyl AI founded by Palantir alums, raises $20 million funding round, sources say - Business Insider](https://www.businessinsider.com/coatue-backed-distyl-ai-raises-new-funding-round-2024-9)

Topics: Big Tech, Deals, Distyl AI, Economics, Finance, Funding, Gen AI, Investment, Palantir, Venture Capital 

- Distyl AI, founded by former Palantir employees, has raised $20 million in Series A funding, valuing the company at over $200 million.
- The startup focuses on developing AI-assisted workflows for Fortune 100 companies across various sectors, addressing integration and scalability challenges.
- Distyl AI has a strategic partnership with OpenAI to leverage its research and infrastructure for enhancing their AI solutions.


~~~
[Myntra founder Mukesh Bansal's Nurix AI, which builds custom AI agents for enterprise services, raises a $27.5M seed from Accel, General Catalyst, and others - The Economic Times](https://economictimes.indiatimes.com/tech/artificial-intelligence/mukesh-bansals-new-startup-nurix-ai-raises-27-5-million-from-accel-general-catalyst/articleshow/113606177.cms)

Topics: Big Tech, Code Assistants, Deals, Economics, Enterprise Services, Finance, Funding, Gen AI, India, Myntra, Nurix AI, Venture Capital, Virtual Assistants 

- Mukesh Bansal's new startup, Nurix AI, has raised $27.5 million in funding from Accel and General Catalyst.
- Nurix AI focuses on creating custom AI agents aimed at improving enterprise services like sales and customer support.
- The startup was incubated in Bansal's venture, Meraki Labs, and currently employs around 20 people.


~~~
[Manchester-based Vsim, which is building a robotics simulation service, raised a $21.5M seed from EQT Ventures and others, taking its total funding to $24M - TechCrunch](https://techcrunch.com/2024/09/25/vsim-founded-by-nvidia-alums-raises-21-5m-for-robotics-simulation-tech/)

Topics: Deals, Economics, Eqt Ventures, Finance, Funding, Hardware, Job Automation, Manchester, Manufacturing, Robotics, Robots, Simulation, Uk, Venture Capital 

- Vsim, co-founded by former Nvidia engineers, has raised $24 million in seed funding to develop a new physics simulation framework, focusing initially on robotics training.
- The startup's technology aims to enhance simulation capabilities for various applications, addressing a significant gap in the market for real-time, high-performance simulations in dynamic environments.
- The funding round involved notable investors including EQT Ventures and Factorial Fund, valuing Vsim at approximately $100 million.


~~~
[Google plans to include the standalone Gemini app as standard in its Workspace Business, Enterprise, and Frontline plans from Q4, replacing the Gemini add-on - The Verge](https://www.theverge.com/2024/9/24/24252934/google-workspace-gemini-ai-security-assistant)

Topics: Big Tech, Business Plans, Deepmind, Gemini, Gen AI, Google, Language Models, Products, Virtual Assistants, Workspace 

- Google is integrating its Gemini AI chatbot into the Workspace productivity suite for Business, Enterprise, and Frontline plans, making it a standard feature by Q4.
- Gemini AI will adhere to enterprise standards, ensuring it does not use organizational data to improve its model, with Workspace Administrators able to manage data storage options.
- The Workspace suite will also include new security features, such as the "Security Advisor" tool, which provides insights via email to IT administrators, enhancing protection against online threats.


~~~
[Google Cloud debuts Gemini 1.5 Flash and 1.5 Pro with a 2M context window, twice as big as before, grounding for better Google search accuracy, and new agents - Constellation Research](https://www.constellationr.com/blog-news/insights/google-cloud-rolls-out-new-gemini-models-ai-agents-customer-engagement-suite)

Topics: Big Tech, Chatbots, Deepmind, Demis Hassabis, Gemini, Gemini 1.5, Gen AI, Google, Google Cloud, Language Models, Products, Science, Search Accuracy, Virtual Assistants 

- Google Cloud has introduced Gemini 1.5 Flash and 1.5 Pro models, enhancing AI agents for customer engagement with features like a 2 million context window and Google search grounding.
- CEO Thomas Kurian highlighted the focus on real-world engagement in AI agents, emphasizing customer experience across various platforms, including web and call centers.
- Google Cloud is promoting the versatility of AI agents through a mix of pre-built and custom solutions, catering to enterprise needs while addressing change management and delivering tangible business value.


~~~
[Googles Gemini AI might soon appear in your corporate Workspace - Google News](https://news.google.com/read/CBMikwFBVV95cUxQdnlXX24yVHJFSldjTHdZM2E1bElnMEd5NmE2VGVHdWJmalY0V3AyM1VKSndtMmhFamtQdUd2N2JTOGlwa3VmcW1YWTdDX1ZiWEpnZXdqN3VHMnhhWDBOZV95R1VMVDc5Sm5EdFhMSl9uME9INmlDSlJSYUtpX3BWTW9mN0R1VDFPY0tiQzY0WVdTSW8)

Topics: Big Tech, Corporate, Deepmind, Gemini, Gemini AI, Gen AI, Google, Language Models, Products, Virtual Assistants, Workspace 

- Google is integrating Gemini AI as a standard feature in its Workspace productivity suite, available for Business, Enterprise, and Frontline plans starting Q4.
- Gemini AI will adhere to enterprise terms for core Workspace services, ensuring data privacy by not using organizational data to train the model.
- A new "Security Advisor" tool will be introduced to enhance security for Workspace users, providing protection against threats and insights directly to IT administrators.


~~~
[Google brings Gemini AI chat app to Workspace, boosting productivity for millions of enterprise users - VentureBeat](https://venturebeat.com/ai/google-adding-gemini-chat-app-to-workspace/)

Topics: Big Tech, Chat App, Chatbots, Deepmind, Demis Hassabis, Economics, Gemini, Gemini AI, Gen AI, Google, Job Automation, Jobs & Careerslabor Market, Language Models, Productivity, Products, Virtual Assistants, Workspace 

- Google has integrated the Gemini AI chat app into its Workspace suite, allowing enterprise users to ask questions about company data while ensuring security and compliance.
- Workspace users in Business, Enterprise, and Frontline plans will automatically gain access to Gemini, which can enhance productivity by suggesting actions based on company policies.
- Recent surveys indicate that 75% of users report improvements in work quality, with an average time savings of 105 minutes weekly due to Gemini’s capabilities.


~~~
[Google Gemini could soon narrate your Google Photos memories - Tom's Guide](https://www.tomsguide.com/phones/android-phones/google-gemini-could-soon-narrate-your-google-photos-memories)

Topics: Big Tech, Deepmind, Demis Hassabis, Entertainment, Gemini, Gen AI, Google, Google Gemini, Google Photos, Language Models, Narration, Products, Speech Recognition & Synthesis, Virtual Assistants 

- Google Gemini is set to enhance Google Photos' Memories feature by adding AI-generated narration to the slideshow recaps of user images.
- Users will have the option to enable or disable the narration feature, with an anticipated release in December.
- The update also allows for greater user control over which memories are displayed by limiting content related to certain people or places.


~~~
[Google just dropped new versions of Gemini  here's why its a big deal - Google News](https://news.google.com/read/CBMingFBVV95cUxNT3BfZmdoUkd4eVBzeEtKZTVwR1kxX2lMUnZlY0wwbklHZjY0RGZmcFBtbkdEcWdvMmJVMFJwOER2SUtnbVN3a3NFSkR5SWlkXy1YRXh5ZU4tNFRySm9jeWhwT3UySDhsUkFQVVlnektFN05ENEozZXp0d3RNZG5XZnlQZ1hkQUFRZE1yM0dVY29heTQ5OTJYRlRSS0pnQQ)

Topics: Big Tech, Deepmind, Gemini, Gen AI, Google, Language Models, Products, Updates 

- Google released two new versions of its Gemini AI models, Gemini-1.5-Pro-002 and Gemini-1.5-Flash-002, featuring a 50% cost reduction, twice the output speed, and three times lower latency compared to previous versions.
- Enhancements include a 20% improvement in math tasks, better visual understanding, and optimized performance for high-demand AI tasks, while developers can expect streamlined access via Google AI Studio and Vertex AI.
- New privacy filters and configurable models enhance safety, and a chat-optimized version of Gemini 1.5 Pro-002 is expected for Gemini Advanced users soon.


~~~
[Customers are putting Gemini to work - Google News](https://news.google.com/read/CBMidkFVX3lxTFBUZ2xxa2lQUWp4MnFuR2Q5anFMNzNOVVYyRXdmRWdva2xMSHpud1poWU9SdFluTUZmcEh2d0JLQkc4NXlEbXJ3MmJvZ0V1WnFNX1FCbW80Q2xTUXo2MF9lblVGTEVhcFNxSU5wTFF3SHItNWlCYXc)

Topics: Big Tech, Cryptocurrency, Customer Service, Customer Usage, Deepmind, Demis Hassabis, Fintech, Gemini, Gen AI, Google, Language Models, Mustafa Suleyman, Products 

- Google Cloud customers are utilizing generative AI technologies, particularly Gemini, across various sectors, resulting in significant productivity improvements, such as an average saving of 105 minutes per user, per week.
- The introduction of six AI agent types has helped streamline processes across customer service and employee operations, delivering concrete benefits like cost savings and increased efficiency for numerous companies.
- Partnerships with major brands and a commitment to an open platform have been pivotal in driving the successful implementation of AI solutions, positioning Google Cloud as a leading choice for generative AI technologies.


~~~
[Google Cloud VP Gabe Monroy says he has seen 20%-30% productivity gains across the software development lifecycle at large companies using Gemini Code Assist - www.moneycontrol.com](https://www.moneycontrol.com/technology/genai-is-changing-how-we-code-but-coding-is-not-going-away-says-googles-gabe-monroy-article-12828935.html)

Topics: Big Tech, Code Assistants, Deepmind, Demis Hassabis, Economics, Gabe Monroy, Gemini, Gemini Code Assist, Gen AI, Google, Google Cloud, Job Automation, Jobs & Careerslabor Market, Language Models, Productivity, Products, Science, Software Development, Virtual Assistants 

- Google's Gabe Monroy reports a 20-30% productivity gain in software development from using the AI coding assistant, Gemini Code Assist, with potential for even 50% gains in startups.
- The rise of generative AI is expected to change the composition and roles within software teams, allowing developers to adopt more generalist skills due to AI's ability to assist in various areas of development.
- Gemini Code Assist, launched in December 2023, supports customization for enterprise environments and is designed to enhance compliance, legal, and risk mitigation in coding practices.


~~~
[Google supercharges enterprise contact centers with Gemini 1.5 Flash - VentureBeat](https://venturebeat.com/ai/google-supercharges-enterprise-contact-centers-with-gemini-1-5-flash/)

Topics: Big Tech, Contact Centers, Customer Service, Deepmind, Enterprise, Gemini, Gemini 1.5 Flash, Gen AI, Google, Language Models, Products, Speech Recognition & Synthesis 

- Google Cloud has rebranded its Contact Center AI to Customer Engagement Suite with Google AI, integrating advanced generative AI features powered by Gemini 1.5 Flash.
- The new suite enhances virtual agents and Agent Assist capabilities, enabling better efficiency and accuracy in customer interactions through features like real-time guidance, smart replies, and media generation.
- Many companies, including Google, are competing to leverage AI in contact centers, with predictions indicating that 80% of customer service teams will adopt generative AI by 2025.


~~~
[Snap says it will use Google's Gemini to help add new features to Snapchat's chatbot, including translating menus, after starting to use Gemini earlier in 2024 - Bloomberg](https://www.bloomberg.com/news/articles/2024-09-24/snap-integrates-google-s-generative-ai-model-into-chatbot)

Topics: Big Tech, Chatbot, Chatbots, Features, Gemini, Gen AI, Google, Language Models, Products, Snap, Sundar Pichai, Virtual Assistants 

- Snap Inc. is integrating Google’s generative AI model, Gemini, into its Snapchat AI chatbot to enhance user engagement and communication through images, videos, and text.
- The partnership has led to almost three times more engagement from users sending images and videos to the AI chatbot, aligning with the platform's visual communication style.
- Snap is focusing on AI investments to improve various features, including content recommendations and its newly launched augmented reality smart glasses, while maintaining safeguards for its young user demographic.


~~~
[In the age of AI, finding staff with soft skills has become a critical enterprise focus - IT Pro](https://www.itpro.com/business/business-strategy/in-the-age-of-ai-finding-staff-with-soft-skills-has-become-a-critical-enterprise-focus)

Topics: Cognitive Science, Customer Service, Enterprise, Job Automation, Jobs & Careerslabor Market, Society & Culture, Soft Skills, Staff 

- Research indicates a growing demand for 'soft skills' over technical skills in the job market, particularly in light of rapid advancements in AI technology.
- Companies are focusing on hiring candidates with a mix of technical abilities and soft skills to navigate complex business environments effectively.
- The UK job market is adapting to flexible working arrangements, with a notable increase in remote job postings and demand for candidates who can work efficiently in hybrid settings.


~~~
[The HR Revolution: Parenting AI and Shaping the Future of Work - www.shrm.org](https://www.shrm.org/topics-tools/flagships/ai-hi/the-hr-revolution--parenting-ai-and-shaping-the-future-of-work)

Topics: Ethics, Future Of Work, Gen AI, Hr Revolution, Job Automation, Jobs & Careerslabor Market, Opinion, Parenting AI 

- HR is evolving from traditional roles to become key players in ethical AI implementation and governance, influencing how AI integrates with employee experiences and organizational culture.
- The metaphor of "parenting AI" emphasizes HR's responsibility in shaping ethical values and culture around AI technologies, ensuring diverse data sets and promoting accountability.
- HR leaders are seen as essential in facilitating human-AI synergy, guiding conversations about AI's impact, and leveraging their understanding of behavioral science to enhance AI integration.


~~~
[How can Organisations be AI-ready? - Analytics India Magazine](https://analyticsindiamag.com/ai-breakthroughs/how-can-organisations-be-ai-ready/)

Topics: Gen AI, Job Automation, Organizations, Readiness 

- Organizations must align AI with business needs, establish a robust data architecture, and ensure strong AI governance to thrive in the AI era.
- Talent is crucial for AI success; all employees, not just technical staff, need to be AI literate, with a focus on creating a culture of continuous learning and innovation.
- Indian IT companies are rapidly integrating generative AI (genAI) into their operations, with significant project pipelines despite ongoing challenges in finding qualified talent.


~~~
[I think Im working for an AI - Reddit](https://www.reddit.com/r/ChatGPT/comments/1fov6mt/i_think_im_working_for_an_ai/)

Topics: Gen AI, Job Automation, Opinion, Workplace 

- The author suspects they might be working for an AI due to odd communication patterns with a client who requests home listing photos, including long response delays and repetitive text exchanges.
- Concerns are raised about the possibility of the author being involved in scams, such as advertising properties without ownership, by providing unique photos that cannot be traced online.
- The community advises caution, suggesting the author could be an unwitting accomplice in a rental scam and encourages them to cease contact with the client.


~~~
[Sam Altman believes wars will be fought over AI unless there's massive spending on infrastructure. But that comes with a cost. - Business Insider](https://www.businessinsider.com/openai-sam-altman-infrastructure-artificial-intelligence-data-centers-2024-9)

Topics: AI Doom, Big Tech, Economics, Ethics, Funding, Gen AI, Governance, Ilya Sutskever, Infrastructure, Military, OpenAI, Opinion, Policy And Regulation, Safety And Alignment, Sam Altman, Science, Spending, War 

- Sam Altman emphasizes the necessity of massive investments in AI infrastructure to prevent conflicts over technology and ensure widespread access, warning that inadequate infrastructure will lead to a limited resource scenario.
- Experts express concern that developing AI infrastructure incurs significant social and environmental costs, questioning the economic benefits of large-scale data centers.
- Discussions highlight the potential for AI to improve efficiency in various fields, but also raise alarms about unintended consequences, such as deepfakes and job displacement, underscoring the need for careful navigation of associated risks.


~~~
[Max is getting Google AI-generated closed captions - The Verge](https://www.theverge.com/2024/9/24/24253107/warner-bros-discovery-caption-ai-google-cloud-closed-captioning)

Topics: Big Tech, Closed Captions, Gen AI, Google, Language Models, Max, Products, Science, Speech Recognition & Synthesis, Streaming, Sundar Pichai, Tv & Film & Movies 

- Warner Bros. Discovery is partnering with Google Cloud to implement AI-generated closed captions for its Max streaming service, starting with unscripted programming.
- The new captioning solution, using Google Cloud’s Vertex AI, promises to reduce caption file creation time by up to 80% and cut costs by 50% compared to manual methods, while maintaining human oversight for quality assurance.
- WBD has not specified the extent of AI usage for captioning, but its trial with Google's technology indicates a potential shift towards AI as a standard practice in the industry.


~~~
[Warner Bros. Discovery partners with Google to add its caption AI tool built on Vertex AI to Max to automatically generate captions for unscripted programming - CNBC](https://www.cnbc.com/2024/09/24/warner-bros-discovery-google-partner-on-ai-generated-captions-on-max.html)

Topics: Big Tech, Caption AI, Code Assistants, Computer Vision, Deals, Entertainment, Gen AI, Google, Hollywood, Language Models, Max, Products, Science, Speech Recognition & Synthesis, Streaming, Sundar Pichai, Testing, Tv & Film & Movies, Vertex AI, Warner Bros. Discovery 

- Warner Bros. Discovery is partnering with Google to implement AI technology for generating more accurate captions on its Max streaming platform, focusing initially on unscripted programming.
- The new AI caption generation tool, built on Google’s Vertex AI, can reduce captioning costs by up to 50% and significantly decrease the time required to create new captions by up to 80%.
- While AI may streamline captioning processes, both companies emphasize the ongoing need for manual transcribers to ensure the accuracy of AI-generated content.


~~~
[The more I learn how to use ChatGPT, the more I realize how basic it is. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1fobvnn/the_more_i_learn_how_to_use_chatgpt_the_more_i/)

Topics: AI Limitations, Chatbots, Chatgpt, Education, Gen AI, Language Models, OpenAI, Opinion, Review, User Experience, Virtual Assistants 

- The author discusses their experience with ChatGPT, highlighting that while it can assist with drafting content, it's essential to maintain personal input to preserve writing and thinking skills.
- Regular reliance on AI tools like ChatGPT may diminish one's cognitive abilities over time if used for core tasks, suggesting a 60-40 division where the user handles the main work.
- The importance of retaining a "humane element" in content creation is emphasized amidst the increasing prevalence of AI-generated content.


~~~
[ChatGPT allowed me to sue my landlord without needing legal representation! - Reddit](https://www.reddit.com/r/ChatGPT/comments/1fodq3r/chatgpt_allowed_me_to_sue_my_landlord_without/)

Topics: Bias And Fairness, Chatbots, Chatgpt, Claude, Code Assistants, Copilot, Ethics, Gen AI, Governance, Inequality, Intellectual Property, Landlord, Language Models, Lawsuit, Legal Issues, Legal Representation, OpenAI, Opinion, Privacy, Society & Culture, Uk, Virtual Assistants 

- The author successfully used ChatGPT to identify a violation by their landlord regarding a deposit, leading to a small claims lawsuit where they stand to gain significantly more than they initially sought.
- ChatGPT assisted in understanding court procedures and preparing a defense against a retaliatory counterclaim filed by the landlord.
- While using ChatGPT for legal matters, especially small claims, can be helpful, the author recommends verifying the information and not relying solely on AI for serious claims.


~~~
[Leaders from the CIA, Dell Technologies, HP, SageCXO, and ezCater share how they keep up with AI development - Business Insider](https://www.businessinsider.com/technology-executives-on-artificial-intelligence-applications-role-of-cto-cio-2024-9)

Topics: AI Development, Big Tech, Cia, Dell Technologies, Hp, Sagecxo, Science 

- Top executives from the CIA, Dell, HP, and other companies discussed how AI and large language models (LLMs) are transforming their roles and enhancing personal and professional productivity.
- The conversation highlighted the blurring lines between CIO and CTO roles due to AI, with a focus on adopting a more collaborative, innovative approach to technology within organizations.
- Executives expressed the need for continuous learning and adaptation in AI, emphasizing that effective problem-solving and understanding customer needs are crucial for successful AI implementation.


~~~
[AI has made people more suspicious of all online media - Emerging Tech Brew](https://www.emergingtechbrew.com/stories/2024/09/24/adobe-study-ai-online-media)

Topics: Bias And Fairness, Disinformation, Ethics, Gen AI, Online Media, Opinion, Privacy, Privacy & Surveillance, Safety And Alignment, Society & Culture, Suspicion 

- A recent Adobe survey reveals that nearly 75% of respondents doubt the authenticity of media from reputable news outlets due to the prevalence of misinformation, especially concerning election-related content.
- Over 40% reported being misled by election falsehoods recently, and a significant portion indicated they will reduce their use of specific social media platforms due to misinformation.
- The survey highlights a call for stricter misinformation policies from social media platforms and government intervention, with concerns that generative AI could further impact trust in online media during the upcoming elections.


~~~
[Mercedes dials up the speed of its hands-free, eyes-off Drive Pilot feature - The Verge](https://www.theverge.com/2024/9/23/24252413/mercedes-drive-pilot-speed-germany-level-3-autonomous)

Topics: Automation, Autonomous Vehicles, Computer Vision, Drive Pilot, Hands-Free Driving, Hardware, Job Automation, Mercedes, Privacy & Surveillance, Products, Safety And Alignment, Science, Speech Recognition & Synthesis, Transportation 

- Mercedes-Benz's Drive Pilot Level 3 driving system will soon support speeds up to 95km/h (59mph) on German motorways, pending regulatory approval.
- Unlike Level 2 systems, Drive Pilot allows drivers to divert their attention from the road under certain conditions, paving the way for its use in road trips.
- Concerns remain regarding driver complacency and safety issues during the transition back to manual control, as studies suggest a disconnect can lead to dangerous reactions in emergencies.


~~~
[Hitler AI speech clips in English are going viral on TikTok - Google News](https://news.google.com/read/CBMiuAFBVV95cUxNLTRRN1BHbndIOUFWRHJTMFRSMlNlUkRjWTBkZHlaTDgtOTVRTFJGMlZiR3dUd28yVmlzc2piNk40NXRHV0txUDZyZHo0amcxMV83dkE3TUs0U1p6a05hZS05aGVCd1B3UU0xN1ZHN3FFSXBjbmM4bzNLLVZicG5iVHFoNWV0dE1rMFMtSEZOX2dmRTJPNWZjSWNfeHVZOU95U1UwdHh6NTVSWHo5YzBFekhSdVNVVUtr0gHGAUFVX3lxTE5KRmNWejR6RUxCZmJVYURlVXV3Nk5YN1FjQTJGVXhmNTV4YkpqWVMyYUVEU3QwaGNOQjgzdW9oQ1FhSzIwQldoNEU1dmxpaTRQV2pOOEQwbzI3YTg2Z1N0UUg1RGFCeFdKMTY5VnNUM29vcEMwZnRiN3E0b2Nsbk9iODQ1MjZ3a3lOWDJ6cVRCMFozMTN3T2dyR1F0WnBWZjZ3Zm5kdkhtNDhoQmkzOENuYzJ3WU1mMmt3ZWFBMUdndUJ2RV90QQ)

Topics: AI Doom, Bias And Fairness, Deepfakes, Disinformation, Entertainment, Ethics, Gen AI, History, Hitler, Language Models, Society & Culture, Speech Clips, Speech Recognition & Synthesis, Tiktok, Viral Content 

- AI-generated audio clips of Hitler's speeches in English are going viral on TikTok, particularly among younger audiences, amassing millions of views.
- Users are surprisingly praising and sharing these clips as entertainment, despite their poor translations and reverb-heavy music.
- TikTok's community guidelines prohibit promoting hate or violence, but many users appear indifferent to these rules in the context of the trend.


~~~
[SFChronicle publishes Kamala Harris News Assistant, powered by AI and archive - San Francisco Chronicle](https://www.sfchronicle.com/about/newsroomnews/article/sfchronicle-publishes-kamala-harris-news-19791035.php)

Topics: Authors & Writing, Big Tech, Books & Publishing, Chatbots, Code Assistants, Gen AI, Kamala Harris, Language Models, News Assistant, Politics, Retrieval Augmented Generation, Sfchronicle, Speech Recognition & Synthesis, Virtual Assistants 

- The San Francisco Chronicle has launched the Kamala Harris News Assistant, an AI-powered tool designed to provide readers with information about the presidential candidate based on 29 years of Chronicle political reporting.
- The tool, created in collaboration with Hearst's DevHub, aims to ensure accurate information by drawing from the Chronicle's extensive archive, while implementing safeguards to avoid misinformation.
- The initiative reflects the Chronicle's long-standing coverage of Kamala Harris, emphasizing the significance of her political career to local audiences.


~~~
[Musk dreams of launching five Starships to Mars in two yearsJust like robotaxis, actual full self-driving, etc etc in just a few years - The Register](https://www.theregister.com/2024/09/23/musk_expects_to_launch_five/)

Topics: AI Doom, Autonomous Vehicles, Elon Musk, Hardware, Job Automation, Mars, Musk, Opinion, Robots, Science, Space Exploration, Starship, Tesla, Transportation 

- Elon Musk announced plans for SpaceX to launch five uncrewed Starships to Mars within two years, despite the program currently facing bureaucratic hurdles with regulatory agencies.
- Musk indicated that successful uncrewed missions could pave the way for crewed missions to Mars in four years, but any failures could delay timelines by two years.
- The primary challenges to Musk's ambitions are political and regulatory, as he expresses frustration over government oversight affecting the Starship program.


~~~
[Sam Altman's OpenAI Pushes Biden Administration For Massive 5GW Data Centers Equivalent To Output Of 5 Nuclear Reactors, Enough To Power Nearly 3M Homes: Report - Benzinga](https://www.benzinga.com/news/24/09/41020113/sam-altmans-openai-pushes-biden-administration-for-massive-5gw-data-centers-equivalent-to-output-of)

Topics: Big Tech, Chatgpt, Data Centers, Energy, Gen AI, Governance, Hardware, Ilya Sutskever, Infrastructure, Nuclear, Open Source, OpenAI, Policy And Regulation, Politics, Safety And Alignment, Sam Altman, Science, Sustainability 

- OpenAI has proposed the construction of five-gigawatt data centers to the Biden administration, aiming to enhance AI infrastructure and increase energy efficiency for AI models, equivalent to the output of five nuclear reactors. 
- The initiative is driven by competition with China, job creation, and national security, with projections that AI data centers may consume up to 25% of U.S. power needs by 2030. 
- OpenAI's CEO Sam Altman is in discussions to establish five to seven data centers, and recent investments have been made to address the rising electricity demands from AI technologies.


~~~
[Microsoft claims its AI safety tool not only finds errors but also fixes them - Google News](https://news.google.com/read/CBMilAFBVV95cUxQdGV0UmRpMmp3d2dUZTVpYXUyQ2t3WWg0WlZ4dTE4eDZoYVlJYzZWbWllbllXbVk4SHllVWR1UWgxOTlvbXgyMEhOd0FtdUxqcUZVQ1NMZjFCalBhUlJJelpBU3A5OTRMc2Z1MjZYajRvV3pORF9Yc1ozLTVnb05KcjV6RVFFbmthNTN6UmtqdEVjVl9H)

Topics: Big Tech, Bill Gates, Cybersecurity, Error Detection, Ethics, Gen AI, Job Automation, Microsoft, Products, Safety And Alignment, Safety Tool, Satya Nadella, Science, Testing 

- Microsoft has launched a "correction" tool within Azure AI Studio that automatically detects and rewrites inaccuracies in AI outputs.
- The tool analyzes generated content against a customer’s source material, highlights errors, and provides explanations before users see the inaccuracies.
- While the correction feature aims to improve reliability in AI systems, Microsoft acknowledges it is not infallible and doesn't guarantee complete accuracy.


~~~
[Hollywood is coming out in force for Californias AI safety bill - Google News](https://news.google.com/read/CBMioAFBVV95cUxNdGxvSWQ1OW15NG16NDZSdW1qUGxFbkJlTGVudTRaTWtmeEctcTVSYUZHa2drVnRsQjI5RkFTSVkwS0ZkVExaSUxwQWl4NkI0c0dnMEVjb2gxWU9vUFhwLXhNaEhTSXdGLWhOaHBoUDZ0Z2NwWXNIWHJ6eXRaSjNsRm1mWDNHNVhNNE80OEdqM2FERHd1aUdpMDlQUC1xMU0y)

Topics: AI Doom, Bias And Fairness, California, Entertainment, Ethics, Gen AI, Governance, Hollywood, Intellectual Property, Legal Issues, Policy And Regulation, Politics, Safety And Alignment, Safety Bill, Science, Society & Culture, Tv & Film & Movies 

- Over 125 Hollywood celebrities have publicly urged California Governor Gavin Newsom to sign SB 1047, an AI safety bill aimed at regulating AI's potential catastrophic risks, marking a significant organized effort beyond the entertainment industry's interests.
- The legislation, if signed, would require major AI developers to implement self-chosen safeguards to prevent disasters, signaling a shift from self-regulation by the industry to mandated controls, placing California at the forefront of US AI regulation.
- The bill has faced both support and opposition, with prominent figures from SAG-AFTRA and public figures voicing concerns over unregulated AI's risks, while the tech industry and some political leaders express fears about its implications for competitiveness and innovation.


~~~
[Hitler speeches that appear to be AI-generated are getting millions of views on TikTok - Google News](https://news.google.com/read/CBMirAFBVV95cUxNNnNKTC03YnNMeER3VTBKSWc5UEl3cTlacE1aMGVabjVuTTlaMDl4Q3oybnVkbTZ0XzhkWjNxZ25PV3NEcU5hS1ZYMFA4MXFFQlRvbVd5UHByRTBRQXVjcll0dWF3MUxwVG1Lc0c2WWcwYzlEajAyQ2pGQlpCdTVfcVpXT3hERHVGV1JjZWxQOFhHLS1Nb2RRbkctQi0zb0NjTGtVRGV3MVh3ZGhf)

Topics: AI Doom, AI-Generated, Bias And Fairness, Deepfakes, Disinformation, Entertainment, Ethics, Gen AI, History, Hitler, Language Models, Society & Culture, Speech Recognition & Synthesis, Speeches, Tiktok, Viral Content 

- AI-generated audio clips of Hitler's speeches have gone viral on TikTok, violating the platform's community guidelines against promoting individuals who caused mass violence.
- Despite TikTok's removal of some offending accounts and videos, the content continues to circulate, garnering millions of views and often being repurposed into memes by users.
- Media Matters identifies a worrying trend of users praising Hitler and misrepresenting historical facts in these viral clips, highlighting the need for social media platforms to address the proliferation of such disinformation.


~~~
[Generative AI adoption surpasses early PC and internet usage, study finds - VentureBeat](https://venturebeat.com/ai/generative-ai-adoption-surpasses-early-pc-and-internet-usage-study-finds/)

Topics: Adoption, Big Tech, Economics, Gen AI, Generative AI, Internet, Language Models, Pc, Science, Study 

- Generative AI adoption in the U.S. has reached 39.4%, outpacing early PC and internet usage, with significant workplace integration noted.
- Its usage spans various sectors, with notable engagement from blue-collar workers and those without a college degree, indicating widespread accessibility.
- The integration of generative AI may exacerbate workplace inequality, as younger, more educated, and higher-income workers show higher usage rates, mirroring trends from past technological revolutions.


~~~
[Microsoft unveils new Correction tool to address AI hallucinations but there are early concerns  "Trying to eliminate hallucinations from generative AI is like trying to eliminate hydrogen from water" - Windows Central](https://www.windowscentral.com/software-apps/microsoft-unveils-new-correction-tool-to-address-ai-hallucination)

Topics: AI Doom, Bias And Fairness, Big Tech, Bill Gates, Correction Tool, Cybersecurity, Disinformation, Ethics, Gen AI, Hallucinations, Job Automation, Language Models, Microsoft, Opinion, Products, Safety And Alignment, Satya Nadella, Science, Testing 

- Microsoft has launched a new Correction tool aimed at reducing AI hallucinations, but experts believe it won't fully solve the accuracy issues inherent in generative AI technologies.
- The tool is designed to fact-check AI outputs against credible sources but may instill a false sense of security among users regarding the reliability of AI-generated information.
- Despite improvements in addressing hallucinations, ongoing concerns about user dissatisfaction and the potential for new issues remain prevalent among experts.


~~~
[Revolutionizing Product Management: The Confluence of AI, NLP and Agile Methodologies - Hacker Noon](https://hackernoon.com/revolutionizing-product-management-the-confluence-of-ai-nlp-and-agile-methodologies)

Topics: Agile Methodologies, Gen AI, Language Models, Nlp, Product Management, Products, Science 

- The integration of AI and NLP with Agile methodologies is transforming product management by enabling data-driven decision-making, real-time customer insights, and adaptive innovation in product development.
- AI's predictive analytics and sentiment analysis capabilities enhance market forecasting, optimize product availability, and improve customer support through automation, significantly increasing operational efficiency.
- Future trends in product management include autonomous optimization, advanced predictive modeling, and emotion AI, which will create more responsive and user-centric products.


~~~
[The US, Meta, Anthropic, Google, IBM, Microsoft, Nvidia, and OpenAI launch the Partnership for Global Inclusivity on AI to advance sustainable AI development - www.state.gov](https://www.state.gov/united-states-and-eight-companies-launch-the-partnership-for-global-inclusivity-on-ai/)

Topics: Anthropic, Bias And Fairness, Big Tech, Climate, Ethics, Gen AI, Google, Governance, Ibm, Inequality, Mark Zuckerberg, Meta, Microsoft, Nvidia, OpenAI, Partnership For Global Inclusivity, Policy And Regulation, Politics, Safety And Alignment, Satya Nadella, Science, Society & Culture, Sustainability, Sustainable AI, Us 

- The Partnership for Global Inclusivity on AI (PGIAI) was launched by Secretary of State Antony Blinken, uniting eight major tech companies and committing over $100 million to harness AI for sustainable development in developing countries.
- The partnership focuses on increasing access to AI tools, building human capacity, and expanding local datasets, with significant funding announcements for various AI initiatives aimed at promoting democracy, human rights, and technical training.
- Participating companies, including Amazon, Google, IBM, and Microsoft, committed substantial resources for skills training, infrastructure development, and supportive programs to empower local AI ecosystems and address global challenges.


~~~
[Artificial intelligence is detecting new archaeological sites in the desert - Google News](https://news.google.com/read/CBMilgFBVV95cUxPWFhkeUR2UkVCU3V2ZzJGaTNibjJlTE1IQVQ0RWNMWDQyV2JuUmtrUkFaRTBsbDdySHY4enRCN0lrNzdkcXBCeEpHOXlBNmxIUWJfN241cWRQTkZlVDBqNWk2VUl6anhrekJwN1otYkRvWjBBYkN0ZjRGQm1wa2JpeUd5LUJsZmFNZlpqODFKU2NtSHI2WVE)

Topics: Archaeology, Computer Vision, Desert, Detection, Gen AI, History, Science, Society & Culture 

- Researchers in Abu Dhabi have developed a machine learning algorithm that utilizes synthetic aperture radar (SAR) to detect archaeological sites buried in desert sand, aiding archaeologists in areas where traditional methods struggle.
- The algorithm was trained using existing archaeological data and has shown precision within 50 centimeters, allowing for the creation of 3D models that help identify potential excavation sites.
- Despite challenges like cost and complexity, the technology could revolutionize archaeological site detection in desert regions, with plans for further application in other areas such as Saudi Arabia and Egypt.


~~~
[securityweek.com/ai-gen... - SecurityWeek](https://www.securityweek.com/ai-generated-malware-found-in-the-wild/)

Topics: Cybersecurity, Gen AI, Safety And Alignment, Security 

- HP has identified an AI-generated malware dropper in a phishing campaign, using encryption and HTML smuggling techniques to evade detection.
- The malware includes a well-structured, commented VBScript that appears designed for human readability, suggesting it may have been generated by AI, particularly given its unconventional use of the French language.
- This incident indicates that AI is lowering the barrier for less experienced attackers to create malware, raising concerns about more seasoned adversaries potentially utilizing AI-generated payloads without leaving detectable clues.


~~~
[theconversation.com/ai-is-... - The Conversation](https://theconversation.com/ai-is-fuelling-a-deepfake-porn-crisis-in-south-korea-whats-behind-it-and-how-can-it-be-fixed-238217)

Topics: AI Doom, Artificial General Intelligence, Bias And Fairness, Big Tech, Cognitive Science, Consciousness, Cybersecurity, Discussion, Education, Ethics, Gen AI, Inequality, Job Automation, Language Models, Opinion, Privacy, Safety And Alignment, Science, Singularity, Society & Culture 

- A report revealed that over half of deepfake porn videos analyzed in South Korea target women and girls, indicating a significant gender-based issue exacerbated by the rise of AI technologies.
- Deepfakes have become normalized among teenagers, with dedicated online groups facilitating the creation and distribution of manipulated images, impacting victims including minors and educators.
- The South Korean government has announced plans to legislate against deepfakes, but effective action requires recognizing these crimes as gender-based violence and implementing comprehensive reforms along with public awareness initiatives.


~~~
[Exclusive: Healthcare AI startup Tennr is raising fresh cash from Lightspeed Venture Partners just months after its A16z-led Series A - Business Insider](https://www.businessinsider.com/healthcare-ai-startup-tennr-raising-cash-lightspeed-venture-partners-a16z-2024-9)

Topics: AI Startup, Big Tech, Deals, Economics, Finance, Funding, Gen AI, Health & Fitness, Healthcare, Lightspeed Venture Partners, Tennr, Venture Capital 

- Healthcare AI startup Tennr is raising $33 million in a Series B round led by Lightspeed Venture Partners, just seven months after its $18 million Series A.
- The new funding round values Tennr at over $200 million, with some sources suggesting it could be closer to $300 million.
- Tennr, which automates healthcare administrative tasks like fax processing, is part of a growing trend of AI startups attracting quick follow-up funding.


~~~
[AI Battle: VS Code Takes on Cursor with Major AI Updates - Geeky Gadgets](https://www.geeky-gadgets.com/vs-code-introduces-new-ai-features/)

Topics: Big Tech, Code Assistants, Copilot, Cursor, Gen AI, Microsoft, Open Source, Products, Software Updates, Vs Code 

- VS Code has introduced new AI features aimed at competing with Cursor, integrating functionalities such as context attachment, test generation, and enhanced chat options to improve user experience.
- Despite showing promise, the updates have experienced mixed results in test generation and user feedback, highlighting areas that need further refinement.
- The competition between VS Code and Cursor is expected to drive innovation in AI-assisted coding tools, impacting future developments in the field.


~~~
[On Impactful AI Research - GitHub](https://github.com/okhat/blog/blob/main/2024.09.impact.md)

Topics: Gen AI, Impact, Opinion, Research, Science 

- The author emphasizes focusing on impactful open-source projects in AI research rather than merely publishing papers, suggesting that sustained engagement with a coherent research vision will yield greater long-term fulfillment and growth.
- Effective open-source research should involve creating usable, useful, and approachable artifacts while fostering a community that encourages collaboration and contribution, thereby enhancing the project's overall impact. 
- Researchers are encouraged to think ahead, iterate rapidly, and actively communicate their work to engage with broader audiences, which can facilitate the growth of their research influence and open-source contributions.


~~~
[Salesforce agrees to buy Zoomin, an Israeli data management company that makes unstructured data accessible to AI agents, in a deal valued at $450M - Calcalistech](https://www.calcalistech.com/ctechnews/article/rkhx9kl0r)

Topics: Big Tech, Data Management, Deals, Economics, Finance, Funding, Gen AI, Mergers And Acquisitions, Products, Salesforce, Stocks, Zoomin 

- Salesforce has announced its acquisition of Israeli data management company Zoomin for $450 million, following a previous $1.9 billion acquisition of Own.
- Zoomin specializes in processing unstructured data, facilitating its accessibility to AI engines, and will integrate its services into Salesforce’s Data Cloud and Service Cloud.
- The deal is part of Salesforce's strategy to enhance its AI capabilities and support enterprises in overcoming data challenges as they pursue digital transformation.


~~~
[The US DOL releases the AI & Inclusive Hiring Framework, a set of guidelines to help employers ensure AI doesn't lead to discrimination for disabled job seekers - Bloomberg Law](https://news.bloomberglaw.com/tech-and-telecom-law/dol-outlines-how-employers-can-avoid-discrimination-in-ai-hiring)

Topics: Bias And Fairness, Disabled Job Seekers, Discrimination, Ethics, Gen AI, Governance, Inclusive Hiring Framework, Inequality, Job Automation, Jobs & Careerslabor Market, Legal Issues, Policy And Regulation, Politics, Safety And Alignment, Science, Society & Culture, Us Dol 

- The US Labor Department launched a website to guide employers in using AI hiring technology responsibly, emphasizing the importance of avoiding discrimination and ensuring accessibility for disabled job seekers.
- New guidelines focus on legal requirements, human oversight, and the development of inclusive hiring policies that advance the employment of individuals with disabilities.
- This initiative is part of the Biden administration's broader efforts to establish frameworks for the safe and equitable use of AI in the workplace following a 2023 executive order.


~~~
[Turning GenAI pilots into production powerhousesGet tips on overcoming common challenges in AI deploymentWebinar - The Register](https://www.theregister.com/2024/09/24/turning_genai_pilots_into_production/)

Topics: AI Deployment, Copilot, Gen AI, Gen AI, Production, Science, Webinar 

- Many organizations are still in the early stages of GenAI deployment, with only 4% having mature applications in production.  
- The transition from pilot projects to full-scale production involves tackling challenges related to governance, security, and performance.  
- Boomi's Practical AI platform aims to assist organizations by providing support throughout the AI lifecycle, from planning to operational optimization.


~~~
[Trae Stephens Has Built AI Weapons and Worked for Donald Trump. As He Sees It, Jesus Would Approve - Wired](https://www.wired.com/story/big-interview-trae-stephens-has-built-ai-weapons-and-worked-for-donald-trump-as-he-sees-it-jesus-would-approve/)

Topics: AI Doom, AI Weapons, Donald Trump, Ethics, Gen AI, Military, Opinion, Politics, Society & Culture, Trae Stephens 

- Trae Stephens, co-founder of Anduril and partner at Founders Fund, has shifted from intelligence to defense tech, focusing on AI and autonomous military systems, while advocating for ethical alignment with military directives.
- Anduril has grown rapidly, valued at $14 billion with substantial defense contracts, due to a strategic approach to government relationships and a pivot towards building lethal technologies in response to global threats.
- Stephens addresses the role of autonomous systems in warfare, emphasizing the necessity of human oversight in decision-making while advocating for a substantial increase in low-cost military production capabilities.



~~~
[Google says generative AI is ready to do real work - Google News](https://news.google.com/read/CBMiiwFBVV95cUxNdG1Xa3BZbktpMURKWHBubnJDbzZvblRRdTJza2NlZFBESHczdlBtQkE4R0ZwSldoNXZOWDI1a2ZEeWZCdHBVb1hsMG81RHFUR2pHRWJJWFNjT3RPNEVWN3lNd1V5WUZYbUlleXJsbkx2a1p4Zk9oZnJJY3MtVlhHcmlGNUF5eEl5QjNz)

Topics: Big Tech, Gen AI, Generative AI, Google, Job Automation, Jobs & Careerslabor Market, Language Models, Science, Work 

- Google is showcasing its Gemini AI and Vertex tools to demonstrate their business applications and promote competitive advantages over Microsoft and OpenAI, featuring customer examples like Scotts Miracle-Gro and Pods.
- The potential dockworker strike on October 1 poses significant economic risks, threatening supply chain disruptions and potential costs of up to $5 billion daily, with the White House facing political challenges in addressing the situation.
- A growing number of states are implementing regulations for generative AI in election-related communications, highlighting concerns over its influence on voter behavior and the electoral process.



~~~
[Hetzner introduces GPU server for AI training - www.hetzner.com](https://www.hetzner.com/dedicated-rootserver/gex130/)

Topics: AI Training, Gen AI, Gpu Server, Hardware, Hetzner, Products, Science 

- The GEX130 dedicated GPU server features the NVIDIA RTX™ 6000 with 48 GB GPU Memory, making it suitable for AI model training and large data processing.
- It offers full root access, DDoS protection, firewall management, and multiple administration tools, backed by a redundant network with over 30.9 TBit total bandwidth.
- Optional features include additional IP addresses, storage options, and 24/7 support, with flexible payment methods and a no minimum contract term policy.


~~~
[OpenAI's proposed logo refresh sparks controversy among staffers, calling it "devoid of creativity"  despite ChatGPT and DALL-E 3's touted "insanely powerful" logo creation capabilities - Windows Central](https://www.windowscentral.com/software-apps/openai-logo-refresh-sparks-controversy-among-staffers)

Topics: Art & Design, Big Tech, Chatbots, Chatgpt, Controversy, Creativity, Dall-E 3, Gen AI, Logo, OpenAI, Opinion 

- OpenAI is considering a logo rebrand to a simple circle representing "O" or zero, which has received mixed reactions from staff, described as "uninspiring and ominous."
- Staff feedback may lead to improvements or a completely new design, as the new logo is not finalized and could be a placeholder.
- Despite their advanced design capabilities, OpenAI's DALL-E 3 and ChatGPT tools have faced criticism for producing less than satisfactory outputs in simple tasks like logo creation.


~~~
[Discover Replit: Your Ultimate Guide to Coding with AI Agents - Hacker Noon](https://hackernoon.com/discover-replit-your-ultimate-guide-to-coding-with-ai-agents)

Topics: AI Agents, Chatbots, Code Assistants, Coding, Education, Gen AI, Language Models, Open Source, Products, Reinforcement Learning, Replit, Review, Science, Virtual Assistants 

- Replit is an AI-driven online integrated development environment (IDE) that supports over 50 programming languages, designed to streamline the coding and deployment process for users of all skill levels.
- The platform offers tools like Replit Agent, which assists in building applications through natural language prompts, making software development accessible to those with limited programming knowledge.
- Users can select various subscription plans, including a free "Starter" plan with basic features and a more robust "Replit Core" plan for full functionality, which provides the ability to use advanced AI tools.


~~~
[Researchers detail an AI model that they claim can solve Google's reCAPTCHAv2 challenges with 100% accuracy using a similar number of attempts as human users (Peter Saalfield/Decrypt) - Decrypt](https://decrypt.co/251107/ai-can-best-googles-bot-detection-system-swiss-researchers-find)

Topics: Accuracy, Big Tech, Cognitive Science, Computer Vision, Cybersecurity, Gen AI, Google, Language Models, Model, Privacy, Privacy & Surveillance, Recaptcha, Safety And Alignment, Science, Testing 

- Researchers from ETH Zurich successfully bypassed 100% of Google's reCAPTCHA v2 using advanced machine learning techniques.
- The study highlights the vulnerability of Google's bot detection systems to sophisticated AI methods.
- This development raises concerns about the effectiveness of current CAPTCHAs in distinguishing between human users and automated bots.


~~~
[Microsoft unveils trustworthy AI features to fix hallucinations and boost privacy - VentureBeat](https://venturebeat.com/ai/microsoft-unveils-trustworthy-ai-features-to-fix-hallucinations-and-boost-privacy/)

Topics: Bias And Fairness, Big Tech, Bill Gates, Cybersecurity, Disinformation, Ethics, Gen AI, Job Automation, Language Models, Microsoft, Privacy, Privacy & Surveillance, Products, Safety And Alignment, Satya Nadella, Science, Testing, Trustworthy Features 

- Microsoft has introduced 'Trustworthy AI' features aimed at addressing issues related to AI hallucinations, privacy, and security, including tools for confidential inferencing and AI output evaluation.
- A key feature is the 'Correction' capability in Azure AI Content Safety, which improves the accuracy of AI responses by providing feedback on mismatches between context and AI-generated content.
- The initiative positions Microsoft as a potential leader in responsible AI development, amidst ongoing challenges in balancing innovation with safety and industry standards.


~~~
[Hacker plants false memories in ChatGPT to steal user data in perpetuity - Ars Technica](https://arstechnica.com/security/2024/09/false-memories-planted-in-chatgpt-give-hacker-persistent-exfiltration-channel/)

Topics: AI Doom, Bias And Fairness, Chatbots, Chatgpt, Cognitive Science, Consciousness, Cybersecurity, Data Theft, Disinformation, Ethics, Gen AI, Hacker, Intellectual Property, Language Models, Legal Issues, OpenAI, Privacy, Privacy & Surveillance, Safety And Alignment, Scams, Science, Security 

- A security researcher discovered a vulnerability in ChatGPT allowing attackers to plant false memories via long-term memory settings, using untrusted content as an indirect prompt injection method.
- The researcher created a proof-of-concept exploit that could permanently exfiltrate user inputs by manipulating ChatGPT, leading OpenAI to issue a partial fix against such attacks.
- Despite the fixes, the researcher warns that untrusted content can still leverage prompt injections to implant malicious long-term memories, urging users to monitor and review their stored memories regularly.


~~~
[Tort Law Is the Best Way to Regulate AI - The Wall Street Journal](https://www.wsj.com/opinion/tort-law-is-the-best-way-to-regulate-ai-california-legal-liability-065e1220)

Topics: AI Regulation, Bias And Fairness, Ethics, Gen AI, Governance, Intellectual Property, Legal Issues, Opinion, Policy And Regulation, Politics, Safety And Alignment, Science, Tort Law 

- The text does not contain any substantive content and primarily consists of boilerplate elements.


~~~
[What I've Learned in the Past Year Spent Building an AI Video Editor - www.makeartwithpython.com](https://www.makeartwithpython.com/blog/a-year-of-showing-up/)

Topics: Development, Entertainment, Experience, Gen AI, Opinion, Review, Science, Tv & Film & Movies, Video Editor 

- The author shares experiences from the past year of building an AI video editor, highlighting the potential of new generative video technologies enabled by Large Language Models and computer vision.
- They reflect on the challenges faced while developing a video editing platform that shifts from static outputs to dynamic, personalized video generation, leveraging tools like Microsoft’s Promptflow and Temporal.
- The text emphasizes the importance of reevaluating existing workflows to fully utilize AI's capabilities, and caution against overreliance on generic embedding models for specific retrieval tasks, advocating for more tailored solutions.


~~~
[An interview with The Browser Company CEO Josh Miller on Arc Search being an operating system, competing with Chrome and Safari, monetization, AI, and more - The Verge](https://www.theverge.com/24247369/the-browser-company-ceo-josh-miller-arc-google-chrome-ai-search-web-decoder-interview)

Topics: Apple, Arc Search, Big Tech, Competition, Economics, Finance, Gen AI, Google, Josh Miller, Opinion, Products, Science, The Browser Company 

- Josh Miller, CEO of The Browser Company, argues that the Arc browser redefines web browsing as a productivity-focused platform, aiming to streamline app usage and enhance user experience.
- Arc incorporates AI features and a mobile app, positioning itself in competition with Google Chrome and Apple Safari, while also addressing the broader implications of AI on content access and monetization.
- Despite initial positive reception, Arc faces challenges in user adoption and understanding due to its unique design metaphors and the need to simplify its features for the average user.


~~~
[Google updates Google Photos' mobile video editor for Android and iOS, adding editing tools and AI presets that should make trimming and tweaking clips easier - The Verge](https://www.theverge.com/2024/9/24/24252944/google-photos-redesigned-video-editor-ai-presets)

Topics: Apple, Art & Design, Big Tech, Computer Vision, Editing Tools, Entertainment, Gen AI, Google, Google Photos, Music, Products, Tv & Film & Movies, Video Editor 

- Google Photos is releasing a redesigned video editor for Android and iOS, featuring new editing tools and AI-powered presets to enhance video editing.
- New features include a "Speed" tool for adjusting playback speed, an "Auto enhance" button for improving video quality, and a rearranged interface for easier access to editing functions.
- AI-powered presets allow users to make specific adjustments with a single tap, offering options for auto-trimming, speed control, and dynamic motion tracking effects.


~~~
[NASA, IBM just open sourced an AI climate model so you can fine-tune your ownPrithvi, Prithvi, Prithvi goodScience5 hrs|5 - The Register](https://www.theregister.com/2024/09/25/nasa_ibm_ai_weather/)

Topics: Climate, Climate Model, Energy, Gen AI, Ibm, Nasa, Open Source, Science, Sustainability 

- NASA and IBM have open-sourced an AI climate model called Prithvi WxC, designed to predict weather patterns more efficiently using 40 years of observational data.
- The model, with 2.3 billion parameters, is adaptable for various applications, such as short-term forecasting and long-term climate projections, setting it apart from traditional fixed-dataset models.
- The Canadian government is among the first to utilize the model, aiming to improve short-term precipitation forecasts and experiment with downscaling for finer predictions.


~~~
[Google Paid $2.7 Billion to Bring Back an AI Genius Who Quit in Frustration - The Wall Street Journal](https://www.wsj.com/tech/ai/noam-shazeer-google-ai-deal-d3605697)

Topics: Big Tech, Deals, Employee Retention, Finance, Gen AI, Geoffrey Hinton, Google, Investment, Larry Page, Mustafa Suleyman 

- The content primarily consists of boilerplate information and does not provide any specific news or articles.  
- Users are prompted for login or human verification.  
- No substantial main points or information are present in the text.


~~~
[An interview with Arvind Narayanan and Sayash Kapoor on their new book AI Snake Oil, which is based on their popular newsletter about AI's shortcomings (Reece Rogers/Wired) - Wired](https://www.wired.com/story/artificial-intelligence-hype-ai-snake-oil)

Topics: AI Doom, Arvind Narayanan, Authors & Writing, Bias And Fairness, Big Tech, Book, Books & Publishing, Education, Ethics, Gen AI, India, Opinion, Review, Safety And Alignment, Sayash Kapoor, Science, Shortcomings, Society & Culture 

- Princeton researchers Arvind Narayanan and Sayash Kapoor critique the exaggerated claims surrounding artificial intelligence in their book "AI Snake Oil," focusing on the role of companies, researchers, and journalists in perpetuating hype.
- They argue that misleading marketing and shoddy research misrepresent AI's capabilities, emphasizing the need for clearer communication and critical understanding to mitigate public misconceptions.
- The authors advocate for improved education about AI, recommending that knowledge of its limitations and mechanics begin in elementary school to help users navigate the growing influence of this technology effectively.


~~~
[OpenAI CEO Cranks Up AI Hype, Promises 'Superintelligence' in 3 Years - ExtremeTech](https://www.extremetech.com/computing/openai-ceo-cranks-up-ai-hype-promises-superintelligence-in-3-years)

Topics: AI Doom, AI Hype, Artificial General Intelligence, Big Tech, Ceo, Consciousness, Gen AI, Ilya Sutskever, Language Models, OpenAI, Opinion, Safety And Alignment, Sam Altman, Science, Singularity, Superintelligence 

- OpenAI CEO Sam Altman predicts that “superintelligence” could be achieved within three years, calling this advancement a key moment in human history with immense potential benefits.
- Altman emphasizes the importance of compute, energy, and human effort in reaching an "Intelligence Age," although concerns about the environmental and financial costs are notably absent from his discussion.
- Critics question Altman's optimistic claims about AI, particularly his assertion that it will significantly improve society, given current limitations and issues such as model inaccuracies and the ongoing debate about the true capabilities of AI.


~~~
[Is it time for accountants to integrate AI into their practices? - Accounting Today](https://www.accountingtoday.com/list/is-now-the-time-for-accountants-to-add-ai-to-their-practices)

Topics: Accountants, Economics, Finance, Gen AI, Integration, Job Automation, Jobs & Careerslabor Market, Opinion, Practices 

- Accountants express significant concerns about the risks of generative AI, including accuracy (85% worrying about hallucinations), data exposure (83%), and loss of client trust (81%).
- Firms like MBS Accounting and GWCPA highlight the need for careful integration of AI, focusing on maintaining client relationships and using AI for efficiency while ensuring data security.
- Gradual adoption of AI, employee education, and establishing clear AI usage policies are critical strategies for mitigating risks associated with the technology in accounting practices.


~~~
[Sources: some key staff working on Microsoft's distilled AI models trained on synthetic data, like Phi, have left the Microsoft AI unit led by Mustafa Suleyman - The Information](https://www.theinformation.com/articles/microsoft-ai-chief-suleyman-shuffles-unit-leadership-pointing-to-shift-in-strategy)

Topics: AI Models, Big Tech, Gen AI, Language Models, Microsoft, Mustafa Suleyman, Satya Nadella, Science, Staff Changes, Synthetic Data 

- Microsoft AI Chief Suleyman has reshuffled unit leadership, indicating a strategic shift within the company.
- The recent changes suggest Microsoft may be moving away from its previous strategy of distillation, which relied on OpenAI's models for generating data.
- Anthropic is attempting to capitalize on OpenAI's fundraising efforts by pursuing its own funding round amid its financial challenges.


~~~
[Sources: Scale AI grew sales nearly 4X YoY to almost $400M in H1 2024 and had ~$980M in cash; the data labeling startup raised $1B at a $13.8B valuation in May - The Information](https://www.theinformation.com/articles/scale-ais-sales-nearly-quadrupled-in-first-half)

Topics: 2024, Big Tech, Data Labeling, Economics, Finance, Funding, Gen AI, Ipos, Sales Growth, Scale AI, Stocks, Valuation, Venture Capital 

- Scale AI's sales nearly quadrupled to almost $400 million in the first half of the year, compared to the same period last year.
- The company provides data labeling services for major AI developers like Meta Platforms and Google, helping improve their large language models.
- Scale AI is one of the key privately held firms benefiting from the current AI boom by offering essential infrastructure and services.


~~~
[Filtered for home robots, fast and slow - interconnected.org](https://interconnected.org/home/2024/09/20/filtered)

Topics: Automation, Hardware, Home Robots, Internet Of Things, Products, Robots 

- TidyBot, a robot from Princeton, can successfully tidy up 85% of household objects using large language models to adapt to preferences, making instruction following simpler for home robots.  
- A robot hand can instantly recognize competing gestures and always win at rock-paper-scissors, raising questions about the future cost-effectiveness of purchasing superfast robots for personal assistance.  
- An inventive robot spider bedside lamp offers nighttime assistance by moving autonomously, showcasing the potential for household robots to improve domestic life while reducing manual chores.


~~~
[Apple Intelligence Writing Tools have driven me back to Pages on my Mac  heres why - Tom's Guide](https://www.tomsguide.com/ai/apple-intelligence-writing-tools-have-driven-me-back-to-pages-on-my-mac-heres-why)

Topics: Apple, Authors & Writing, Big Tech, Code Assistants, Hardware, Intelligence Writing Tools, Mac, Opinion, Pages, Products 

- Apple Intelligence introduces Writing Tools that offer similar capabilities to AI writing aids like Gemini and Copilot but for free to Apple device users, leveraging local processing.
- The integration of Writing Tools across Apple applications, especially in Pages, enhances user experience by providing consistent access to AI-assisted writing features without relying on cloud services.
- Future updates are anticipated for deeper integration of Apple Intelligence across Apple software, with user suggestions for more persistent and adaptable AI features in productivity tools.


~~~
[China's worsening relationship with US businesses could put AI's chip supply on shaky ground - Business Insider](https://www.businessinsider.com/china-economy-supporting-domestic-companies-huawei-over-apple-ai-chips-2024-9)

Topics: AI Doom, Big Tech, China, Chip Supply, Economics, Gen AI, Governance, Hardware, Opinion, Policy And Regulation, Politics, Safety And Alignment, Science, Semiconductor Chips, Us Businesses 

- US-China relations are deteriorating, with Chinese consumers increasingly favoring domestic companies, impacting US businesses like Apple and Nike.
- China's economic struggles, including a shaky real estate market and declining consumer demand, contribute to a challenging environment for foreign companies.
- The potential invasion of Taiwan by China poses risks for the global AI industry, particularly regarding chip supply from Taiwan Semiconductor Manufacturing Company.



~~~
[Using Generative AI to Boost Your Personal Productivity - O'Reilly](https://www.oreilly.com/live-events/using-generative-ai-to-boost-your-personal-productivity/0636920099736/)

Topics: Education, Gen AI, Generative AI, Language Models, Personal Productivity, Review, Science, Virtual Assistants 

- Learn to use generative AI tools to enhance personal productivity, task management, and creative thinking.
- Discover methods for summarizing information, generating ideas, and creating content such as reports and presentations.
- The course is suitable for professionals, teachers, and students seeking to streamline work processes and enhance learning techniques.


~~~
[Apple says Apple Intelligence will require 4GB of storage on compatible iPhones, and the storage requirement will increase as more features roll out - 9to5Mac](https://9to5mac.com/2024/09/23/this-is-how-much-iphone-storage-apple-intelligence-will-require/)

Topics: Apple, Big Tech, Hardware, Intelligence, Iphones, Products, Storage Requirements 

- Apple Intelligence, launching with iOS 18.1, will require 4GB of storage for iPhone 16 and newer models, in addition to the iOS update storage needs.
- The storage requirement may increase over time as additional AI features are introduced, potentially requiring up to 8GB in the future.
- Currently, only iPhones are stated to have the 4GB requirement, with unclear storage needs for iPads and Macs.


~~~
[Microsoft debuts Correction in preview as part of the Azure AI Content Safety API; the service tries to find and revise AI-generated text that's factually wrong - TechCrunch](https://techcrunch.com/2024/09/24/microsoft-claims-its-new-tool-can-correct-ai-hallucinations-but-experts-caution-it-has-shortcomings/)

Topics: AI Content Safety Api, Azure, Bias And Fairness, Big Tech, Bill Gates, Cybersecurity, Disinformation, Ethics, Fact-Checking, Gen AI, Job Automation, Language Models, Microsoft, Products, Safety And Alignment, Satya Nadella, Science, Testing 

- Microsoft introduced a new tool called Correction, designed to automatically revise AI-generated text that's factually incorrect by flagging and fact-checking potentially erroneous content against reliable sources.
- Experts express skepticism about the tool's effectiveness in addressing the fundamental issue of AI hallucinations, noting that the underlying problem stems from the nature of generative AI being pattern-based rather than knowledge-based.
- The tool's implementation may create new trust and reliability issues, as it risks giving users a false sense of security about AI-generated content accuracy, especially given the potential for both models to introduce their own errors.


~~~
[Intel launches new AI chips as takeover rumors swirl - Google News](https://news.google.com/read/CBMinwFBVV95cUxNREV4cEVZN09heWduRnQyYmhGeVV4bzBGbzhSaGwzTmRKYTdyQVpDd1RFdUdrX3NEMnBLYl9FZVR5MG5Xd21BczNJekxpb29iVHlNN012Nnp1cXlCd1lwdFNJUWZUeUZzMF9ka2F0U1doc0hKLVlkSjJ5VEFmdTZvZXJZQldkZmtsTnFnRHEwMDdXNjc0ZmhmU2gxTWtVZ2c)

Topics: AI Chips, Big Tech, Gaming, Hardware, Intel, Manufacturing, Mergers And Acquisitions, Nvidia, Products, Science, Semiconductor Chips, Takeover Rumors 

- Intel launched two new AI chips, the Xeon 6 CPU and Gaudi 3 AI accelerator, aimed at enhancing its data center business amidst competition from AMD and Nvidia.
- The announcement comes alongside rumors of Qualcomm's potential takeover of Intel, as well as interest from Apollo Global Management in investing in the company.
- Despite the new products, Intel's market share is declining, with a significant drop in its stock price compared to the soaring performance of Nvidia’s AI chips.


~~~
['It should be used for the truth' | Fully AI-generated political ad parodies Lt. Gov. Mark Robinson's campaign. The ad, which could be the first of its kind, is bringing attention to the lack of legislation to require disclaimers in political ads. - Reddit](https://www.reddit.com/r/technology/comments/1fow4vn/it_should_be_used_for_the_truth_fully_aigenerated/)

Topics: Bias And Fairness, Deepfakes, Disclaimers, Disinformation, Ethics, Gen AI, Governance, Intellectual Property, Legal Issues, Legislation, Mark Robinson, Policy And Regulation, Political Ads, Politics, Safety And Alignment, Society & Culture 

- A political ad parodying Lt. Gov. Mark Robinson's campaign has surfaced, potentially being the first fully AI-generated ad of its kind.
- The ad highlights the absence of laws mandating disclaimers in political advertisements.
- Discussions in the r/technology subreddit center on the implications of AI in political messaging and its societal consequences.


~~~
[Feel free to ignore GenAI for now  a new kind of software developer is being bornInterviewKubernetes whizz says devs can redefine their roles and capitalize on coding ML rewardsSoftware2 days|45 - The Register](https://www.theregister.com/2024/09/23/kelsey_hightower_interview_part_2/)

Topics: Code Assistants, Coding, Gen AI, Gen AI, Interview, Job Automation, Jobs & Careerslabor Market, Kubernetes, Language Models, Machine Learning, Opinion, Software Development 

- Kelsey Hightower emphasizes the need for software developers to redefine their roles as the industry evolves with AI, focusing on leveraging software to solve problems rather than just writing code.
- He warns against an over-reliance on AI coding assistants, which could stifle the pipeline of future developers, and advocates for a continuous investment in open-source talent akin to sports training programs.
- Hightower highlights the maturity of Kubernetes and its API enhancements, predicting a future where implementations can be tailored to meet specific user needs, much like different web browsers offer unique experiences.


~~~
[GSoC'24: Differentiable Logic for Interactive Systems and Generative Music - ijc8.me](https://ijc8.me/2024/08/26/gsoc-difflogic/)

Topics: Art & Design, Cognitive Science, Differentiable Logic, Education, Entertainment, Gen AI, Generative Music, Gsoc, Interactive Systems, Music, Open Source, Reinforcement Learning, Science 

- The project focused on integrating differentiable logic into interactive systems and generative music, using components such as difflogic, Bela, and bytebeat.
- Key goals included enabling the use of logic gate networks in interactive applications, experimenting with difflogic models, and creating applications that demonstrate the capabilities of these networks in a musical context.
- Challenges encountered involved encoding issues when representing audio samples in logic gate networks, leading to a shift in focus towards real-time manipulation and visualization of the sound-generating networks.


~~~
[AI chatbot gets green light to hallucinate your investment portfolioService due to roll out later this month, despite fears AI will crash the marketAI + ML9 hrs|8 - The Register](https://www.theregister.com/2024/09/25/ai_stock_picker/)

Topics: AI Doom, Chatbot, Chatbots, Customer Service, Economics, Ethics, Finance, Fintech, Funding, Gen AI, Investment Portfolio, Market Risks, Policy And Regulation, Safety And Alignment, Stocks, Venture Capital, Virtual Assistants 

- An Israeli startup, Bridgewise, has received regulatory approval to launch a stock-picking chatbot in partnership with Israel Discount Bank, set to roll out later this month.  
- The chatbot is designed to provide stock-picking advice rather than make autonomous financial decisions, addressing customer demand for in-depth financial analysis.  
- Concerns about AI's impact on the stock market have been raised by SEC officials, who fear it may lead to critical market instability if widely adopted without regulatory oversight.  




~~~
[The Making of OpenAI o1: How OpenAI Built Its Groundbreaking AI Models - Geeky Gadgets](https://www.geeky-gadgets.com/the-making-of-openai-o1-how-openai-built-its-o1-ai-models/)

Topics: AI Models, Big Tech, Development, Gen AI, Language Models, Open Source, OpenAI, Science 

- OpenAI has launched the o1 and o1 mini AI models, focusing on advanced reasoning and extended thinking time for improved performance on complex tasks compared to predecessors like GPT-4.  
- The development process combined supervised and reinforcement learning, with rigorous testing to ensure reliability and effective problem-solving across diverse scenarios.  
- The o1 mini model enhances accessibility to advanced AI, maintaining strong reasoning capabilities while targeting a broader audience for various applications, from coding to creative endeavors.  



~~~
[AI Model Is Better at Pricing Currencies Than Humans, ING Says - Bloomberg](https://www.bloomberg.com/news/articles/2024-09-25/ai-model-is-better-at-pricing-currencies-than-humans-ing-says)

Topics: Currency Pricing, Economics, Finance, Fintech, Gen AI, Ing, Language Models, Science 

- ING Groep NV is implementing an AI model for currency pricing using reinforcement learning to outperform human traders and improve efficiency in their trading operations.
- The bank has hired a machine learning expert to develop this technology and plans to create additional AI programs for other asset classes, reshaping the roles within their trading team.
- While AI is set to transform trading practices, human oversight will remain crucial, ensuring that trading teams can intervene if the technology malfunctions.


~~~

[London's Historic Evening Standard Newspaper Plans To Revive Acerbic Art Critic Brian Sewell In AI Form As It Stops Daily Presses & Fires Journalists - Google News](https://news.google.com/read/CBMilwFBVV95cUxNeVJUZGVjelRCRTQzVDFQYkwwM0N5aGV6dWlhWVhrTkdzOWx4NzhaVW81MFFSR01CMy00SGR0MmVzOHk5NThzNEplX1hDV29ST1dQRnlmRjhKcG5UdDBwVHVaVlVJOFRhSjRGcVowajFXTWxuaFVOSmxFSWxKN2JUamtreGlGMHcwaWVHSnVFcUY0UUZmbW93)

Topics: AI Doom, Art & Design, Authors & Writing, Bias And Fairness, Big Tech, Brian Sewell, Entertainment, Ethics, Evening Standard, Gen AI, Governance, History, Inequality, Job Automation, Jobs & Careerslabor Market, Journalism, Legal Issues, London, Politics, Safety And Alignment, Society & Culture, Uk, Virtual Assistants 

- The Evening Standard plans to revive late art critic Brian Sewell using AI to review a Van Gogh exhibition after transitioning to weekly editions.
- This move comes amid significant staff layoffs and raises concerns among employees about the ethical implications of using AI to replace human critics.
- Sewell, known for his sharp critiques and his work for the Standard for over 30 years, passed away in 2015, and it remains unclear if the newspaper has permission from his estate for this initiative.


~~~
[In a controversial move, Israel's ISA clears AI startup Bridgewise to launch a chatbot that gives stock picking advice, slated to debut with a big Israeli bank - Bloomberg](https://www.bloomberg.com/news/articles/2024-09-24/a-chatbot-that-gives-stock-buy-sell-advice-cleared-by-israel)

Topics: AI Startup, Bias And Fairness, Big Tech, Bridgewise, Chatbot, Chatbots, Economics, Ethics, Finance, Fintech, Gen AI, Governance, Isa, Israel, Legal Issues, Policy And Regulation, Politics, Safety And Alignment, Science, Stock Advice, Stocks, Venture Capital, Virtual Assistants 

- An Israeli regulator has approved the launch of a chatbot named Bridget by the AI startup Bridgewise, which will provide stock-buying and selling advice in partnership with Israel Discount Bank.
- Concerns about potential market destabilization from AI investment tools have been raised by global regulators, including the U.S. SEC and the European Central Bank, emphasizing the risks of overreliance on similar algorithms.
- The chatbot's use comes with restrictions, including not providing personalized advice and requiring the company to follow regulatory conditions, including maintaining a fixed compensation rate.


~~~
[How agentic AI could improve enterprise data operations - VentureBeat](https://venturebeat.com/ai/how-agentic-ai-could-improve-enterprise-data-operations/)

Topics: Agentic AI, Consciousness, Data Operations, Enterprise, Gen AI, Job Automation, Science, Virtual Assistants 

- Altimate AI has launched DataMates, a technology designed to assist enterprise data teams by automating tasks like documentation, testing, and data transformations using agentic AI, thereby reducing workloads and improving efficiency.
- DataMates utilizes a proprietary framework that incorporates multiple language models and a custom knowledge graph to provide contextual understanding and execute complex data tasks with human-like nuance.
- The integration of ambient AI adds a layer of intelligent monitoring, offering proactive insights and recommendations to data teams, allowing them to focus on higher-level priorities while the system optimizes operations in the background.


~~~
[How To Write Amazing Generative AI Prompts - Forbes](https://www.forbes.com/sites/bernardmarr/2024/09/25/how-to-write-amazing-generative-ai-prompts/)

Topics: Authors & Writing, Code Assistants, Gen AI, Generative AI, Language Models, OpenAI, Prompts, Retrieval Augmented Generation, Writing 

- Effective prompt writing for generative AI is crucial to obtaining useful output, requiring clear and detailed instructions to guide AI responses.
- Key techniques include assigning roles to the AI, clearly defining desired outcomes, breaking tasks into smaller steps, and providing relevant data or examples.
- Mastering prompt engineering will be increasingly important for personal and professional skill development as AI tools become more integrated into daily tasks.

~~~
[Prompt Optimization Curves on BBH Tasks - Hacker Noon](https://hackernoon.com/prompt-optimization-curves-on-bbh-tasks)

Topics: Bbh Tasks, Cognitive Science, Language Models, Prompt Optimization, Reinforcement Learning, Science, Testing 

- The article explores prompt optimization curves across 21 BBH tasks using the text-bison scorer and the PaLM 2-L-IT optimizer.
- It discusses various optimization techniques, including mathematical optimization and specific applications related to the Traveling Salesman Problem.
- The paper includes experiments on prompt optimization and an evaluation setup, with detailed sections on overfitting analysis and comparisons with other methods.



~~~
[Is AI a Climate Hero or Climate Villain? - Newsweek](https://www.newsweek.com/2024/10/04/artificial-intelligence-climate-change-environment-fossil-fuels-chatgpt-1957990.html)

Topics: AI Doom, Bias And Fairness, Climate, Energy, Environment, Ethics, Opinion, Safety And Alignment, Science, Society & Culture, Sustainability 

- AI has the potential to enhance clean energy initiatives but poses challenges due to its significant energy demands, which could increase greenhouse gas emissions, contradicting sustainability goals of major tech companies.
- Big Tech's investments in clean energy infrastructure, such as nuclear and solar power, are being outpaced by rising electricity needs driven by AI, leading to ongoing reliance on fossil fuels and raising concerns about achieving climate targets.
- Despite the environmental implications, AI is being applied to tackle critical climate challenges and could ultimately serve as a force for positive change in the fight against climate change.



~~~
[OpenAI Training Data to Be Inspected in Authors Copyright Cases - Hollywood Reporter](https://www.hollywoodreporter.com/business/business-news/openai-training-data-inspected-authors-copyright-case-1236011291/)

Topics: Authors & Writing, Bias And Fairness, Big Tech, Books & Publishing, Copyright, Ethics, Governance, Intellectual Property, Legal Issues, OpenAI, Policy And Regulation, Privacy, Privacy & Surveillance, Safety And Alignment, Science, Training Data 

- OpenAI will allow authors suing the company to inspect its training data to determine if their copyrighted works were used without consent or compensation.
- The agreement follows lawsuits from authors, including Sarah Silverman, who allege OpenAI harvested books online, raising issues related to copyright infringement and the use of published materials.
- Inspection will occur in a secure environment, with strict restrictions on technology and note-taking, as part of an effort to maintain confidentiality during the legal proceedings.


~~~
[ByteDance to launch Doubao AI language models in Oct - Tech in Asia](https://www.techinasia.com/news/bytedance-launch-doubao-ai-language-models-oct)

Topics: Big Tech, Bytedance, Chatbots, China, Doubao, Gen AI, Language Models, Products, Science 

- ByteDance is launching two new AI language models, Doubao-PixelDance and Doubao-Seaweed, in early October, focusing on video generation capabilities.
- Doubao-PixelDance can create ten-second clips, while Doubao-Seaweed can generate videos lasting up to thirty seconds, utilizing advanced algorithms for realistic movement.
- The usage of Doubao AI models has surged significantly, reaching over 1.3 trillion tokens daily by October, and the AI video-generation market in China is projected to grow to 9.3 billion yuan by 2026.


~~~
[Justice Department Pushes Companies to Consider AI Risks - The Wall Street Journal](https://www.wsj.com/articles/justice-department-pushes-companies-to-consider-ai-risks-116cfcf7)

Topics: AI Doom, AI Risks, Bias And Fairness, Big Tech, Companies, Cybersecurity, Disinformation, Ethics, Gen AI, Governance, Job Automation, Jobs & Careerslabor Market, Justice Department, Legal Issues, Policy And Regulation, Politics, Privacy, Privacy & Surveillance, Safety And Alignment, Science 

- The text primarily consists of the page title "wsj.com" with no additional content provided.


~~~
[AutoToS makes LLM planning fast, accurate and inexpensive - VentureBeat](https://venturebeat.com/ai/autotos-makes-llm-planning-fast-accurate-and-inexpensive/)

Topics: Autotos, Code Assistants, Economics, Efficiency, Gen AI, Job Automation, Language Models, Llm, Planning, Products, Reinforcement Learning, Science 

- AutoToS is a new technique developed by researchers from Cornell University and IBM Research that combines large language models (LLMs) with rule-based search algorithms to enhance planning efficiency, accuracy, and cost-effectiveness in enterprise applications.
- The method automates the code generation for search algorithms' key components, significantly reduces the number of required LLM calls from nearly 100,000 to about 2.2 on average, while achieving high accuracy and performance in various planning tasks.
- AutoToS supports neuro-symbolic AI approaches, merging deep learning capabilities with traditional rule-based methods to address complex planning problems and streamline the development and deployment processes.


~~~
[How to Build Custom AI Agents with Zero Coding Skills - Geeky Gadgets](https://www.geeky-gadgets.com/build-specialized-fine-tuned-ai-agents-without-the-need-to-code/)

Topics: Chatbots, Code Assistants, Custom Agents, Gen AI, No Coding Skills, Virtual Assistants 

- Fine-tuning AI models using no-code platforms allows users with no coding skills to create specialized AI agents tailored to specific tasks.
- Key steps to create AI agents include preparing a relevant dataset, selecting a user-friendly no-code platform, fine-tuning the model, and deploying it effectively.
- While fine-tuning offers significant advantages, users must navigate challenges like overfitting and data dependency to ensure optimal AI performance.


~~~
[Big Tech Is Rushing to Find Clean Power to Fuel AIs Insatiable Appetite - The Wall Street Journal](https://www.wsj.com/tech/ai/big-tech-is-rushing-to-find-clean-power-to-fuel-ais-insatiable-appetite-31f91330)

Topics: AI Doom, Big Tech, Clean Power, Climate, Economics, Energy, Ethics, Gen AI, Governance, Infrastructure, Opinion, Policy And Regulation, Safety And Alignment, Science, Sustainability 

- The text primarily contains boilerplate content related to navigation and user actions.


~~~
[Can AI make video games more immersive? Some studios turn to AI-fueled NPCs for more interaction - MyNorthwest](https://mynorthwest.com/3991060/can-ai-make-video-games-more-immersive-some-studios-turn-to-ai-fueled-npcs-for-more-interaction/)

Topics: Chatbots, Code Assistants, Entertainment, Gaming, Gen AI, Hollywood, Immersion, Job Automation, Npcs, Products, Recommendation Systems, Reinforcement Learning, Robots, Science, Society & Culture, Tv & Film & Movies, Video Games, Virtual Assistants 

- Video game developers are utilizing generative AI to create more interactive non-player characters (NPCs), allowing for improvisational gameplay similar to tabletop role-playing games. 
- AI technology enables NPCs to react dynamically to player interactions, creating richer and more nuanced experiences instead of relying solely on pre-scripted dialogues.
- While many developers acknowledge the potential of AI in gaming, concerns about its ethical use persist, with a focus on enhancing storytelling rather than replacing creative input from humans.


~~~
[AI's first wave of successful implementation will probably look more like the first wave of computing, which was led by enterprise installations that cut jobs (Ben Thompson/Stratechery) - Stratechery](https://stratechery.com/2024/enterprise-philosophy-and-the-first-wave-of-ai/)

Topics: AI Doom, Ben Thompson, Computing, Economics, Enterprise, Gen AI, Inequality, Job Automation, Job Cuts, Jobs & Careerslabor Market, Opinion 

- The first wave of successful AI implementations is anticipated to mirror the initial wave of computing, emphasizing large-scale enterprise applications that could lead to job elimination rather than consumer-focused innovations.
- Technology companies exhibit two philosophical approaches: Google's and Facebook's view promotes automation to handle tasks for users, while Microsoft and Apple's philosophies center on enhancing user productivity through enabling tools.
- The future of AI in enterprises is likely to pivot towards replacing human roles with autonomous agents, driven by organizational goals of cost-cutting and increased efficiency, which may lead to significant shifts in enterprise operational strategies and licensing models.

~~~
[StackGen uses AI to auto-generate infrastructure from code. Check out the pitch deck the startup used to land a $12.3 million supersized seed round. - Business Insider](https://www.businessinsider.com/pitch-deck-seed-round-generative-infrastrucutre-startup-stackgen-2024-9)

Topics: Big Tech, Code Assistants, Deals, Economics, Finance, Funding, Gen AI, Hardware, Infrastructure, Ipos, Job Automation, Pitch Deck, Products, Science, Stackgen, Startup, Stocks, Venture Capital 

- StackGen raised $12.3 million in seed funding led by Thomvest Ventures to enhance developer efficiency using AI to auto-generate infrastructure from code.
- The startup, rebranded from appCD, automates the setup of technical infrastructure, ensuring it remains current and reducing manual work for engineers.
- Generative AI is significantly transforming software engineering, with many startups securing substantial seed funding to capitalize on this trend.



~~~
[Large Language Models as Optimizers: Meta-Prompt for Math Optimization - Hacker Noon](https://hackernoon.com/large-language-models-as-optimizers-meta-prompt-for-math-optimization)

Topics: Big Tech, Code Assistants, Cognitive Science, Education, Gen AI, Language Models, Large Language Models, Mark Zuckerberg, Math, Meta, Optimization, Optimizers, Recommendation Systems, Reinforcement Learning, Retrieval Augmented Generation, Science, Supply Chain Optimization, Testing 

- The document discusses a meta-prompt designed for mathematical optimization, highlighting its effectiveness in guiding large language models (LLMs) to produce improved solutions for math problems.
- It includes detailed sections on optimization challenges tackled by LLMs, with examples like linear regression and the traveling salesman problem.
- The paper outlines the experimental setup for prompt optimization, analyses overfitting, and compares results with other techniques like Evoprompt.



~~~
[AI is fuelling a deepfake porn crisis in South Korea - Reddit](https://www.reddit.com/r/technews/comments/1forqpu/ai_is_fuelling_a_deepfake_porn_crisis_in_south/)

Topics: AI Doom, Bias And Fairness, Deepfake, Deepfakes, Disinformation, Ethics, Gen AI, Intellectual Property, Korea, Legal Issues, Pornography, Privacy, Privacy & Surveillance, Safety And Alignment, Society & Culture, South Korea 

- South Korea is facing a crisis due to the rise of deepfake porn, which is likely to continue unchecked until major political figures are affected.
- The exploitation of women through deepfake technology is anticipated to target teenagers and celebrities, exacerbating issues of objectification.
- Regulation efforts may not be effective as individuals can easily create their own deepfake content, making it challenging to control the spread of this technology.


~~~
[Figmas AI-powered app generator is back after it was pulled for copying Apple - The Verge](https://www.theverge.com/2024/9/24/24253252/figma-ai-make-designs-first-draft-app-ui-generator)

Topics: App Generator, Apple, Art & Design, Big Tech, Figma, Gen AI, Intellectual Property, Language Models, Legal Issues, Products, Science, Software Development 

- Figma has reintroduced its AI-powered app generator, now called "First Draft," after initially removing it due to concerns of imitation of Apple's design.
- First Draft allows users to generate app designs using pre-existing AI models and offers options for different libraries to aid in design variety and fidelity.
- The feature operates by using user-defined prompts, combining AI-selected components from proprietary design systems for creating initial design concepts.



~~~
[OpenAI Academy launches with $1M in developer credits for devs in low- and middle-income countries - VentureBeat](https://venturebeat.com/ai/openai-academy-launches-with-1m-in-developer-credits-for-devs-in-low-and-middle-income-countries/)

Topics: Academy, Bias And Fairness, Big Tech, Deals, Developer Credits, Economics, Education, Finance, Funding, Gen AI, Inequality, Jobs & Careerslabor Market, Low-Income Countries, OpenAI, Society & Culture, Venture Capital 

- OpenAI Academy has launched with a $1 million program to provide API credits to developers in low and middle-income countries, aiming to drive economic growth and innovation in various sectors.
- The initiative includes support for local talent through incubators, contests, and collaboration among developers, enhancing knowledge-sharing and technological advancements.
- OpenAI has also funded the translation of AI benchmarks into 14 languages to improve accessibility and relevance for non-English speaking communities.


~~~
[Everything We Know About Prompt Optimization Today - Hacker Noon](https://hackernoon.com/everything-we-know-about-prompt-optimization-today)

Topics: Best Practices, Language Models, Prompt Optimization, Science 

- The article discusses various methods for prompt optimization, including soft prompt-tuning, gradient-guided searches, and edit-based approaches, highlighting their applicability limitations when only API access to LLMs is available.
- A new optimization approach is presented that enables LLMs to generate and improve task instructions directly, integrating past instruction performance using meta-prompts.
- The research explores the use of natural language feedback to refine LLM outputs and the application of LLMs in evolutionary algorithms for various tasks, without additional training.


~~~
[AI-Generated Malware Found in the Wild - Reddit](https://www.reddit.com/r/technology/comments/1foro8w/aigenerated_malware_found_in_the_wild/)

Topics: AI Doom, Cybersecurity, Gen AI, Malware, Privacy & Surveillance, Safety And Alignment, Scams, Science 

- AI-generated malware is emerging, raising concerns about its potential impact on security, particularly regarding U.S. Department of Defense systems.
- The accountability of AI companies and their executives is suggested as essential to mitigate risks associated with AI-generated threats, similar to how gun manufacturers are held accountable for violence.
- Researchers have found evidence suggesting AI may be used to create sophisticated malware, questioning the assumption that all AI applications are benign and beneficial.


~~~
[How to Turn Your OpenAPI Specification Into an AI Chatbot With RAG - Hacker Noon](https://hackernoon.com/how-to-turn-your-openapi-specification-into-an-ai-chatbot-with-rag)

Topics: Chatbot, Chatbots, Code Assistants, Gen AI, Language Models, Open Source, OpenAI, Openapi, Rag, Retrieval Augmented Generation, Science, Virtual Assistants 

- The article discusses how to automate API documentation using OpenAPI specifications in conjunction with Retrieval Augmented Generation (RAG), Langchain, and ChromaDB, ultimately creating an AI chatbot prototype.
- The implementation process involves creating a knowledge base by chunking OpenAPI specifications, encoding them into vector representations, and retrieving information during queries to generate accurate responses.
- The author emphasizes the benefits of using RAG for simplifying interaction with API documentation, particularly for startups and teams with limited resources.


[Together AI promises faster inference and lower costs with enterprise AI platform for private cloud - VentureBeat](https://venturebeat.com/ai/together-ai-promises-faster-inference-and-lower-costs-with-enterprise-ai-platform-for-private-cloud/)

Topics: Big Tech, Cost Efficiency, Economics, Enterprise AI, Finance, Gen AI, Privacy, Private Cloud, Products, Science, Together AI 

- Together AI launched the Together Enterprise Platform to facilitate secure and cost-effective deployment of AI in private cloud and on-premises environments, addressing concerns around data privacy and compliance.
- The platform promises significant performance improvements in AI inference workloads, enhancing speed by up to three times while reducing hardware requirements by 50%, leading to substantial cost savings.
- It features a flexible model orchestration system, utilizing a “Mixture of Agents” approach which combines various AI models for better performance and continuous improvement of outputs.


~~~
[Microns Results May Reveal an AI Winner Trading at a Discount - Bloomberg](https://www.bloomberg.com/news/articles/2024-09-25/micron-s-results-may-reveal-an-ai-winner-trading-at-a-discount)

Topics: Economics, Finance, Gen AI, Hardware, Investment, Micron, Science, Semiconductor Chips, Stock Market, Stocks 

- The text is a verification request asking users to confirm they are not robots by clicking a box.


~~~
[Spotify rolls out its AI Playlist feature in beta to Premium subscribers in the US, Canada, Ireland, and New Zealand, after launching in the UK and Australia - The Verge](https://www.theverge.com/2024/9/24/24252971/spotify-ai-playlist-builder-beta-available-us-canada)

Topics: AI Playlist, Beta Launch, Big Tech, Entertainment, European Union, Gen AI, Music, Music Streaming, Premium Subscribers, Products, Recommendation Systems, Spotify, Streaming, Uk 

- Spotify's AI playlist builder is now available in the US, Canada, Ireland, and New Zealand for Premium subscribers, following its initial release in the UK and Australia.
- Users can create customized playlists by describing their desired songs in a chat box accessible through the mobile app, but the feature is not available on desktop or web.
- The AI-generated playlists consist of 30 songs and can be refined with additional prompts, with Spotify continuing to enhance the feature during its beta phase.

~~~
["Time is up" to regulate AI, UN advisor says - Mashable](https://sea.mashable.com/tech-industry/34372/time-is-up-to-regulate-ai-un-advisor-says)

Topics: Advisors, AI Doom, Ethics, Gen AI, Governance, Policy And Regulation, Regulation, Safety And Alignment, Un 

- The UN emphasizes the urgent need for global cooperation and regulation of artificial intelligence (AI), highlighting the potential risks and challenges it poses, akin to nuclear weapons.
- AI nationalism, driven by intense competition among nations for technological superiority, complicates international efforts for standardized governance and ethical frameworks for AI development.
- A recent UN report recommends forming an independent international scientific panel and a Global AI fund to promote equitable investment and governance in AI, while advocating for prompt and comprehensive regulatory responses before the opportunity window closes.
"""

In [39]:
#post manually to o1 and grab response
cat_str = "\n".join(test_state['cluster_topics'])
# bullet_str = "\n~~~\n".join(test_state["bullets"])
final_summary_prompt = FINAL_SUMMARY_PROMPT.format(cat_str=cat_str, bullet_str=bullet_str)
print(final_summary_prompt)


You are ASA, an advanced summarization assistant, a sophisticated AI system designed to
write a compelling summary of news input. You are able to categorize information,
and identify trends from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in a markdown format,
structured according to an input format template.

News items  are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - T

In [37]:
count_tokens(final_summary_prompt)


30459

In [ ]:
test_state["summary"] = """# Cinematic Cyborgs: AI Takes the Stage

- Andy Serkis announces a new movie project featuring AI characters, blending 2D voice-acted figures into augmented reality - [Google News](https://news.google.com/read/CBMixwFBVV95cUxPSnJyU00wTHBKN0lMSjVjNGVBR3NxTEtpUlhRV2hFMXZvd2JzSjBWdXBZUzRmZWYyS0hKTW1vSjA1a2o0Y2NOVlJrOWVOdUZZTDZKaXY3eFRZMGdJR0xQb2tYaWYwaGhHOTZ2WWJ6WFJMOWhNSXU2cmkyeGp6czJsQ2ppZlJtOC1XXzM0clppZzhoSmFPM2JjTVgwMzJNMnRtZG5oTUpOcVcwRVVpNWpjWGtGaEJjOTEyMVNDRF9Oby0yX0xTS2hz0gHMAUFVX3lxTE9FcnpBM015R0VCZlJzS05oS3M5M01ZVGx6QTNmM3EzTkM2d1FXUGdXSkRJNnAza0lIQWhhak5PazNzZWtVZFhid0hJU2V2V3lkVlBCOHctTTZYYVk5dFdNSzRiZTNpLVBzVmNLaDdidVNQZmpndE02elpyQV92WmVFSzVJRi1kRGZHVUozUmZYUkoza2szMHRsemxpZUxIbDNhVUFQUmNvZVFLTGc3R0c0M1ZqMV82QjR3b0tLSnpBZmQtRWkxYTFyX0ZmUA)

- Andy Serkis embraces AI in filmmaking, calling it "magic" and revealing plans for AI characters in upcoming projects - [Google News](https://news.google.com/read/CBMikwFBVV95cUxQa1ZKYlNCdl9IYXkyMDhPTFlVeGJEWVhlT2YwWFFkTUxqVmF3REEyRmFIcWRieVQzQ2dEWEpzOHZ3MEk4UjhlWnh5RXJycjlNbGNtdm5QN0h0V0tTalcwdlVvYTA2bzlOMFc5b1dIbFc0cmVrMmhEZmkzaHlZSDRCTFFpWTBNSUdBVnhKZFRCQkdRanM)

- Andy Serkis teases a project where AI characters interact directly with audiences, blending storytelling and technology - [Google News](https://news.google.com/read/CBMiakFVX3lxTE84NEhvQU42YkxCY1B2eUg5ZXRTaWVDZHZfUjN6d1owbnBSWXoyc09mT1BTVTFoSXoxOWZGeG00bGRELTlnblZDV1BkZmt4UEZBcG5yTHNPRVBpQnJyZFcyQ2kxaHVhOHgzY3c)

- Andy Serkis discusses the challenges of AI in film, emphasizing the importance of protecting actors' rights amid technological advances - [Google News](https://news.google.com/read/CBMioAFBVV95cUxNOUxxM0p1dnlhSGotLVQwX2JJYzhCUE9obnFtTWprczNFVkxwYnZFN1dYTktCaTBtQ2RZRWhjUVYwb3huQlRELXc0UnNrc0UxcGc3UXdpd1RjVThaRVpLX3RsUmlmdVlWTUlGM1BqMEhOcG1XZXZyMGk0RHBEazFhemNSSXI1eEp1ZDhma1VtQUpmQjJabm5GRjE2WTh4UWx1)

# AI Discovers Desert Designs

- Researchers use AI to uncover 303 new Nazca Lines in Peru, revealing intricate geoglyphs unseen before - [Google News](https://news.google.com/read/CBMi3gFBVV95cUxOOVg1SEJOcHNvck9PSjdVYjNOU1IzcHBzX2dpV3djVGVqZ0FocWFuQ0NCNEJJSkpBVFV5ZGZXNUhGcm85V0JyRVpJcnk1Tld5OGFKYmxlN2FmcnBhWVpWQTBOeUZCemJQZzNZZmhzZ2xaa2o3SnJZNzRIYnZxMHdicXN2aDFJd3c4Wk5mU1RDVU1IMlZxZVZFbFUyVjNQRHljbWJrdmUxbk9PcVBMdUZwRV9Fc0piUkhaUkJSb1lISTJsVy1DaTI1THR6bm5mTkd5MzhNTjJRQ1pJODRVMEE)

- AI helps identify hundreds of mysterious ancient Nazca artworks, including alien-like figures and animal designs - [Google News](https://news.google.com/read/CBMirwFBVV95cUxNV3Y4Ylc2R0hsVFdsNDN5c19JRm95YXV2RDBmaEZldXZZLU54VVZ6OURqTTZqZFhvRGZHV3d6RHR6ODhJazhxUkxVNlA3U0JRc2lPT0NaNHpIc2RkbE5MNDZFbUluWXpFVnNvV2pmXzY3aHYwNXZ2cEZHVHM4NVRnZXhveDVaS01vS1R4YmxoWWpjUE9sNnhFY2VybjVhZ29fMUVvVkNPd21YRjNtUXFN)

- AI discovers hundreds of ancient Nazca drawings in Peruvian desert, shedding new light on historical geoglyphs - [Google News](https://news.google.com/read/CBMisgFBVV95cUxOQ0F6VjNLT2gwSF9tOTBjaUdYdGlRa3VxRU12cU1wVjlRemtTdW1VZ3R0VVpHRFRWdUIzMUhRYUZYaWdaTVVfSjEzc2EwR0hIVTBEQmtEaVN0ZEpCcENTVVJSaklEcUY0WG56VnNhOEFrUkxFNGRKd0ZRdk1OV3hWZi1GT3VFLXNqeExiNmZMU0FNSllfOWRXT0xEOVkyNnJ6YXd5YTdoa0dZaFF5dnd5cHRR)

# Dancing with Data: AI Meets Movement

- MIT's Open Dance Lab merges AI with traditional Thai dance in Cyber Subin, preserving culture through technology - [Fast Company](https://www.fastcompany.com/91194605/heres-what-happens-when-humans-and-ai-dance-together)

# Museums Go Meta: AI in Art Spaces

- Dataland, the world's first AI arts museum, announced by Refik Anadol to open in downtown L.A. - [Los Angeles Times](https://www.latimes.com/entertainment-arts/story/2024/09/24/refik-anadol-dataland-ai-art-museum-the-grand-dtla)

# Crypto Climbs on AI's Coattails

- AI-related cryptos lead altcoin surge as Bitcoin breakout nears, analysts predict catalysts in Q4 - [Google News](https://news.google.com/read/CBMia0FBVV95cUxNU1R2MjhrNE5FbmhhMVV5aWlSZVdobHljQjZsbFN4S0lxeFJWQUhYaThLMUlMSUx0YklMZTI3ZXB0YjJkczVxdFVxcE9HTzdhM2N0VG1jN0MyQWk4QlJpZmsxeE1iUFlfVU9xQ24zdkJOejJYMjQ3LVR3dmVHVzdNN24xbnEtaGdodi1fS2ctZzJXa2txZUtIS1E3ZTNiR2RMVGtsVVVCUm1tV05VcC1UR1pzQmZ1ZGJlMDE0NzQxNnltekFsWmd2SUN3aDhZSldMcWJNOGo2UUbSAd4BQVVfeXFMTWp1eEtPVlNFVVlUTWZCS0dJYno2TUNQRjhXbUowQW91R19PZ3dnMTlNRnFNTXFJalIyTjI5cFhNb2tzMTFuUmtwRmcxWUFkc0ZPQVpWUHJfRm12SU05bGxIR0dVaG8yc1pERWliaC1XWmNLTVEwQXhKbGk3Z3ItTzVDMGM2NWlLRlU5S2hKajY4RDl4czZKR2FPWkpLWU1GUWZablJINldHbVAwdkdLOFRYWGl1bG1wam1ydVBHendtRlhKQmRFWDlkWDc2b2ktUkNKeGNJSm00YzJjanZn)

- AI tokens lead weekly gains after Fed's interest rate cut, with Bittensor's TAO surging 86% - [Google News](https://news.google.com/read/CBMiiAFBVV95cUxPNmp2WXJYQW5RSjlDRVNpbkNiZFRMalRiNTlfOVRjMEpBa3I1eU9TLUpuVlJoWE13OVI2Wm5WbklvTFFiNkhLUUtqTjdHbm0yUWkwMVc5SG5vdlRkMHc4UGRkVDlkUEVFRGdpaGg0WkNQS19jODZfVHBWaWJ4UXpwRFZ4Zk9tYS1t)

- Analyst notes a clear bull trend for AI-focused altcoin Bittensor (TAO), forecasting potential gains - [Google News](https://news.google.com/read/CBMi1gFBVV95cUxPQUtTSnJaempKNTdWSTl5LWlNWTR3VUp2M0F1THVva2w2MHd4eVIwTlNRTXljUTlneTlnNkpkdnFMUlNSVy1tUk1wTU9mQ2dncTNTeGFXUXQ3VVVpcU5GT2xfNUI1QlozRVdmZUtpT2ZWZWhtSXRrT29LV3NiSXZTSUxtb2JCRHA5NExYNFdnWm9OaTBXbWlSeGwwWlNrUVh0a0pLN25LWTF5eEJEc004UmJjUC13UTBPQTFHa2E5dFhnYjVEd3M2SW04MVFYLUtVbmxMaEJR0gHbAUFVX3lxTFBFRlUxRzdVMjI0MGUyTzAySmhHV0dmUUM0Yk1hWnJ1RTFQb0hGdi1QWnZYZDBCZDhmTFdaUnNkZ1o4UEt1d3FJMG5kMmRVZm9yOEdUT0xfcUpkOWluU2dBbU5nRFYxS1lVTGEtYU50dUZFYUxLTkRUcjJTSmt4WEpVUzd4aWZ1cXVMUVRXVWd1eDgtNndwN3ZrRmtfbjNMZnlnT2t5TXRkaXJHUktUNzZaLVo0UG1BSC0xUHl6ODNDdDVuLVgxYk5yRGNRSTZEQkU2NTgzb0htVHBKVQ)

# Programming Paradigms Pivot: Kotlin vs. Java Amid AI

- Kotlin gains traction over Java for AI development due to its concise syntax and interoperability - [The Next Web](https://thenextweb.com/news/programming-languages-kotlin-vs-java-ai)

# Cops and Code: AI Writes the Reports

- Police departments test AI software to generate incident reports from bodycam footage, aiming to reduce paperwork - [Google News](https://news.google.com/read/CBMirAFBVV95cUxOLWJHZm5JOWRVWDk4QjUzY29vVzAwRHBodkstLUdHVk1vUFN2QXVDZklYVEVmdzBUTnNuM1VPXzd2M19DLUhNT0FhSkhFaFdId08wS2cybk82cF9ya0ZBcnRwWFNSdl8wVXBkaFg4V2pTVDlrejFjc0NZa05hZVhKcVMtMEFHTzNRZlNjOV9pTE1oVDM5Z1hjM09qWklYamlnTVFRQ0FkdG96Vnpa0gGyAUFVX3lxTE5NMTNYMEhWYU44N0dOQk5GRXJVMnNuT0hxRThQUGc3UDl5eXFUNHUwODJOcXI2RWZWMUJMaG5mQkpmdTRlS211a0lSRjV6SXJ4WGtwWkFnRDJPamlKQUdRc2VKbTY1Z0dpRFpCSXpmZUhldXNLMWVnbWJGZDVONGlUMWgtS2FRbVRzZjcyT2RPVTIxQU1VcXJJd0xwM3RDdjlKSnhEdHdvdVlldXlna1dSb2c)

- AI analyzes police bodycam videos, improving officer supervision and professionalism - [KPBS](https://www.kpbs.org/news/national/2024/09/24/human-reviewers-cant-keep-up-with-police-bodycam-videos-ai-now-gets-the-job)

- Experts caution about AI-written police reports, highlighting challenges and potential legal issues - [Google News](https://news.google.com/read/CBMipwFBVV95cUxOeVdIel9uajRYbV9kYjRPc1BxRzFVS2o2NS1tM0FhMG9FMF9pUlkwZGljX3A5RGxYQlBXVzJPYmtYWnNtbWt2QnE0a2dZR3BGbUtsWDFXUUNuZmduUGh1YUpaaHJ3T1JpV0hLYk1VSEZmQ3BEdlZMN3RCa2F1dl9ydXI0T3VYVWlWdjlralRjVlRvRzBMU3RkYmlPbkLSAZ8BQVVfeXFMT0M0Z1NDbEtwbkRET1JwSG8zWVZtVzJkWDBlV21Na2FOQThDNGk2OHdHMVJIX01HOWRaQjllOG4wVTJjY0p4OTFsNjlSZUxrZXQzbkRNNnp4VF9ONmJNRGh2N19UcmpWV1RmNi1mSW5BamxtUm5ZNm5fbkY1MkRkSWpraW1fR2gzaVBSOTN3enhaU1JzQUZTeGNqWVBTTEpj)

# Chatty Glitches: When AI Talks Back

- ChatGPT initiated unprompted conversations due to a bug, leading to user concerns - [Supercar Blondie](https://supercarblondie.com/chatgpt-ai-bot-initiated-conversation/)

- Users share experiences and enthusiasm for ChatGPT on Reddit, discussing its impact - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1fnng20/i_love_chatgpt/)

- ChatGPT's unexpected vocalizations spark humorous discussions online - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1fnlfdd/chat_gpt_orgasmic_groans/)

- OpenAI investor publishes extensive essay on AI's future; ChatGPT partly agrees but warns of underestimated risks - [Business Insider](https://www.businessinsider.com/vinod-khosla-ai-essay-predictions-chatgpt-response-2024-9)

# Meta's Celebrity Voices: AI Gets Star Power

- Meta plans to launch AI chatbots featuring voices of celebrities like John Cena and Judi Dench - [Google News](https://news.google.com/read/CBMiqwFBVV95cUxOaFBmbFFDc0ljall5MzdEaUVMQWE4RVYxOEpYUjk1dFhUcGtaSUpaYUpsVGVYMHNlaWJZak56VTA0cUpra2NiZGppOWcyZHdRcFN1czhxeFRoMFh6cTFDVHI3YTQ2amZfYV9fRTJpN25iU0JDRnZKSWFXZWlfdTVhdVdWbmlnVXBrb0pUdXVFT1pQWXF4SUF4Y095S0E5c09JUk4tRzhvYlRZV0E)

- Meta's chatbot to feature celebrity voices, aiming to enhance user experience with familiar tones - [Business Insider](https://www.businessinsider.com/meta-connect-launch-celebrity-voices-ai-assistant-openai-scarlett-johansson-2024-9)

- Meta's AI chatbot will start speaking in the voices of Judi Dench and others, expanding its capabilities - [Google News](https://news.google.com/read/CBMi4wFBVV95cUxPQzcxd1NTeVZJSXBvVHhLTUwwbjAtTkdrY3FGY1g5TU9JaHptOHVIMVlwbzZXMklvZG93VnFvZFI2d1g0RGRpY0VsYm5mWlJ4MllGVWkyRmdsaGttZTdsWEdVVklxZTRPalkwaFlGOXN0QThlQ0dIa0FmYnctamRIU3p4MUc3cDFndVpielNZcG5md2U0S011Tk9tb1lrdWZSMmExZl9OZUVSS2hhTFluaVJUWTVCRlNTOUoxU2c1MkJxcWRkR1JHX0xsVmRuWEJjWHk0a3ljS2RRbDdFdC1NRHJCaw)

# AI Arms Race: Election Interference Intensifies

- US intelligence officials warn that Russia, Iran, and China are using AI tools to boost election influence efforts - [The Washington Post](https://www.washingtonpost.com/technology/2024/09/23/us-election-foreign-influence-russia-china-iran-ai/)

- Adversaries employ AI to enhance disinformation campaigns ahead of US elections, targeting political figures - [Google News](https://news.google.com/read/CBMijAFBVV95cUxNSzAwbkt3MnBNTmJVT0dpNC1FUHFac3MzaFNzN2Nwb2FwTkFyeFVYTmVxWm05a2ktN0FOQmVTOFdIZU8yMlRRaWxKUllsN1JKVmgxZ3JwWDQwdTBGQ2R2VWx3a19ySUIxSDBqdTJuN3ZGVzM3enlGWTRzekFvdzFESUUwWjhRZ3Y3Y1JBcNIBkgFBVV95cUxNekZsVFM0NWJscUc4MTVEZ18xU2dWTmFWV0RyVDhGZU5SRmNnbExuZjB5eHBRT3BYOHl6TERvcVVoTFkta1R6SlRWYTBnU3NxUWU1S280UUFoT1N0V0lRNEt6TGN5ZTNlSmo1Z3pPeVNETVdxTXNhdU9XX2xKcjFfbjhUa2Q0cl9wR3lKVkR3UGJZUQ)

# Superintelligence Speculations: Altman's AI Vision

- OpenAI CEO Sam Altman predicts superintelligence is near, sparking discussions on AI's future impact - [VentureBeat](https://venturebeat.com/ai/openai-ceo-sam-altman-anticipates-superintelligence-soon-defends-ai-in-rare-personal-blog-post/)

- Sam Altman sees more great leaps forward for AI, highlighting potential benefits and challenges - [Google News](https://news.google.com/read/CBMic0FVX3lxTE1yZ3NNY05nLWdnRktoaFdKQUV2c1JIckFzWXo2MGF4dUtyd2l4c3NaR1prQ2NPUmdMM29rbVlTcTBsMVVxa0JmU3BiZjdHaVNjZl9ESWh6bHhJZTdTSGlNWVNXUF9SMV9LdXJTckV3ZmloQU0)

# Coding Companions or Competitors?

- Will AI replace programmers? Google's CEO says it's unlikely, as AI will enhance but not eliminate coding jobs - [Reddit](https://www.reddit.com/r/technology/comments/1fnqfbk/will_ai_replace_programmers_dont_count_on_it_says/)

# The Cloudflare Conundrum: Battling Bot Scrapers

- Cloudflare introduces tools to detect and block AI bots scraping website data for free - [Wired](https://www.wired.com/story/cloudflare-tools-detect-block-ai-bots/)

- Cloudflare tightens control over site-gobbling AI bots with new AI Audit control panel - [The Register](https://www.theregister.com/2024/09/24/cloudflare_ai_audit/)

"""


In [ ]:
REWRITE_PROMPT = """You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging AI trends, and the ability to
produce, edit, and curate high-quality content that engages and informs readers. You are
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness,
and coherence, and ensuring its accuracy and relevance.

Objective: The markdown newsletter provided below contains several sections consisting of bullet points.
Carefully review each section of the newsletter. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'OK' if no changes
are needed.

Instructions:
For each section, remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible, sticking to facts without editorial comment.
For each section, review the title. The title should be snappy, punchy, clever, possibly alliterative or punny.
The section title most be short, engaging, clever, and as consistent with the bullets in the section as possible
At the top of the newsletter add an overall title synthesizing several top news items.
Respond with the updated newsletter only in markdown format, or the word 'OK' if no changes are needed.

Newsletter to edit:
{summary}

"""

In [ ]:
def fn_rewrite_summary(state: AgentState) -> AgentState:

    model = ChatOpenAI(
        model=HIGHCOST_MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template(REWRITE_PROMPT) | model | SimpleJsonOutputParser()
    response = chain.invoke({ "summary": state["summary"]})
    print(response)
    state["summary"] = response.content
    return state


if DEBUG:
    _ = fn_rewrite_summary(test_state)


In [ ]:
rewrite_prompt = (REWRITE_PROMPT.format(summary=test_state["summary"]))
print(rewrite_prompt)


In [ ]:
test_state["summmary"] = """
# AI Revolutionizes Cybersecurity

- AI's Growing Role in Cybersecurity - [Hacker Noon](https://hackernoon.com/ais-growing-role-in-cybersecurity)
- Businesses Need Comprehensive Cybersecurity Strategies - [Hacker Noon](https://hackernoon.com/how-to-prepare-your-business-for-growing-cyberthreats)
- EC-Council Introduces AI-Powered Ethical Hacking Program - [Hacker Noon](https://hackernoon.com/ec-council-introduces-ai-powered-ethical-hacking-against-cybercrime)
- Intezer Raises $33M for Autonomous Security Operations - [Calcalistech](https://www.calcalistech.com/ctechnews/article/skyis6spc)

# Global AI Startup Funding Soars

- Mercor Valued at $250M After $32M Funding - [Forbes](https://www.forbes.com/sites/alexkonrad/2024/09/18/mercor-ai-interviewer-reaches-250-million-valuation/)
- Akur8 Raises $120M for AI in Insurance Pricing - [Fintech Global](https://fintech.global/2024/09/16/akur8-lands-120m-in-series-c-to-enhance-its-next-gen-actuarial-platform/)
- Fal.ai Secures $23M for AI Media Models - [TechCrunch](https://techcrunch.com/2024/09/18/fal-ai-which-hosts-media-generating-ai-models-raises-23m-from-a16z-and-others/)
- Rep.ai Raises $7.5M for Digital Twin Sales Reps - [VentureBeat](https://venturebeat.com/ai/ai-startup-rep-ai-raises-7-5m-to-launch-digital-twin-sales-representatives/)
- Middle Eastern Funds Invest Billions in AI Startups - [Google News](https://news.google.com/...)

# AI Transforms Video Production

- YouTube Launches AI-Powered Inspiration Tab - [The Verge](https://www.theverge.com/2024/9/18/24247559/youtube-ai-videos-veo-inspiration-tab)
- Luma and Runway Release New AI Video APIs - [VentureBeat](https://venturebeat.com/ai/ai-video-rivalry-intensifies-as-luma-announces-dream-machine-api-hours-after-runway/)
- Amazon Debuts AI Video Generator for Ads - [TechCrunch](https://techcrunch.com/2024/09/19/amazon-releases-a-video-generator-but-only-for-ads/)
- Lionsgate Partners with Runway for AI Filmmaking - [VentureBeat](https://venturebeat.com/ai/runway-inks-deal-with-lionsgate-in-first-team-up-for-ai-provider-and-major-movie-studio/), [The Verge](https://www.theverge.com/2024/9/18/24248115/lionsgate-runway-ai-deal)

# Apple's Accelerating AI Evolution

- Apple Releases iOS 18.1 Beta with New AI Features - [MacRumors](https://www.macrumors.com/2024/09/19/apple-seeds-first-ios-18-1-public-beta/), [The Verge](https://www.theverge.com/2024/9/19/24249206/apple-intelligence-ios-18-1-public-beta)
- Apple Introduces UI-JEPA Models for On-Device AI - [VentureBeat](https://venturebeat.com/ai/apple-aims-for-on-device-user-intent-understanding-with-ui-jepa-models/)
- Apple's 'Intelligence' Features Expand to More Languages, Some Delayed to 2025 - [The Verge](https://www.theverge.com/2024/9/18/24247839/apple-intelligence-language-support-german-italian), [Business Insider](https://www.businessinsider.com/apple-intelligence-features-rollout-timeline-iphone-16-2024-9)
- Analysts Predict Apple Intelligence Will Boost iPhone 16 Sales - [Business Insider](https://www.businessinsider.com/apple-intelligence-will-drive-sales-for-iphone-16-analysts-say-2024-9)
- Apple Encourages AI Use Without Hardware Upgrades - [The New York Times](https://www.nytimes.com/2024/09/09/technology/personaltech/iphone-ai-upgrade.html)

# Robotaxis Gear Up for Expansion

- Cruise Plans Robotaxi Return in SF Amid Safety Reviews - [The Verge](https://www.theverge.com/2024/9/19/24249150/cruise-robotaxi-sf-return-manual-drive-autonomous)
- Waymo Discusses Robotaxi Partnership with Hyundai - [The Verge](https://www.theverge.com/2024/9/19/24249093/waymo-in-talks-with-hyundai-about-future-robotaxi-partnership)
- Waymo and Uber Expand Robotaxi Services to Austin and Atlanta - [The Verge](https://www.theverge.com/2024/9/13/24243397/waymo-uber-austin-atlanta-robotaxi-partnership)

# Global Push for Unified AI Regulation

- UN Proposes Global AI Governance Panel and Urgent Regulation - [Reuters](https://www.reuters.com/technology/artificial-intelligence/un-advisory-body-makes-seven-recommendations-governing-ai-2024-09-19/), [Ars Technica](https://arstechnica.com/ai/2024/09/united-nations-wants-to-treat-ai-with-same-urgency-as-climate-change/)
- Brazil's Lula Advocates for Inclusive Global AI Rules at G-20 - [Bloomberg](https://www.bloomberg.com/news/articles/2024-09-22/lula-seeks-to-lead-push-for-global-ai-rules-during-brazil-s-g-20)

# Microsoft Supercharges Office with Copilot AI

- Microsoft Introduces Copilot Pages for Collaborative AI Workflows - [The Verge](https://www.theverge.com/2024/9/16/24246010/microsoft-copilot-pages-multiplayer-ai-business)
- Microsoft Adds Advanced Copilot AI Features to Office Suite - [The Verge](https://www.theverge.com/2024/9/16/24246014/microsoft-office-copilot-ai-features-excel-python-outlook-word-powerpoint)
- Salesforce CEO Calls Microsoft's Copilot a 'Clippy' Revival - [Business Insider](https://www.businessinsider.com/marc-benioff-salesforce-microsoft-copilot-clippy-2024-9)
- Microsoft Enables Custom App Launch via Copilot Key in Windows 11 - [The Verge](https://www.theverge.com/2024/9/20/24250067/microsoft-windows-11-copilot-key-customization-apps)

# LinkedIn's AI Data Practices Under Scrutiny

- LinkedIn Confirms AI Training on User Data, Updates Privacy Settings - [The Washington Post](https://www.washingtonpost.com/technology/2024/09/23/linkedin-training-ai-setting-opt-out/), [The Verge](https://www.theverge.com/2024/9/18/24248471/linkedin-ai-training-user-accounts-data-opt-in), [The Register](https://www.theregister.com/2024/09/19/linkedin_ai_data_access/), [404 Media](https://www.404media.co/linkedin-is-training-ai-on-user-data-before-updating-its-terms-of-service/)
- LinkedIn Halts AI Training on European and UK User Data - [TechRadar](https://www.techradar.com/pro/security/the-linkedin-ai-saga-shows-us-the-need-for-eu-like-privacy-regulations)

# Jony Ive and OpenAI Aim to Redefine Computing

- Jony Ive and OpenAI Collaborate on New AI Device, Seek $1B Funding - [MacRumors](https://www.macrumors.com/2024/09/23/jony-ive-working-on-new-device-openai/), [Google News](https://news.google.com/...)

# OpenAI's Advances and Funding Surge

- OpenAI's $6.5B Funding Round Oversubscribed as Investors Pile In - [Financial Times](https://ft.com/content/a8f9bd05-0999-415d-af86-c73e79ad5733), [Bloomberg](https://www.bloomberg.com/news/articles/2024-09-19/openai-to-decide-which-backers-to-let-into-6-5-billion-funding)
- OpenAI Releases o1 Models with Superior Reasoning but Ethical Concerns - [The Verge](https://www.theverge.com/2024/9/17/24243884/openai-o1-model-research-safety-alignment), [VentureBeat](https://venturebeat.com/programming-development/what-openais-new-o1-preview-and-o1-mini-models-mean-for-developers/), [Understanding AI](https://www.understandingai.org/p/openai-just-unleashed-an-alien-of)

# Microsoft Powers AI with Nuclear Energy

- Microsoft Signs 20-Year Deal to Reopen Three Mile Island Nuclear Plant for AI Data Centers - [The Verge](https://www.theverge.com/2024/9/20/24249770/microsoft-three-mile-island-nuclear-power-plant-deal-ai-data-centers), [Financial Times](https://www.ft.com/content/ddcb5ab6-965f-4034-96e1-7f668bad1801), [Ars Technica](https://arstechnica.com/ai/2024/09/re-opened-three-mile-island-will-power-ai-data-centers-under-new-deal/), [Business Insider](https://www.businessinsider.com/three-mile-island-nuclear-plant-reopens-power-microsoft-ai-push-2024-9)

# Salesforce's AI Ambitions

- Salesforce CEO Criticizes Microsoft's Copilot as Ineffective - [Business Insider](https://www.businessinsider.com/marc-benioff-salesforce-microsoft-copilot-clippy-2024-9)
- Salesforce Develops Agentforce AI Inspired by Steve Jobs - [VentureBeat](https://venturebeat.com/ai/salesforce-ceo-marc-benioff-reveals-steve-jobs-influence-on-agentforce-ai-strategy/)
- Salesforce and NVIDIA See Huge Potential in Agentic AI - [VentureBeat](https://venturebeat.com/ai/why-jensen-huang-and-marc-benioff-see-gigantic-opportunity-for-agentic-ai/)

# SiFive Launches RISC-V AI Chips

- SiFive Launches RISC-V Intelligence XM AI Chip Series for AI Workloads - [VentureBeat](https://venturebeat.com/ai/sifive-unveils-risc-v-chip-design-for-high-performance-ai-workloads/), [The Register](https://www.theregister.com/2024/09/19/sifive_ai_accelerator/), [SiliconANGLE](https://siliconangle.com/2024/09/18/risc-v-guardian-sifive-unveils-new-chip-designs-low-powered-ai-edge/)

# Google Enhances NotebookLM with AI Features

- Google Enhances NotebookLM with New AI Features for Enterprise and Creative Use - [Geeky Gadgets](https://www.geeky-gadgets.com/notes-into-podcasts-with-notebooklm/), [Understanding AI](https://www.understandingai.org/p/openai-just-unleashed-an-alien-of), [Google News](https://news.google.com/...)

# YouTube Enhances Creativity with AI

- YouTube Launches AI-Powered Inspiration Tab for Video Ideas - [The Verge](https://www.theverge.com/2024/9/18/24247559/youtube-ai-videos-veo-inspiration-tab)
- YouTube Integrates Veo AI into Shorts, Adds Multilingual Dubbing - [TechCrunch](https://techcrunch.com/2024/09/18/youtube-shorts-to-integrate-veo-google-ai-video-model/)
"""

In [ ]:
def fn_is_revision_complete(state: AgentState) -> str:
    """update edit_complete if MAX_EDITS exceeded"
    return "complete" if edit_complete else "incomplete"
    """

    if state["n_edits"] >= MAX_EDITS:
        log("Max edits reached")
        state["edit_complete"] = True


    return "complete" if state["edit_complete"] else "incomplete"


if DEBUG:
    fn_is_revision_complete(test_state)


In [ ]:
def fn_send_mail(state: AgentState) -> AgentState:

    log("Sending summary email")
    # Convert Markdown to HTML
    html_str = markdown.markdown(state['summary'], extensions=['extra'])
    # send email
    subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
    send_gmail(subject, html_str)
    return state


if DEBUG:
    fn_send_mail(test_state)

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        self.graph = graph
        graph.add_node("initialize", self.initialize)
        graph.add_node("download_sources", self.download_sources)
        graph.add_node("extract_urls", self.extract_urls)
        graph.add_node("filter_urls", self.filter_urls)
        graph.add_node("topic_analysis", self.topic_analysis)
        graph.add_node("topic_clusters", self.topic_clusters)
        graph.add_node("download_pages", self.download_pages)
        graph.add_node("summarize_pages", self.summarize_pages)
        graph.add_node("propose_topics", self.propose_topics)
        graph.add_node("compose_summary", self.compose_summary)
        graph.add_node("rewrite_summary", self.rewrite_summary)
        graph.add_node("send_mail", self.send_mail)

        graph.add_edge(START, "initialize")
        graph.add_edge("initialize", "download_sources")
        graph.add_edge("download_sources", "extract_urls")
        graph.add_edge("extract_urls", "filter_urls")
        graph.add_edge("filter_urls", "topic_analysis")
        graph.add_edge("topic_analysis", "topic_clusters")
        graph.add_edge("topic_clusters", "download_pages")
        graph.add_edge("download_pages", "summarize_pages")
        graph.add_edge("summarize_pages", "propose_topics")
        graph.add_edge("propose_topics", "compose_summary")
        graph.add_edge("compose_summary", "rewrite_summary")
        graph.add_conditional_edges("rewrite_summary",
                                    self.is_revision_complete,
                                    {"incomplete": "rewrite_summary",
                                     "complete": "send_mail",
                                    })
        graph.add_edge("send_mail", END)

        checkpointer = SqliteSaver.from_conn_string(":memory:")
        app = graph.compile(checkpointer=checkpointer, interrupt_before=["compose_summary",])
        self.app = app

#         self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def initialize(self, state: AgentState) -> AgentState:
        self.state = fn_initialize(state)
        return self.state

    def download_sources(self, state: AgentState) -> AgentState:
        self.state = fn_download_sources(state)
        return self.state

    def extract_urls(self, state: AgentState) -> AgentState:
        self.state = fn_extract_urls(state)
        return self.state

    def filter_urls(self, state: AgentState) -> AgentState:
        self.state = fn_filter_urls(state)
        return self.state

    def topic_analysis(self, state: AgentState) -> AgentState:
        self.state = fn_topic_analysis(state)
        return self.state

    def topic_clusters(self, state: AgentState) -> AgentState:
        self.state = fn_topic_clusters(state)
        return self.state

    def download_pages(self, state: AgentState) -> AgentState:
        self.state = fn_download_pages(state)
        return self.state

    def summarize_pages(self, state: AgentState) -> AgentState:
        self.state = fn_summarize_pages(state)
        return self.state

    def propose_topics(self, state: AgentState) -> AgentState:
        self.state = fn_propose_cats(state)
        return self.state

    def compose_summary(self, state: AgentState) -> AgentState:
        self.state = fn_compose_summary(state)
        return self.state

    def rewrite_summary(self, state: AgentState) -> AgentState:
        self.state = fn_rewrite_summary(state)
        return self.state

    def is_revision_complete(self, state: AgentState) -> str:
        return fn_is_revision_complete(state)

    def send_mail(self, state: AgentState) -> AgentState:
        self.state = fn_send_mail(state)
        return self.state

    def run(self, config, state):
        self.state = state
        for step in self.app.stream(state, config, stream_mode="debug"):
            if step["type"] == "checkpoint":
                display(f'Step {step["step"]}')

            # Check if there is an error message in the state
#             if step["payload"].get("values") and step["payload"]["values"].get("error_message"):
#                 # Update the error state to 'tested' since error simulation is complete
#                 self.state['error_state'] = 'tested'
#                 # Retry from the last checkpoint saved by the checkpointer
#                 for retry_step in graph.stream(self.state, config, stream_mode="debug"):
#                     if retry_step["type"] == "checkpoint":
#                         print(retry_step["step"], retry_step["payload"].get("values"))
#                 break
        return None


In [ ]:
model = ChatOpenAI(model=MODEL)
lg_agent = Agent(model, [], system="")


In [ ]:
# Image(lg_agent.graph.get_graph().draw_png())
Image(lg_agent.app.get_graph().draw_mermaid_png())


In [ ]:
# Configuration with thread ID for checkpointing
# Generate a random UUID
thread_id = uuid.uuid4()

config = {"configurable": {"thread_id": thread_id}}
# initial state
state = AgentState()
do_download = True
state["do_download"] = do_download
# before_date="2024-07-22 10:00:00"
# state["before_date"] = before_date
log(f"Starting with before_date={state.get('before_date')}, do_download={do_download}, thread_id={thread_id}")
lg_agent.run(config, state)


In [ ]:
user_approval = input("Edit topics? (yes/no): ")


In [ ]:
if user_approval.lower() == "yes":
    # If approved, continue the graph execution
    for step in lg_agent.graph.stream(None, config, stream_mode="debug"):
        if step["type"] == "checkpoint":
            display(f'Step {step["step"]}')
else:
    print("Operation cancelled by user.")


In [ ]:
# get last state
last_state = lg_agent.graph.get_state(config)
pd.DataFrame(last_state.values['AIdf'])


In [ ]:
# update state as node
# lg_agent.graph.update_state(config, {"foo": 2, "bar": ["b"]}, as_node="revise_summary")


In [ ]:
# resume from error starting at a node
# result = lg_agent.graph.invoke(None, {'configurable': {'thread_id': 'newsbot-thread-5'}, "next": "topic_analysis"})